In [2]:
import numpy as np
import pandas as pd
import statistics as ss
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchmetrics import MeanSquaredLogError as msle
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics 
from torch.utils.data import Dataset, DataLoader
import gc

In [4]:
CSV_FILE='data/survival_data.csv'
device= torch.device('cuda')
EPOCH = 200
BATCH_SIZE=10
MODEL_PATH = 'log/baseline/regression_pytorch/'
LEARNING_RATE = 0.01


In [5]:
# Load training cvs data into the program.
train=pd.read_csv("data/survival_data.csv")

# Perform feature reduction to discard any irrelevent data feature, namely 'ResectionStatus'.
#We proceed by discarding columns mentioned above from the dataframe.
train=train.drop(['ResectionStatus'],axis=1)

# drop rows contain NA value from the data 
train = train.dropna(subset=['Survival'])
X= train[['Age']].astype('float')
X=torch.tensor(X.values)
Y= train['Survival'].astype('float')
Y=torch.tensor(Y.values)

In [6]:
class data(Dataset):
    def __init__(self,train):
        self.source = train
        self.source.dropna(subset=['Survival','Age'])

        self.X = self.source[['Age']].astype('float')
        self.X = torch.tensor(self.X.values)
        
        self.Y = self.source['Survival'].astype('float')
        self.Y = torch.tensor(self.Y.values)
       
    def __len__(self):
        return len(self.source)
    def __getitem__(self,idx):
        x = self.X[idx]
        
        y= self.Y[idx]
       
        return x,y
def getdata(t,batch_size=BATCH_SIZE,shuffle=True,pin_memory=True):
    da=data(t)
    leng = [int(0.7* len(da)),len(da) - int(0.7 * len(da))]
    train,eva = torch.utils.data.random_split(da,leng)
    train=DataLoader(train,batch_size=batch_size,shuffle=shuffle,pin_memory=pin_memory)
    eva=DataLoader(eva,batch_size=batch_size,shuffle=shuffle,pin_memory=pin_memory)
    return train, eva

In [43]:
class model(nn.Module):
    def blk(self):
        block = nn.Sequential(
            nn.Linear(1,8),
            nn.LeakyReLU(),
            nn.Linear(8,32),
            nn.LeakyReLU(),
            nn.Linear(32,32),
            nn.LeakyReLU(),
            nn.Linear(32,8),
            nn.LeakyReLU(),
            nn.Linear(8,1),
        )
        return block
    
    def __init__(self):
        super(model, self).__init__()
        self.b = self.blk()
    def forward(self,x):
        return self.b(x)
    

In [44]:
def training(data,model,opt):
    print("Training in session: ")
    epoch_loss=0.0
    loss_fn = torch.nn.MSELoss().to(device)
    l = len(data.sampler)
    data=tqdm(data)
    model.train()
    for index, batch in enumerate(data):

        # reshape the data and load the data to gpu
        x, y= batch 
        x=x.float().to(device)
        y=y.float().to(device)
        # training model
        res = model(x)
        res=res.to(device)
        a,b = res.shape
        res=res.reshape(a*b)
        loss = loss_fn(res,y).to(device)
        opt.zero_grad()
        
        loss.backward()
        
        opt.step()
        
     
        
        print("Batch_id: ", index,"Training loss: ", loss.item())
        epoch_loss+=loss.item()

    return epoch_loss/(l/BATCH_SIZE)

In [45]:
def eva(data,model,opt):
    print("Eval in session: ")
    epoch_loss=0.0
    loss_fn = torch.nn.MSELoss().to(device)
    l = len(data.sampler)
    data=tqdm(data)
    model.eval()

    with torch.no_grad():
        for index, batch in enumerate(data):

            # reshape the data and load the data to gpu
            x, y= batch 
            x=x.float().to(device)
            y=y.float().to(device)
            
            
        
            res = model(x)
            a,b = res.shape
            res=res.reshape(a*b)
            loss = loss_fn(res,y).to(device)
           
            print("Batch_id: ", index,"Eval loss: ", loss.item())
            epoch_loss+=loss.item()

    return epoch_loss/(l/BATCH_SIZE)
    

In [46]:
def main(train):
    train_data,eval_data = getdata(train)
    BEST_VAL=10000000
    m = model().to(device)
    opt = optim.Adam(m.parameters(), lr=LEARNING_RATE)
    tra=[]
    ev = []
    index=[]
    ind =0
    sc = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,patience=10,factor = 0.5,threshold=0.1,mode = 'min',verbose = True)
    for i in range(0,EPOCH):
        tl = training(train_data,m,opt)
        el= eva(eval_data,m,opt)
        sc.step(el)
        tra.append(tl)
  
        ev.append(el)
        index.append(i)
        print('Epoch ',i,'Training loss: ',tl)
        print('Epoch ',i,'Evaluation loss: ',el)

        if BEST_VAL> el and i > 20:
            torch.save({
            'model_state_dict': m.state_dict(),
            'optim_state_dict': opt.state_dict(),
            'epoch': i,
            'train loss': tl,
            'eval loss': el,
            },MODEL_PATH+'best')
            print("Best Model saved!")
            print("Best Loss value: ", BEST_VAL)
            BEST_VAL=el
            ind = i
    plt.plot(index, tra, label ="Training ")
    plt.plot(index,ev, label = "Validation")
    plt.title("Training and Validation Loss Curve: ")
    plt.xlabel("Epoch:")
    plt.ylabel("Average Loss ")
    plt.legend()
    plt.savefig(MODEL_PATH+ 'Training and Eval Loss plot.png')
    plt.show()
    
   
        
    print("Best epoch: ",ind," and loss value is ", BEST_VAL)    
        

In [47]:
gc.collect()
torch.cuda.empty_cache()

Training in session: 


100%|██████████| 15/15 [00:00<00:00, 399.96it/s]


Batch_id:  0 Training loss:  644142.125
Batch_id:  1 Training loss:  267130.59375
Batch_id:  2 Training loss:  64937.1015625
Batch_id:  3 Training loss:  296809.0625
Batch_id:  4 Training loss:  107240.9296875
Batch_id:  5 Training loss:  412709.125
Batch_id:  6 Training loss:  295812.09375
Batch_id:  7 Training loss:  314444.9375
Batch_id:  8 Training loss:  337692.84375
Batch_id:  9 Training loss:  235595.90625
Batch_id:  10 Training loss:  77859.3125
Batch_id:  11 Training loss:  187802.234375
Batch_id:  12 Training loss:  210967.859375
Batch_id:  13 Training loss:  307392.21875
Batch_id:  14 Training loss:  236979.828125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1719.68it/s]


Batch_id:  0 Eval loss:  191634.515625
Batch_id:  1 Eval loss:  65004.4140625
Batch_id:  2 Eval loss:  118402.1015625
Batch_id:  3 Eval loss:  179754.53125
Batch_id:  4 Eval loss:  157323.609375
Batch_id:  5 Eval loss:  243492.03125
Batch_id:  6 Eval loss:  57879.60546875
Epoch  0 Training loss:  270102.4440456081
Epoch  0 Evaluation loss:  158357.93884277344
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 364.37it/s]


Batch_id:  0 Training loss:  201093.484375
Batch_id:  1 Training loss:  216583.75
Batch_id:  2 Training loss:  218443.703125
Batch_id:  3 Training loss:  211517.453125
Batch_id:  4 Training loss:  322229.0625
Batch_id:  5 Training loss:  65618.8984375
Batch_id:  6 Training loss:  343599.21875
Batch_id:  7 Training loss:  73770.3046875
Batch_id:  8 Training loss:  226682.65625
Batch_id:  9 Training loss:  167925.765625
Batch_id:  10 Training loss:  114100.1796875
Batch_id:  11 Training loss:  49156.98046875
Batch_id:  12 Training loss:  241986.421875
Batch_id:  13 Training loss:  389727.09375
Batch_id:  14 Training loss:  51487.390625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1365.52it/s]


Batch_id:  0 Eval loss:  61795.28125
Batch_id:  1 Eval loss:  111776.515625
Batch_id:  2 Eval loss:  273328.40625
Batch_id:  3 Eval loss:  129731.5
Batch_id:  4 Eval loss:  61932.48046875
Batch_id:  5 Eval loss:  147486.09375
Batch_id:  6 Eval loss:  52938.4765625
Epoch  1 Training loss:  195535.29481630068
Epoch  1 Evaluation loss:  131091.99279785156
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 338.73it/s]


Batch_id:  0 Training loss:  463566.21875
Batch_id:  1 Training loss:  356812.59375
Batch_id:  2 Training loss:  208646.484375
Batch_id:  3 Training loss:  302617.09375
Batch_id:  4 Training loss:  130385.3515625
Batch_id:  5 Training loss:  58601.92578125
Batch_id:  6 Training loss:  102444.75
Batch_id:  7 Training loss:  139520.671875
Batch_id:  8 Training loss:  444951.40625
Batch_id:  9 Training loss:  36820.48828125
Batch_id:  10 Training loss:  70421.9609375
Batch_id:  11 Training loss:  176268.671875
Batch_id:  12 Training loss:  36052.83203125
Batch_id:  13 Training loss:  166867.65625
Batch_id:  14 Training loss:  58711.3203125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1410.46it/s]


Batch_id:  0 Eval loss:  296438.65625
Batch_id:  1 Eval loss:  135066.859375
Batch_id:  2 Eval loss:  49998.38671875
Batch_id:  3 Eval loss:  60594.15234375
Batch_id:  4 Eval loss:  64141.52734375
Batch_id:  5 Eval loss:  80433.1015625
Batch_id:  6 Eval loss:  38073.078125
Epoch  2 Training loss:  185992.52876900337
Epoch  2 Evaluation loss:  113241.52526855469
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 375.27it/s]

Batch_id:  0 Training loss:  120569.0234375
Batch_id:  1 Training loss:  62831.93359375
Batch_id:  2 Training loss:  71661.2421875
Batch_id:  3 Training loss:  182988.171875
Batch_id:  4 Training loss:  176861.359375
Batch_id:  5 Training loss:  201009.28125
Batch_id:  6 Training loss:  295885.5
Batch_id:  7 Training loss:  362424.5625
Batch_id:  8 Training loss:  285553.78125
Batch_id:  9 Training loss:  455145.90625
Batch_id:  10 Training loss:  125389.25
Batch_id:  11 Training loss:  145721.578125
Batch_id:  12 Training loss:  45518.55859375
Batch_id:  13 Training loss:  74458.7109375
Batch_id:  14 Training loss:  74828.109375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1425.80it/s]


Batch_id:  0 Eval loss:  129939.6796875
Batch_id:  1 Eval loss:  34336.421875
Batch_id:  2 Eval loss:  42734.08984375
Batch_id:  3 Eval loss:  222981.484375
Batch_id:  4 Eval loss:  184107.953125
Batch_id:  5 Eval loss:  54367.3515625
Batch_id:  6 Eval loss:  108224.15625
Epoch  3 Training loss:  181138.30869932432
Epoch  3 Evaluation loss:  121357.99011230469
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 362.06it/s]

Batch_id:  0 Training loss:  111742.6875
Batch_id:  1 Training loss:  107246.4140625
Batch_id:  2 Training loss:  238468.40625
Batch_id:  3 Training loss:  305800.21875
Batch_id:  4 Training loss:  124475.8046875
Batch_id:  5 Training loss:  161748.578125
Batch_id:  6 Training loss:  45941.30859375
Batch_id:  7 Training loss:  154564.421875
Batch_id:  8 Training loss:  307929.03125
Batch_id:  9 Training loss:  96383.078125
Batch_id:  10 Training loss:  274544.21875
Batch_id:  11 Training loss:  81978.7734375
Batch_id:  12 Training loss:  57086.79296875
Batch_id:  13 Training loss:  204449.609375
Batch_id:  14 Training loss:  405666.3125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 779.34it/s]


Batch_id:  0 Eval loss:  91791.6953125
Batch_id:  1 Eval loss:  214394.65625
Batch_id:  2 Eval loss:  82727.0703125
Batch_id:  3 Eval loss:  129423.625
Batch_id:  4 Eval loss:  47824.30859375
Batch_id:  5 Eval loss:  37601.765625
Batch_id:  6 Eval loss:  243655.25
Epoch  4 Training loss:  180947.67947635133
Epoch  4 Evaluation loss:  132409.12048339844
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  45874.4453125
Batch_id:  1 Training loss:  149625.21875
Batch_id:  2 Training loss:  78891.875
Batch_id:  3 Training loss:  139676.234375
Batch_id:  4 Training loss:  244846.828125
Batch_id:  5 Training loss:  88575.8984375
Batch_id:  6 Training loss:  136374.25
Batch_id:  7 Training loss:  336585.96875
Batch_id:  8 Training loss:  293869.375
Batch_id:  9 Training loss:  178389.375
Batch_id:  10 Training loss:  76309.109375
Batch_id:  11 Training loss:  213927.75
Batch_id:  12 Training loss:  176403.859375


100%|██████████| 15/15 [00:00<00:00, 379.27it/s]


Batch_id:  13 Training loss:  250437.53125
Batch_id:  14 Training loss:  214348.765625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1159.43it/s]


Batch_id:  0 Eval loss:  349083.84375
Batch_id:  1 Eval loss:  47069.90625
Batch_id:  2 Eval loss:  55467.6953125
Batch_id:  3 Eval loss:  56102.70703125
Batch_id:  4 Eval loss:  109828.6484375
Batch_id:  5 Eval loss:  83770.2109375
Batch_id:  6 Eval loss:  5609.11767578125
Epoch  5 Training loss:  177306.519214527
Epoch  5 Evaluation loss:  110458.14521789551
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 331.90it/s]

Batch_id:  0 Training loss:  207402.796875
Batch_id:  1 Training loss:  207881.875
Batch_id:  2 Training loss:  229387.375
Batch_id:  3 Training loss:  375985.0625
Batch_id:  4 Training loss:  163116.015625
Batch_id:  5 Training loss:  111147.125
Batch_id:  6 Training loss:  240972.78125
Batch_id:  7 Training loss:  304249.21875
Batch_id:  8 Training loss:  252126.421875
Batch_id:  9 Training loss:  157633.25
Batch_id:  10 Training loss:  45882.85546875
Batch_id:  11 Training loss:  69074.828125
Batch_id:  12 Training loss:  79534.9140625
Batch_id:  13 Training loss:  187573.796875
Batch_id:  14 Training loss:  71839.171875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1166.52it/s]


Batch_id:  0 Eval loss:  304873.5
Batch_id:  1 Eval loss:  52955.05859375
Batch_id:  2 Eval loss:  41096.06640625
Batch_id:  3 Eval loss:  89631.234375
Batch_id:  4 Eval loss:  121402.75
Batch_id:  5 Eval loss:  47126.35546875
Batch_id:  6 Eval loss:  292959.125
Epoch  6 Training loss:  182689.6951541385
Epoch  6 Evaluation loss:  148444.38903808594
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  175647.65625
Batch_id:  1 Training loss:  48672.44921875
Batch_id:  2 Training loss:  72005.75
Batch_id:  3 Training loss:  259834.03125
Batch_id:  4 Training loss:  236969.796875
Batch_id:  5 Training loss:  48212.14453125
Batch_id:  6 Training loss:  269712.96875
Batch_id:  7 Training loss:  154231.671875
Batch_id:  8 Training loss:  202671.953125
Batch_id:  9 Training loss:  329383.5625
Batch_id:  10 Training loss:  75782.4140625
Batch_id:  11 Training loss:  362767.3125


100%|██████████| 15/15 [00:00<00:00, 315.98it/s]


Batch_id:  12 Training loss:  117598.4375
Batch_id:  13 Training loss:  208906.421875
Batch_id:  14 Training loss:  150478.515625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1393.99it/s]


Batch_id:  0 Eval loss:  126949.9765625
Batch_id:  1 Eval loss:  69708.4296875
Batch_id:  2 Eval loss:  117229.953125
Batch_id:  3 Eval loss:  41647.10546875
Batch_id:  4 Eval loss:  224332.203125
Batch_id:  5 Eval loss:  124333.6796875
Batch_id:  6 Eval loss:  68261.234375
Epoch  7 Training loss:  183302.3706714527
Epoch  7 Evaluation loss:  120697.27844238281
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  97159.59375
Batch_id:  1 Training loss:  92601.9921875
Batch_id:  2 Training loss:  206813.671875
Batch_id:  3 Training loss:  203873.109375
Batch_id:  4 Training loss:  403667.09375
Batch_id:  5 Training loss:  231634.40625
Batch_id:  6 Training loss:  62557.73046875
Batch_id:  7 Training loss:  259075.859375
Batch_id:  8 Training loss:  139136.859375
Batch_id:  9 Training loss:  290352.78125
Batch_id:  10 Training loss:  84504.8203125
Batch_id:  11 Training loss:  123543.4296875
Batch_id:  12 Training loss:  37288.31640625
Batch_id: 

100%|██████████| 15/15 [00:00<00:00, 326.11it/s]


 13 Training loss:  227951.046875
Batch_id:  14 Training loss:  211221.109375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 991.56it/s]

Batch_id:  0 Eval loss:  236395.296875
Batch_id:  1 Eval loss:  30151.248046875
Batch_id:  2 Eval loss:  75085.3671875
Batch_id:  3 Eval loss:  139610.96875
Batch_id:  4 Eval loss:  40682.15625
Batch_id:  5 Eval loss:  65439.80078125
Batch_id:  6 Eval loss:  282193.5


Epoch  8 Training loss:  180498.77164273648
Epoch  8 Evaluation loss:  135868.49029541016
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  253127.65625
Batch_id:  1 Training loss:  63666.9765625
Batch_id:  2 Training loss:  477375.90625
Batch_id:  3 Training loss:  44856.9375
Batch_id:  4 Training loss:  158909.171875
Batch_id:  5 Training loss:  117717.8984375
Batch_id:  6 Training loss:  143479.75
Batch_id:  7 Training loss:  61944.55859375
Batch_id:  8 Training loss:  317811.59375
Batch_id:  9 Training loss:  207742.421875
Batch_id:  10 Training loss:  254894.546875
Batch_id:  11 Training loss:  130485.9921875
Batch_id:  12 Training loss:  89550.640625
Batch_id:  13 Training loss:  116615.3125


100%|██████████| 15/15 [00:00<00:00, 174.41it/s]


Batch_id:  14 Training loss:  182018.609375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 409.05it/s]

Batch_id:  0 Eval loss:  131395.828125
Batch_id:  1 Eval loss:  85120.4609375
Batch_id:  2 Eval loss:  44351.30078125
Batch_id:  3 Eval loss:  105116.3515625
Batch_id:  4 Eval loss:  44674.63671875
Batch_id:  5 Eval loss:  206117.53125
Batch_id:  6 Eval loss:  203366.84375


Epoch  9 Training loss:  177040.40355785473
Epoch  9 Evaluation loss:  128147.33642578125
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  167764.0
Batch_id:  1 Training loss:  58857.13671875
Batch_id:  2 Training loss:  197822.125
Batch_id:  3 Training loss:  173854.109375
Batch_id:  4 Training loss:  494500.46875
Batch_id:  5 Training loss:  237555.359375
Batch_id:  6 Training loss:  173294.8125
Batch_id:  7 Training loss:  131110.5
Batch_id:  8 Training loss:  171692.515625
Batch_id:  9 Training loss:  183347.125
Batch_id:  10 Training loss:  219488.046875
Batch_id:  11 Training loss:  107688.6484375
Batch_id:  12 Training loss:  52026.2578125
Batch_id:  13 Training loss:  63178.12109375


100%|██████████| 15/15 [00:00<00:00, 283.20it/s]


Batch_id:  14 Training loss:  218328.6875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1407.42it/s]


Batch_id:  0 Eval loss:  157976.375
Batch_id:  1 Eval loss:  234497.359375
Batch_id:  2 Eval loss:  91290.2421875
Batch_id:  3 Eval loss:  32137.537109375
Batch_id:  4 Eval loss:  65249.08984375
Batch_id:  5 Eval loss:  109536.8671875
Batch_id:  6 Eval loss:  69746.8515625
Epoch  10 Training loss:  179088.37257179053
Epoch  10 Evaluation loss:  118817.8628540039
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  157649.890625
Batch_id:  1 Training loss:  41104.32421875
Batch_id:  2 Training loss:  146286.03125
Batch_id:  3 Training loss:  486983.46875
Batch_id:  4 Training loss:  198210.796875
Batch_id:  5 Training loss:  48506.54296875
Batch_id:  6 Training loss:  300226.9375
Batch_id:  7 Training loss:  71838.0625
Batch_id:  8 Training loss:  58221.11328125
Batch_id:  9 Training loss:  169036.484375
Batch_id:  10 Training loss:  229395.484375
Batch_id:  11 Training loss:  184256.78125
Batch_id:  12 Training loss:  368766.1875


100%|██████████| 15/15 [00:00<00:00, 355.01it/s]


Batch_id:  13 Training loss:  94671.484375
Batch_id:  14 Training loss:  77426.4375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1475.16it/s]


Batch_id:  0 Eval loss:  116135.8984375
Batch_id:  1 Eval loss:  239835.453125
Batch_id:  2 Eval loss:  81171.2109375
Batch_id:  3 Eval loss:  57907.11328125
Batch_id:  4 Eval loss:  142673.09375
Batch_id:  5 Eval loss:  53068.66796875
Batch_id:  6 Eval loss:  17553.59765625
Epoch  11 Training loss:  177877.0288745777
Epoch  11 Evaluation loss:  110678.91174316406
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 391.36it/s]

Batch_id:  0 Training loss:  34225.15234375
Batch_id:  1 Training loss:  137275.59375
Batch_id:  2 Training loss:  210842.375
Batch_id:  3 Training loss:  98241.140625
Batch_id:  4 Training loss:  132725.796875
Batch_id:  5 Training loss:  249603.703125
Batch_id:  6 Training loss:  254829.203125
Batch_id:  7 Training loss:  91809.6015625
Batch_id:  8 Training loss:  81576.2109375
Batch_id:  9 Training loss:  59096.52734375
Batch_id:  10 Training loss:  230463.671875
Batch_id:  11 Training loss:  174711.625
Batch_id:  12 Training loss:  211948.875
Batch_id:  13 Training loss:  444242.40625
Batch_id:  14 Training loss:  233297.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1154.96it/s]


Batch_id:  0 Eval loss:  143181.03125
Batch_id:  1 Eval loss:  47538.81640625
Batch_id:  2 Eval loss:  324710.15625
Batch_id:  3 Eval loss:  55423.7265625
Batch_id:  4 Eval loss:  40698.05859375
Batch_id:  5 Eval loss:  53326.5625
Batch_id:  6 Eval loss:  83584.265625
Epoch  12 Training loss:  178708.7674197635
Epoch  12 Evaluation loss:  116947.28393554688
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 383.10it/s]

Batch_id:  0 Training loss:  247039.734375
Batch_id:  1 Training loss:  196630.90625
Batch_id:  2 Training loss:  142066.90625
Batch_id:  3 Training loss:  218554.046875
Batch_id:  4 Training loss:  61192.21484375
Batch_id:  5 Training loss:  347457.5625
Batch_id:  6 Training loss:  203104.796875
Batch_id:  7 Training loss:  52996.88671875
Batch_id:  8 Training loss:  347168.25
Batch_id:  9 Training loss:  152786.953125
Batch_id:  10 Training loss:  45159.9765625
Batch_id:  11 Training loss:  181826.921875
Batch_id:  12 Training loss:  64335.07421875
Batch_id:  13 Training loss:  122611.9921875
Batch_id:  14 Training loss:  268994.6875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1763.58it/s]


Batch_id:  0 Eval loss:  59388.04296875
Batch_id:  1 Eval loss:  80780.0546875
Batch_id:  2 Eval loss:  143880.953125
Batch_id:  3 Eval loss:  47951.76953125
Batch_id:  4 Eval loss:  221160.078125
Batch_id:  5 Eval loss:  70119.71875
Batch_id:  6 Eval loss:  209621.5625
Epoch 00014: reducing learning rate of group 0 to 5.0000e-03.
Epoch  13 Training loss:  179184.2506862331
Epoch  13 Evaluation loss:  130140.96557617188
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 378.22it/s]

Batch_id:  0 Training loss:  80611.8671875
Batch_id:  1 Training loss:  171949.84375
Batch_id:  2 Training loss:  275287.28125
Batch_id:  3 Training loss:  25675.8984375
Batch_id:  4 Training loss:  256006.796875
Batch_id:  5 Training loss:  104620.671875
Batch_id:  6 Training loss:  94899.1875
Batch_id:  7 Training loss:  174017.828125
Batch_id:  8 Training loss:  185594.375
Batch_id:  9 Training loss:  53466.9765625
Batch_id:  10 Training loss:  71196.4375
Batch_id:  11 Training loss:  79296.2734375
Batch_id:  12 Training loss:  625033.625
Batch_id:  13 Training loss:  85259.0390625
Batch_id:  14 Training loss:  360531.59375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1428.65it/s]


Batch_id:  0 Eval loss:  251222.859375
Batch_id:  1 Eval loss:  23450.193359375
Batch_id:  2 Eval loss:  89372.7421875
Batch_id:  3 Eval loss:  58975.96484375
Batch_id:  4 Eval loss:  21706.357421875
Batch_id:  5 Eval loss:  232196.90625
Batch_id:  6 Eval loss:  45745.4609375
Epoch  14 Training loss:  178611.3307643581
Epoch  14 Evaluation loss:  112917.26318359375
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 366.23it/s]

Batch_id:  0 Training loss:  81896.7578125
Batch_id:  1 Training loss:  214568.203125
Batch_id:  2 Training loss:  168692.359375
Batch_id:  3 Training loss:  76101.203125
Batch_id:  4 Training loss:  116128.25
Batch_id:  5 Training loss:  79039.7578125
Batch_id:  6 Training loss:  373892.90625
Batch_id:  7 Training loss:  500263.15625
Batch_id:  8 Training loss:  86382.4609375
Batch_id:  9 Training loss:  311031.375
Batch_id:  10 Training loss:  127558.1796875
Batch_id:  11 Training loss:  102191.3125
Batch_id:  12 Training loss:  79043.375
Batch_id:  13 Training loss:  233466.078125
Batch_id:  14 Training loss:  64632.1328125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1399.63it/s]


Batch_id:  0 Eval loss:  91355.796875
Batch_id:  1 Eval loss:  155234.703125
Batch_id:  2 Eval loss:  236148.703125
Batch_id:  3 Eval loss:  137943.75
Batch_id:  4 Eval loss:  27869.443359375
Batch_id:  5 Eval loss:  39940.8515625
Batch_id:  6 Eval loss:  21752.53125
Epoch  15 Training loss:  176681.58836570944
Epoch  15 Evaluation loss:  110975.90301513672
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  154768.09375
Batch_id:  1 Training loss:  65772.1953125
Batch_id:  2 Training loss:  329866.96875
Batch_id:  3 Training loss:  109672.8984375
Batch_id:  4 Training loss:  146068.078125
Batch_id:  5 Training loss:  225530.703125
Batch_id:  6 Training loss:  379283.21875
Batch_id:  7 Training loss:  216400.578125
Batch_id:  8 Training loss:  52307.5390625
Batch_id:  9 Training loss:  34839.59375
Batch_id:  10 Training loss:  57132.07421875
Batch_id:  11 Training loss:  63290.40234375
Batch_id:  12 Training loss:  182465.40625


100%|██████████| 15/15 [00:00<00:00, 328.39it/s]


Batch_id:  13 Training loss:  518690.59375
Batch_id:  14 Training loss:  97158.84375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1372.55it/s]


Batch_id:  0 Eval loss:  81374.9453125
Batch_id:  1 Eval loss:  40878.9453125
Batch_id:  2 Eval loss:  76948.640625
Batch_id:  3 Eval loss:  234337.703125
Batch_id:  4 Eval loss:  27215.03125
Batch_id:  5 Eval loss:  230389.484375
Batch_id:  6 Eval loss:  25209.740234375
Epoch  16 Training loss:  177922.1072635135
Epoch  16 Evaluation loss:  111930.3890991211
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 351.50it/s]

Batch_id:  0 Training loss:  123909.3046875
Batch_id:  1 Training loss:  172414.796875
Batch_id:  2 Training loss:  121984.6484375
Batch_id:  3 Training loss:  174606.546875
Batch_id:  4 Training loss:  73269.015625
Batch_id:  5 Training loss:  393740.46875
Batch_id:  6 Training loss:  245220.421875
Batch_id:  7 Training loss:  228156.984375
Batch_id:  8 Training loss:  83312.109375
Batch_id:  9 Training loss:  173295.171875
Batch_id:  10 Training loss:  100673.546875
Batch_id:  11 Training loss:  275892.1875
Batch_id:  12 Training loss:  100126.5
Batch_id:  13 Training loss:  141883.828125
Batch_id:  14 Training loss:  168064.4375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1506.11it/s]


Batch_id:  0 Eval loss:  295618.09375
Batch_id:  1 Eval loss:  46323.97265625
Batch_id:  2 Eval loss:  35584.08203125
Batch_id:  3 Eval loss:  50199.4375
Batch_id:  4 Eval loss:  68165.3828125
Batch_id:  5 Eval loss:  137472.25
Batch_id:  6 Eval loss:  163346.9375
Epoch  17 Training loss:  174091.21410472973
Epoch  17 Evaluation loss:  124485.9619140625
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 384.07it/s]

Batch_id:  0 Training loss:  138016.78125
Batch_id:  1 Training loss:  217848.296875
Batch_id:  2 Training loss:  184663.046875
Batch_id:  3 Training loss:  157009.984375
Batch_id:  4 Training loss:  138069.953125
Batch_id:  5 Training loss:  174313.8125
Batch_id:  6 Training loss:  34415.25
Batch_id:  7 Training loss:  92665.9609375
Batch_id:  8 Training loss:  502198.46875
Batch_id:  9 Training loss:  259758.484375
Batch_id:  10 Training loss:  175927.296875
Batch_id:  11 Training loss:  193748.359375
Batch_id:  12 Training loss:  150375.578125
Batch_id:  13 Training loss:  110890.9296875
Batch_id:  14 Training loss:  36265.28125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 882.53it/s]


Batch_id:  0 Eval loss:  15916.9814453125
Batch_id:  1 Eval loss:  213916.578125
Batch_id:  2 Eval loss:  261712.453125
Batch_id:  3 Eval loss:  79061.6015625
Batch_id:  4 Eval loss:  35646.0234375
Batch_id:  5 Eval loss:  56016.69921875
Batch_id:  6 Eval loss:  81644.28125
Epoch  18 Training loss:  173389.6948902027
Epoch  18 Evaluation loss:  116236.65908813477
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  92261.703125
Batch_id:  1 Training loss:  111304.875
Batch_id:  2 Training loss:  95053.3984375
Batch_id:  3 Training loss:  180541.828125
Batch_id:  4 Training loss:  105859.2890625
Batch_id:  5 Training loss:  177606.109375
Batch_id:  6 Training loss:  172190.0
Batch_id:  7 Training loss:  91197.3046875
Batch_id:  8 Training loss:  53114.65625
Batch_id:  9 Training loss:  108375.0546875
Batch_id:  10 Training loss:  218673.03125
Batch_id:  11 Training loss:  560266.4375
Batch_id:  12 Training loss:  335807.21875
Batch_id:  13 Training loss:  44663.0859375


100%|██████████| 15/15 [00:00<00:00, 371.09it/s]


Batch_id:  14 Training loss:  260771.4375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1061.12it/s]


Batch_id:  0 Eval loss:  54552.27734375
Batch_id:  1 Eval loss:  33578.203125
Batch_id:  2 Eval loss:  45827.96875
Batch_id:  3 Eval loss:  338852.71875
Batch_id:  4 Eval loss:  48389.4921875
Batch_id:  5 Eval loss:  152097.15625
Batch_id:  6 Eval loss:  52414.5
Epoch  19 Training loss:  176194.96146537163
Epoch  19 Evaluation loss:  113392.54943847656
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  469281.84375
Batch_id:  1 Training loss:  68312.9375
Batch_id:  2 Training loss:  171406.140625
Batch_id:  3 Training loss:  123465.6796875
Batch_id:  4 Training loss:  247174.203125
Batch_id:  5 Training loss:  225180.203125
Batch_id:  6 Training loss:  195622.578125
Batch_id:  7 Training loss:  324098.90625
Batch_id:  8 Training loss:  79037.4375
Batch_id:  9 Training loss:  64458.3828125
Batch_id:  10 Training loss:  43348.90234375


100%|██████████| 15/15 [00:00<00:00, 326.32it/s]


Batch_id:  11 Training loss:  158197.453125
Batch_id:  12 Training loss:  55835.125
Batch_id:  13 Training loss:  269459.3125
Batch_id:  14 Training loss:  76476.9296875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1384.65it/s]


Batch_id:  0 Eval loss:  244651.953125
Batch_id:  1 Eval loss:  45664.66015625
Batch_id:  2 Eval loss:  113561.9140625
Batch_id:  3 Eval loss:  60802.875
Batch_id:  4 Eval loss:  70444.5234375
Batch_id:  5 Eval loss:  135165.046875
Batch_id:  6 Eval loss:  57985.3203125
Epoch  20 Training loss:  173740.27264569254
Epoch  20 Evaluation loss:  113793.17077636719
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 402.76it/s]

Batch_id:  0 Training loss:  39445.96484375
Batch_id:  1 Training loss:  207085.453125
Batch_id:  2 Training loss:  129413.4140625
Batch_id:  3 Training loss:  51253.71875
Batch_id:  4 Training loss:  195232.84375
Batch_id:  5 Training loss:  211023.28125
Batch_id:  6 Training loss:  48087.984375
Batch_id:  7 Training loss:  99827.78125
Batch_id:  8 Training loss:  206491.84375
Batch_id:  9 Training loss:  345187.125
Batch_id:  10 Training loss:  115753.375
Batch_id:  11 Training loss:  192820.078125
Batch_id:  12 Training loss:  66739.5234375
Batch_id:  13 Training loss:  181008.03125
Batch_id:  14 Training loss:  597791.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1332.31it/s]


Batch_id:  0 Eval loss:  79213.4921875
Batch_id:  1 Eval loss:  146141.640625
Batch_id:  2 Eval loss:  51068.52734375
Batch_id:  3 Eval loss:  28117.255859375
Batch_id:  4 Eval loss:  266813.5
Batch_id:  5 Eval loss:  41860.5703125
Batch_id:  6 Eval loss:  206506.96875
Epoch  21 Training loss:  181565.0197951858
Epoch  21 Evaluation loss:  128081.55548095703
Best Model saved!
Best Loss value:  10000000
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  40673.1640625
Batch_id:  1 Training loss:  192246.953125
Batch_id:  2 Training loss:  75639.328125
Batch_id:  3 Training loss:  131194.65625
Batch_id:  4 Training loss:  79892.2890625
Batch_id:  5 Training loss:  197186.90625
Batch_id:  6 Training loss:  217195.546875
Batch_id:  7 Training loss:  343477.875
Batch_id:  8 Training loss:  199601.671875
Batch_id:  9 Training loss:  130422.2265625
Batch_id:  10 Training loss:  399947.09375
Batch_id:  11 Training loss:  210830.703125
Batch_id:  12 Training loss:  61850.66796875
Batch_id:  13 Training loss:  209877.078125


100%|██████████| 15/15 [00:00<00:00, 337.75it/s]


Batch_id:  14 Training loss:  81086.96875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1731.45it/s]


Batch_id:  0 Eval loss:  93126.5390625
Batch_id:  1 Eval loss:  40734.71484375
Batch_id:  2 Eval loss:  45845.515625
Batch_id:  3 Eval loss:  42318.05859375
Batch_id:  4 Eval loss:  68315.0390625
Batch_id:  5 Eval loss:  200753.828125
Batch_id:  6 Eval loss:  548359.875
Epoch  22 Training loss:  173724.5357369088
Epoch  22 Evaluation loss:  162414.62036132812
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 426.79it/s]

Batch_id:  0 Training loss:  94051.0703125
Batch_id:  1 Training loss:  46993.2265625
Batch_id:  2 Training loss:  226066.296875
Batch_id:  3 Training loss:  325552.65625
Batch_id:  4 Training loss:  127850.015625
Batch_id:  5 Training loss:  244775.625
Batch_id:  6 Training loss:  113119.0
Batch_id:  7 Training loss:  49213.44921875
Batch_id:  8 Training loss:  84497.3515625
Batch_id:  9 Training loss:  192837.6875
Batch_id:  10 Training loss:  302043.21875
Batch_id:  11 Training loss:  55591.80078125
Batch_id:  12 Training loss:  273955.9375
Batch_id:  13 Training loss:  394011.15625
Batch_id:  14 Training loss:  45494.9765625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 766.58it/s]


Batch_id:  0 Eval loss:  77511.578125
Batch_id:  1 Eval loss:  43643.39453125
Batch_id:  2 Eval loss:  244751.609375
Batch_id:  3 Eval loss:  130818.5546875
Batch_id:  4 Eval loss:  30671.087890625
Batch_id:  5 Eval loss:  122803.4140625
Batch_id:  6 Eval loss:  113603.2421875
Epoch  23 Training loss:  174057.66680743243
Epoch  23 Evaluation loss:  119344.20013427734
Best Model saved!
Best Loss value:  128081.55548095703
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  274463.53125
Batch_id:  1 Training loss:  219028.125
Batch_id:  2 Training loss:  181351.109375
Batch_id:  3 Training loss:  211024.546875
Batch_id:  4 Training loss:  434737.5
Batch_id:  5 Training loss:  91310.6875
Batch_id:  6 Training loss:  117761.6484375
Batch_id:  7 Training loss:  132670.546875
Batch_id:  8 Training loss:  110184.6875
Batch_id:  9 Training loss:  66784.390625
Batch_id:  10 Training loss:  33239.26171875
Batch_id:  11 Training loss:  103042.375
Batch_id:  12 Training loss:  301911.5


100%|██████████| 15/15 [00:00<00:00, 393.84it/s]


Batch_id:  13 Training loss:  303289.4375
Batch_id:  14 Training loss:  39341.140625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1382.17it/s]


Batch_id:  0 Eval loss:  65623.03125
Batch_id:  1 Eval loss:  110383.5546875
Batch_id:  2 Eval loss:  44693.2265625
Batch_id:  3 Eval loss:  58475.8125
Batch_id:  4 Eval loss:  120803.0859375
Batch_id:  5 Eval loss:  261533.90625
Batch_id:  6 Eval loss:  78272.796875
Epoch 00025: reducing learning rate of group 0 to 2.5000e-03.
Epoch  24 Training loss:  177036.51947846284
Epoch  24 Evaluation loss:  115591.47094726562
Best Model saved!
Best Loss value:  119344.20013427734
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 403.62it/s]

Batch_id:  0 Training loss:  194468.203125
Batch_id:  1 Training loss:  53510.4375
Batch_id:  2 Training loss:  136079.890625
Batch_id:  3 Training loss:  92975.375
Batch_id:  4 Training loss:  326298.34375
Batch_id:  5 Training loss:  281520.75
Batch_id:  6 Training loss:  448927.75
Batch_id:  7 Training loss:  35508.54296875
Batch_id:  8 Training loss:  63307.13671875
Batch_id:  9 Training loss:  189914.4375
Batch_id:  10 Training loss:  298194.15625
Batch_id:  11 Training loss:  130054.625
Batch_id:  12 Training loss:  48520.88671875
Batch_id:  13 Training loss:  96729.8515625
Batch_id:  14 Training loss:  223581.953125
Eval in session: 



100%|██████████| 7/7 [00:00<00:00, 1168.33it/s]


Batch_id:  0 Eval loss:  34201.5078125
Batch_id:  1 Eval loss:  28661.375
Batch_id:  2 Eval loss:  91139.359375
Batch_id:  3 Eval loss:  23952.458984375
Batch_id:  4 Eval loss:  151258.25
Batch_id:  5 Eval loss:  165953.453125
Batch_id:  6 Eval loss:  498752.21875
Epoch  25 Training loss:  176999.482421875
Epoch  25 Evaluation loss:  155299.78485107422
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 399.87it/s]

Batch_id:  0 Training loss:  210723.609375
Batch_id:  1 Training loss:  60915.5
Batch_id:  2 Training loss:  302681.65625
Batch_id:  3 Training loss:  207149.78125
Batch_id:  4 Training loss:  82869.2734375
Batch_id:  5 Training loss:  82841.9765625
Batch_id:  6 Training loss:  104590.1171875
Batch_id:  7 Training loss:  184847.546875
Batch_id:  8 Training loss:  317772.53125
Batch_id:  9 Training loss:  67015.484375
Batch_id:  10 Training loss:  115193.2265625
Batch_id:  11 Training loss:  133768.046875
Batch_id:  12 Training loss:  198939.96875
Batch_id:  13 Training loss:  310097.96875
Batch_id:  14 Training loss:  217414.90625
Eval in session: 



100%|██████████| 7/7 [00:00<00:00, 1401.04it/s]


Batch_id:  0 Eval loss:  205520.65625
Batch_id:  1 Eval loss:  226168.609375
Batch_id:  2 Eval loss:  80708.078125
Batch_id:  3 Eval loss:  76008.8359375
Batch_id:  4 Eval loss:  48493.48828125
Batch_id:  5 Eval loss:  44477.390625
Batch_id:  6 Eval loss:  22664.79296875
Epoch  26 Training loss:  175460.91849662163
Epoch  26 Evaluation loss:  110006.53930664062
Best Model saved!
Best Loss value:  115591.47094726562
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 403.52it/s]

Batch_id:  0 Training loss:  62760.64453125
Batch_id:  1 Training loss:  181573.953125
Batch_id:  2 Training loss:  59755.27734375
Batch_id:  3 Training loss:  199210.484375
Batch_id:  4 Training loss:  52726.5625
Batch_id:  5 Training loss:  112334.8984375
Batch_id:  6 Training loss:  65556.546875
Batch_id:  7 Training loss:  109104.0234375
Batch_id:  8 Training loss:  192030.09375
Batch_id:  9 Training loss:  77846.078125
Batch_id:  10 Training loss:  268075.5
Batch_id:  11 Training loss:  353972.96875
Batch_id:  12 Training loss:  368050.21875
Batch_id:  13 Training loss:  288016.34375
Batch_id:  14 Training loss:  260155.234375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1261.93it/s]


Batch_id:  0 Eval loss:  228277.375
Batch_id:  1 Eval loss:  43677.0078125
Batch_id:  2 Eval loss:  75955.6640625
Batch_id:  3 Eval loss:  207189.234375
Batch_id:  4 Eval loss:  45534.63671875
Batch_id:  5 Eval loss:  70801.0
Batch_id:  6 Eval loss:  56777.2890625
Epoch  27 Training loss:  179133.02892736485
Epoch  27 Evaluation loss:  113783.15734863281
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 442.34it/s]

Batch_id:  0 Training loss:  44401.52734375
Batch_id:  1 Training loss:  128087.953125
Batch_id:  2 Training loss:  490096.0625
Batch_id:  3 Training loss:  31585.087890625
Batch_id:  4 Training loss:  451560.90625
Batch_id:  5 Training loss:  234809.171875
Batch_id:  6 Training loss:  73762.0546875
Batch_id:  7 Training loss:  113372.828125
Batch_id:  8 Training loss:  140686.796875
Batch_id:  9 Training loss:  83347.328125
Batch_id:  10 Training loss:  205695.453125
Batch_id:  11 Training loss:  177953.296875
Batch_id:  12 Training loss:  67838.9609375
Batch_id:  13 Training loss:  62370.07421875
Batch_id:  14 Training loss:  310508.375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1799.91it/s]


Batch_id:  0 Eval loss:  27315.044921875
Batch_id:  1 Eval loss:  206419.765625
Batch_id:  2 Eval loss:  181828.609375
Batch_id:  3 Eval loss:  122588.765625
Batch_id:  4 Eval loss:  34228.5859375
Batch_id:  5 Eval loss:  79517.3359375
Batch_id:  6 Eval loss:  108396.96875
Epoch  28 Training loss:  176761.88357791383
Epoch  28 Evaluation loss:  118796.10565185547
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 408.54it/s]


Batch_id:  0 Training loss:  380210.59375
Batch_id:  1 Training loss:  84524.671875
Batch_id:  2 Training loss:  174384.203125
Batch_id:  3 Training loss:  85460.7421875
Batch_id:  4 Training loss:  180478.109375
Batch_id:  5 Training loss:  55207.80078125
Batch_id:  6 Training loss:  104219.1171875
Batch_id:  7 Training loss:  144252.75
Batch_id:  8 Training loss:  66369.8671875
Batch_id:  9 Training loss:  289330.09375
Batch_id:  10 Training loss:  410189.4375
Batch_id:  11 Training loss:  91498.234375
Batch_id:  12 Training loss:  237723.078125
Batch_id:  13 Training loss:  192144.234375
Batch_id:  14 Training loss:  85660.140625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1802.23it/s]

Batch_id:  0 Eval loss:  152660.171875
Batch_id:  1 Eval loss:  144002.78125
Batch_id:  2 Eval loss:  239284.859375
Batch_id:  3 Eval loss:  42186.76171875
Batch_id:  4 Eval loss:  41862.39453125
Batch_id:  5 Eval loss:  65903.2109375
Batch_id:  6 Eval loss:  12186.494140625


Epoch  29 Training loss:  174436.0185282939
Epoch  29 Evaluation loss:  109076.04278564453
Best Model saved!
Best Loss value:  110006.53930664062
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 416.25it/s]


Batch_id:  0 Training loss:  116498.2421875
Batch_id:  1 Training loss:  181284.828125
Batch_id:  2 Training loss:  66037.9296875
Batch_id:  3 Training loss:  73104.859375
Batch_id:  4 Training loss:  189163.125
Batch_id:  5 Training loss:  539889.5625
Batch_id:  6 Training loss:  59539.32421875
Batch_id:  7 Training loss:  54023.9453125
Batch_id:  8 Training loss:  204619.5
Batch_id:  9 Training loss:  69682.4453125
Batch_id:  10 Training loss:  215668.609375
Batch_id:  11 Training loss:  306855.03125
Batch_id:  12 Training loss:  218208.0
Batch_id:  13 Training loss:  86252.828125
Batch_id:  14 Training loss:  181061.171875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1437.67it/s]


Batch_id:  0 Eval loss:  43814.59375
Batch_id:  1 Eval loss:  69278.2109375
Batch_id:  2 Eval loss:  324349.84375
Batch_id:  3 Eval loss:  98214.640625
Batch_id:  4 Eval loss:  41598.984375
Batch_id:  5 Eval loss:  78736.34375
Batch_id:  6 Eval loss:  80629.6640625
Epoch  30 Training loss:  173100.63529349663
Epoch  30 Evaluation loss:  115097.2314453125
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  264098.625
Batch_id:  1 Training loss:  238846.609375
Batch_id:  2 Training loss:  224596.609375
Batch_id:  3 Training loss:  115951.703125
Batch_id:  4 Training loss:  205208.015625
Batch_id:  5 Training loss:  92353.328125
Batch_id:  6 Training loss:  77886.765625
Batch_id:  7 Training loss:  191634.078125
Batch_id:  8 Training loss:  180093.28125
Batch_id:  9 Training loss:  369916.34375


100%|██████████| 15/15 [00:00<00:00, 198.50it/s]

Batch_id:  10 Training loss:  148720.0625
Batch_id:  11 Training loss:  132204.28125
Batch_id:  12 Training loss:  131637.5
Batch_id:  13 Training loss:  99076.6640625
Batch_id:  14 Training loss:  91409.2578125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1304.72it/s]


Batch_id:  0 Eval loss:  111898.703125
Batch_id:  1 Eval loss:  86142.640625
Batch_id:  2 Eval loss:  228123.671875
Batch_id:  3 Eval loss:  126838.6171875
Batch_id:  4 Eval loss:  56300.8515625
Batch_id:  5 Eval loss:  43428.21875
Batch_id:  6 Eval loss:  93765.90625
Epoch  31 Training loss:  173218.4543918919
Epoch  31 Evaluation loss:  116640.40771484375
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  65056.54296875


100%|██████████| 15/15 [00:00<00:00, 349.01it/s]

Batch_id:  1 Training loss:  68240.0859375
Batch_id:  2 Training loss:  39363.51953125
Batch_id:  3 Training loss:  45423.62109375
Batch_id:  4 Training loss:  419570.8125
Batch_id:  5 Training loss:  173168.765625
Batch_id:  6 Training loss:  290087.65625
Batch_id:  7 Training loss:  47032.75
Batch_id:  8 Training loss:  448658.59375
Batch_id:  9 Training loss:  107776.2109375
Batch_id:  10 Training loss:  199210.546875
Batch_id:  11 Training loss:  358539.0
Batch_id:  12 Training loss:  180291.40625
Batch_id:  13 Training loss:  48666.02734375
Batch_id:  14 Training loss:  97185.984375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1400.17it/s]

Batch_id:  0 Eval loss:  82527.3515625
Batch_id:  1 Eval loss:  137892.546875
Batch_id:  2 Eval loss:  46989.35546875
Batch_id:  3 Eval loss:  214343.609375
Batch_id:  4 Eval loss:  98262.3515625
Batch_id:  5 Eval loss:  98283.2109375
Batch_id:  6 Eval loss:  21503.09765625


Epoch  32 Training loss:  174883.21104307432
Epoch  32 Evaluation loss:  109343.98803710938
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 394.16it/s]


Batch_id:  0 Training loss:  128898.328125
Batch_id:  1 Training loss:  58029.04296875
Batch_id:  2 Training loss:  265448.0
Batch_id:  3 Training loss:  604137.625
Batch_id:  4 Training loss:  403655.21875
Batch_id:  5 Training loss:  23016.576171875
Batch_id:  6 Training loss:  183517.03125
Batch_id:  7 Training loss:  97240.59375
Batch_id:  8 Training loss:  103757.0078125
Batch_id:  9 Training loss:  191420.75
Batch_id:  10 Training loss:  83361.8984375
Batch_id:  11 Training loss:  172946.046875
Batch_id:  12 Training loss:  87697.828125
Batch_id:  13 Training loss:  38637.73046875
Batch_id:  14 Training loss:  126319.0625
Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  107063.3984375


100%|██████████| 7/7 [00:00<00:00, 997.46it/s]


Batch_id:  1 Eval loss:  135556.53125
Batch_id:  2 Eval loss:  208603.484375
Batch_id:  3 Eval loss:  38241.79296875
Batch_id:  4 Eval loss:  77325.5078125
Batch_id:  5 Eval loss:  86610.140625
Batch_id:  6 Eval loss:  82514.0859375
Epoch  33 Training loss:  173519.1040698902
Epoch  33 Evaluation loss:  114986.70959472656
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  289828.75
Batch_id:  1 Training loss:  32902.04296875
Batch_id:  2 Training loss:  327989.6875
Batch_id:  3 Training loss:  60476.76171875
Batch_id:  4 Training loss:  73288.3671875
Batch_id:  5 Training loss:  141501.046875
Batch_id:  6 Training loss:  244519.65625
Batch_id:  7 Training loss:  426980.0
Batch_id:  8 Training loss:  86727.0625
Batch_id:  9 Training loss:  254546.859375
Batch_id:  10 Training loss:  58524.68359375
Batch_id:  11 Training loss:  114887.3984375


100%|██████████| 15/15 [00:00<00:00, 348.37it/s]


Batch_id:  12 Training loss:  88395.9375
Batch_id:  13 Training loss:  192258.90625
Batch_id:  14 Training loss:  177203.6875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1399.43it/s]


Batch_id:  0 Eval loss:  114271.5
Batch_id:  1 Eval loss:  44776.83203125
Batch_id:  2 Eval loss:  42393.81640625
Batch_id:  3 Eval loss:  62073.4765625
Batch_id:  4 Eval loss:  308843.625
Batch_id:  5 Eval loss:  63499.9375
Batch_id:  6 Eval loss:  123258.4609375
Epoch  34 Training loss:  173650.73294974663
Epoch  34 Evaluation loss:  118612.13256835938
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  238374.609375
Batch_id:  1 Training loss:  337838.75
Batch_id:  2 Training loss:  110904.2109375
Batch_id:  3 Training loss:  187294.140625
Batch_id:  4 Training loss:  220993.0
Batch_id:  5 Training loss:  56143.01171875
Batch_id:  6 Training loss:  186260.34375
Batch_id:  7 Training loss:  74198.6484375
Batch_id:  8 Training loss:  213880.296875
Batch_id:  9 Training loss:  142042.515625
Batch_id:  10 Training loss:  75646.7890625
Batch_id:  11 Training loss:  87878.1171875
Batch_id:  12 Training loss:  327089.375
Batch_id:  13 Training loss:  67441.0


100%|██████████| 15/15 [00:00<00:00, 319.96it/s]


Batch_id:  14 Training loss:  250950.328125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1000.00it/s]


Batch_id:  0 Eval loss:  48018.1015625
Batch_id:  1 Eval loss:  77962.2890625
Batch_id:  2 Eval loss:  301212.1875
Batch_id:  3 Eval loss:  158170.796875
Batch_id:  4 Eval loss:  21553.265625
Batch_id:  5 Eval loss:  65736.21875
Batch_id:  6 Eval loss:  30319.927734375
Epoch 00036: reducing learning rate of group 0 to 1.2500e-03.
Epoch  35 Training loss:  174117.2389674831
Epoch  35 Evaluation loss:  109839.49798583984
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  47309.82421875
Batch_id:  1 Training loss:  146501.390625
Batch_id:  2 Training loss:  184151.109375
Batch_id:  3 Training loss:  89994.46875
Batch_id:  4 Training loss:  210175.671875
Batch_id:  5 Training loss:  245212.421875
Batch_id:  6 Training loss:  438756.8125
Batch_id:  7 Training loss:  96696.6875
Batch_id:  8 Training loss:  56244.67578125
Batch_id:  9 Training loss:  246367.03125
Batch_id:  10 Training loss:  200300.921875
Batch_id:  11 Training loss:  221940.546875


100%|██████████| 15/15 [00:00<00:00, 369.37it/s]


Batch_id:  12 Training loss:  173943.859375
Batch_id:  13 Training loss:  85804.875
Batch_id:  14 Training loss:  103169.2109375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1702.63it/s]


Batch_id:  0 Eval loss:  50183.5390625
Batch_id:  1 Eval loss:  72403.640625
Batch_id:  2 Eval loss:  69262.78125
Batch_id:  3 Eval loss:  91039.7109375
Batch_id:  4 Eval loss:  164008.609375
Batch_id:  5 Eval loss:  224864.90625
Batch_id:  6 Eval loss:  35155.90625
Epoch  36 Training loss:  172065.50728462837
Epoch  36 Evaluation loss:  110456.1083984375
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 336.15it/s]

Batch_id:  0 Training loss:  140369.34375
Batch_id:  1 Training loss:  439006.5
Batch_id:  2 Training loss:  79954.453125
Batch_id:  3 Training loss:  190191.421875
Batch_id:  4 Training loss:  267549.375
Batch_id:  5 Training loss:  138556.765625
Batch_id:  6 Training loss:  229037.28125
Batch_id:  7 Training loss:  160805.703125
Batch_id:  8 Training loss:  73141.6484375
Batch_id:  9 Training loss:  99357.2890625
Batch_id:  10 Training loss:  212759.40625
Batch_id:  11 Training loss:  148408.78125
Batch_id:  12 Training loss:  275602.34375
Batch_id:  13 Training loss:  31035.5625
Batch_id:  14 Training loss:  41005.1875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1396.97it/s]

Batch_id:  0 Eval loss:  176004.21875
Batch_id:  1 Eval loss:  63215.80078125
Batch_id:  2 Eval loss:  63091.1875
Batch_id:  3 Eval loss:  51112.1484375
Batch_id:  4 Eval loss:  44631.16796875
Batch_id:  5 Eval loss:  279040.6875
Batch_id:  6 Eval loss:  17600.791015625


Epoch  37 Training loss:  170728.4501689189
Epoch  37 Evaluation loss:  108546.25030517578
Best Model saved!
Best Loss value:  109076.04278564453
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  117752.4296875
Batch_id:  1 Training loss:  113771.765625
Batch_id:  2 Training loss:  273866.34375
Batch_id:  3 Training loss:  315710.0625
Batch_id:  4 Training loss:  181196.671875
Batch_id:  5 Training loss:  143577.171875
Batch_id:  6 Training loss:  166845.671875
Batch_id:  7 Training loss:  115802.0390625
Batch_id:  8 Training loss:  194579.203125
Batch_id:  9 Training loss:  51212.64453125
Batch_id:  10 Training loss:  67131.0546875
Batch_id:  11 Training loss:  213251.875


100%|██████████| 15/15 [00:00<00:00, 340.68it/s]


Batch_id:  12 Training loss:  358079.71875
Batch_id:  13 Training loss:  142295.21875
Batch_id:  14 Training loss:  78659.5
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1742.54it/s]


Batch_id:  0 Eval loss:  55510.30078125
Batch_id:  1 Eval loss:  47397.39453125
Batch_id:  2 Eval loss:  74019.765625
Batch_id:  3 Eval loss:  75160.9140625
Batch_id:  4 Eval loss:  127416.2109375
Batch_id:  5 Eval loss:  280465.40625
Batch_id:  6 Eval loss:  59618.09375
Epoch  38 Training loss:  171198.06561444257
Epoch  38 Evaluation loss:  112435.63842773438
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 392.85it/s]

Batch_id:  0 Training loss:  341653.40625
Batch_id:  1 Training loss:  182786.71875
Batch_id:  2 Training loss:  204592.65625
Batch_id:  3 Training loss:  61497.94921875
Batch_id:  4 Training loss:  249838.796875
Batch_id:  5 Training loss:  417522.71875
Batch_id:  6 Training loss:  124513.5
Batch_id:  7 Training loss:  282245.65625
Batch_id:  8 Training loss:  86931.890625
Batch_id:  9 Training loss:  76503.1796875
Batch_id:  10 Training loss:  49802.4453125
Batch_id:  11 Training loss:  132821.78125
Batch_id:  12 Training loss:  65287.23828125
Batch_id:  13 Training loss:  106327.4765625
Batch_id:  14 Training loss:  169424.5


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1740.07it/s]


Batch_id:  0 Eval loss:  142810.421875
Batch_id:  1 Eval loss:  98884.125
Batch_id:  2 Eval loss:  17835.650390625
Batch_id:  3 Eval loss:  36761.0078125
Batch_id:  4 Eval loss:  136631.046875
Batch_id:  5 Eval loss:  247613.859375
Batch_id:  6 Eval loss:  7627.02099609375
Epoch  39 Training loss:  172415.5347339527
Epoch  39 Evaluation loss:  107525.48942565918
Best Model saved!
Best Loss value:  108546.25030517578
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 428.37it/s]

Batch_id:  0 Training loss:  142451.453125
Batch_id:  1 Training loss:  68550.640625
Batch_id:  2 Training loss:  118445.4609375
Batch_id:  3 Training loss:  199131.984375
Batch_id:  4 Training loss:  107222.953125
Batch_id:  5 Training loss:  211762.875
Batch_id:  6 Training loss:  278116.78125
Batch_id:  7 Training loss:  135918.734375
Batch_id:  8 Training loss:  235524.65625
Batch_id:  9 Training loss:  82790.890625
Batch_id:  10 Training loss:  309209.40625
Batch_id:  11 Training loss:  427540.5
Batch_id:  12 Training loss:  95712.0546875
Batch_id:  13 Training loss:  51340.6484375
Batch_id:  14 Training loss:  66283.4140625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1754.41it/s]


Batch_id:  0 Eval loss:  48892.47265625
Batch_id:  1 Eval loss:  150395.796875
Batch_id:  2 Eval loss:  37808.6328125
Batch_id:  3 Eval loss:  138296.796875
Batch_id:  4 Eval loss:  231756.296875
Batch_id:  5 Eval loss:  59776.16796875
Batch_id:  6 Eval loss:  41089.8515625
Epoch  40 Training loss:  170946.11169763512
Epoch  40 Evaluation loss:  110627.50244140625
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 398.07it/s]

Batch_id:  0 Training loss:  68850.0703125
Batch_id:  1 Training loss:  107662.0
Batch_id:  2 Training loss:  147704.03125
Batch_id:  3 Training loss:  42411.60546875
Batch_id:  4 Training loss:  163158.484375
Batch_id:  5 Training loss:  126579.9140625
Batch_id:  6 Training loss:  501861.59375
Batch_id:  7 Training loss:  154481.28125
Batch_id:  8 Training loss:  92176.9140625
Batch_id:  9 Training loss:  159870.359375
Batch_id:  10 Training loss:  147656.953125
Batch_id:  11 Training loss:  75003.921875
Batch_id:  12 Training loss:  387887.90625
Batch_id:  13 Training loss:  283718.75
Batch_id:  14 Training loss:  68390.3125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1418.98it/s]


Batch_id:  0 Eval loss:  122587.8046875
Batch_id:  1 Eval loss:  21621.537109375
Batch_id:  2 Eval loss:  204076.484375
Batch_id:  3 Eval loss:  22652.89453125
Batch_id:  4 Eval loss:  195672.796875
Batch_id:  5 Eval loss:  85397.6484375
Batch_id:  6 Eval loss:  78035.203125
Epoch  41 Training loss:  170771.22281461148
Epoch  41 Evaluation loss:  114069.43267822266
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 311.68it/s]

Batch_id:  0 Training loss:  317032.0
Batch_id:  1 Training loss:  39102.90625
Batch_id:  2 Training loss:  122690.25
Batch_id:  3 Training loss:  74101.9140625
Batch_id:  4 Training loss:  284340.59375
Batch_id:  5 Training loss:  363709.84375
Batch_id:  6 Training loss:  154801.84375
Batch_id:  7 Training loss:  281747.9375
Batch_id:  8 Training loss:  181139.171875
Batch_id:  9 Training loss:  55327.66796875
Batch_id:  10 Training loss:  78982.71875
Batch_id:  11 Training loss:  275561.8125
Batch_id:  12 Training loss:  170057.484375
Batch_id:  13 Training loss:  61742.78125
Batch_id:  14 Training loss:  66232.3125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1412.77it/s]


Batch_id:  0 Eval loss:  126454.375
Batch_id:  1 Eval loss:  208877.859375
Batch_id:  2 Eval loss:  48245.9453125
Batch_id:  3 Eval loss:  81823.7265625
Batch_id:  4 Eval loss:  120372.0390625
Batch_id:  5 Eval loss:  93298.6875
Batch_id:  6 Eval loss:  11001.6484375
Epoch  42 Training loss:  170714.2728568412
Epoch  42 Evaluation loss:  107824.1064453125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 389.10it/s]

Batch_id:  0 Training loss:  209591.3125
Batch_id:  1 Training loss:  307156.875
Batch_id:  2 Training loss:  98648.8984375
Batch_id:  3 Training loss:  261633.5
Batch_id:  4 Training loss:  70342.765625
Batch_id:  5 Training loss:  262490.84375
Batch_id:  6 Training loss:  297889.34375
Batch_id:  7 Training loss:  132332.046875
Batch_id:  8 Training loss:  126446.453125
Batch_id:  9 Training loss:  67343.4140625
Batch_id:  10 Training loss:  193439.75
Batch_id:  11 Training loss:  51944.10546875
Batch_id:  12 Training loss:  107023.1875
Batch_id:  13 Training loss:  121848.2734375
Batch_id:  14 Training loss:  257391.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1180.59it/s]


Batch_id:  0 Eval loss:  303373.03125
Batch_id:  1 Eval loss:  35504.38671875
Batch_id:  2 Eval loss:  47192.19921875
Batch_id:  3 Eval loss:  68348.8125
Batch_id:  4 Eval loss:  43876.38671875
Batch_id:  5 Eval loss:  51284.52734375
Batch_id:  6 Eval loss:  332394.0
Epoch  43 Training loss:  173346.12463048985
Epoch  43 Evaluation loss:  137808.3349609375
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 323.78it/s]

Batch_id:  0 Training loss:  329528.25
Batch_id:  1 Training loss:  153461.984375
Batch_id:  2 Training loss:  97756.5546875
Batch_id:  3 Training loss:  220873.375
Batch_id:  4 Training loss:  105874.6875
Batch_id:  5 Training loss:  240966.046875
Batch_id:  6 Training loss:  96409.3125
Batch_id:  7 Training loss:  327014.21875
Batch_id:  8 Training loss:  102185.765625
Batch_id:  9 Training loss:  66729.2578125
Batch_id:  10 Training loss:  75795.9921875
Batch_id:  11 Training loss:  54405.3203125
Batch_id:  12 Training loss:  169942.359375
Batch_id:  13 Training loss:  226831.296875
Batch_id:  14 Training loss:  326350.6875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1794.30it/s]


Batch_id:  0 Eval loss:  145077.171875
Batch_id:  1 Eval loss:  294605.40625
Batch_id:  2 Eval loss:  33618.34375
Batch_id:  3 Eval loss:  101869.96875
Batch_id:  4 Eval loss:  49779.5703125
Batch_id:  5 Eval loss:  37182.01953125
Batch_id:  6 Eval loss:  50019.625
Epoch  44 Training loss:  175278.7236064189
Epoch  44 Evaluation loss:  111273.76647949219
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  117560.328125
Batch_id:  1 Training loss:  75845.2578125
Batch_id:  2 Training loss:  217645.203125
Batch_id:  3 Training loss:  391324.09375
Batch_id:  4 Training loss:  304572.65625
Batch_id:  5 Training loss:  86501.0703125
Batch_id:  6 Training loss:  73309.5625
Batch_id:  7 Training loss:  231257.921875
Batch_id:  8 Training loss:  53958.55859375
Batch_id:  9 Training loss:  65902.3984375
Batch_id:  10 Training loss:  62017.6640625
Batch_id:  11 Training loss:  140895.9375
Batch_id:  12 Training loss:  42928.90625
Batch_id:  13 Training loss:  594185.25


100%|██████████| 15/15 [00:00<00:00, 352.54it/s]


Batch_id:  14 Training loss:  69984.578125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1392.40it/s]


Batch_id:  0 Eval loss:  75250.75
Batch_id:  1 Eval loss:  36295.43359375
Batch_id:  2 Eval loss:  77754.0234375
Batch_id:  3 Eval loss:  204557.734375
Batch_id:  4 Eval loss:  65791.078125
Batch_id:  5 Eval loss:  118792.4296875
Batch_id:  6 Eval loss:  258549.65625
Epoch  45 Training loss:  170803.33694045607
Epoch  45 Evaluation loss:  130779.86022949219
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  91302.953125
Batch_id:  1 Training loss:  43496.62890625
Batch_id:  2 Training loss:  117952.0
Batch_id:  3 Training loss:  428083.90625
Batch_id:  4 Training loss:  384051.5625
Batch_id:  5 Training loss:  344842.25
Batch_id:  6 Training loss:  39187.1484375
Batch_id:  7 Training loss:  52915.33984375
Batch_id:  8 Training loss:  69307.1171875
Batch_id:  9 Training loss:  103053.3046875
Batch_id:  10 Training loss:  138215.953125


100%|██████████| 15/15 [00:00<00:00, 339.57it/s]


Batch_id:  11 Training loss:  186917.484375
Batch_id:  12 Training loss:  184405.765625
Batch_id:  13 Training loss:  95389.8515625
Batch_id:  14 Training loss:  296769.9375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1121.64it/s]


Batch_id:  0 Eval loss:  20384.72265625
Batch_id:  1 Eval loss:  316997.46875
Batch_id:  2 Eval loss:  69775.125
Batch_id:  3 Eval loss:  54026.55859375
Batch_id:  4 Eval loss:  133100.3125
Batch_id:  5 Eval loss:  37972.55859375
Batch_id:  6 Eval loss:  123814.890625
Epoch 00047: reducing learning rate of group 0 to 6.2500e-04.
Epoch  46 Training loss:  174046.70291385133
Epoch  46 Evaluation loss:  118136.19323730469
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  326635.875
Batch_id:  1 Training loss:  205977.46875
Batch_id:  2 Training loss:  81667.5390625
Batch_id:  3 Training loss:  192795.578125
Batch_id:  4 Training loss:  133817.859375
Batch_id:  5 Training loss:  163882.359375
Batch_id:  6 Training loss:  188240.65625
Batch_id:  7 Training loss:  337995.8125
Batch_id:  8 Training loss:  198033.09375
Batch_id:  9 Training loss:  70558.703125
Batch_id:  10 Training loss:  51191.50390625
Batch_id:  11 Training loss:  335885.28125
Batch_id:  12 Training loss:  107379.953125
Batch_id:  13 Training loss:  69924.7109375


100%|██████████| 15/15 [00:00<00:00, 357.91it/s]


Batch_id:  14 Training loss:  58892.33984375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1400.64it/s]


Batch_id:  0 Eval loss:  177376.46875
Batch_id:  1 Eval loss:  9897.1767578125
Batch_id:  2 Eval loss:  282814.0
Batch_id:  3 Eval loss:  68845.515625
Batch_id:  4 Eval loss:  61541.86328125
Batch_id:  5 Eval loss:  55461.625
Batch_id:  6 Eval loss:  65184.32421875
Epoch  47 Training loss:  170464.77934966216
Epoch  47 Evaluation loss:  112675.15213012695
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  60022.1015625
Batch_id:  1 Training loss:  262776.1875
Batch_id:  2 Training loss:  163101.265625
Batch_id:  3 Training loss:  410188.75
Batch_id:  4 Training loss:  89049.1484375
Batch_id:  5 Training loss:  113661.4765625
Batch_id:  6 Training loss:  357068.59375
Batch_id:  7 Training loss:  82612.078125
Batch_id:  8 Training loss:  75837.1953125
Batch_id:  9 Training loss:  18726.216796875
Batch_id:  10 Training loss:  230697.421875
Batch_id:  11 Training loss:  124727.8359375
Batch_id:  12 Training loss:  157672.671875


100%|██████████| 15/15 [00:00<00:00, 363.52it/s]


Batch_id:  13 Training loss:  228973.90625
Batch_id:  14 Training loss:  168630.75
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1460.63it/s]


Batch_id:  0 Eval loss:  106684.578125
Batch_id:  1 Eval loss:  135751.15625
Batch_id:  2 Eval loss:  37989.93359375
Batch_id:  3 Eval loss:  242566.78125
Batch_id:  4 Eval loss:  75531.4140625
Batch_id:  5 Eval loss:  56779.67578125
Batch_id:  6 Eval loss:  64758.3984375
Epoch  48 Training loss:  171874.70267630913
Epoch  48 Evaluation loss:  112509.677734375
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  91787.9140625
Batch_id:  1 Training loss:  289930.625
Batch_id:  2 Training loss:  22659.560546875
Batch_id:  3 Training loss:  91091.203125
Batch_id:  4 Training loss:  130457.578125
Batch_id:  5 Training loss:  492282.90625
Batch_id:  6 Training loss:  123595.203125
Batch_id:  7 Training loss:  56032.32421875
Batch_id:  8 Training loss:  48179.65625
Batch_id:  9 Training loss:  183437.609375
Batch_id:  10 Training loss:  174679.421875
Batch_id:  11 Training loss:  217663.609375
Batch_id:  12 Training loss:  118496.328125


100%|██████████| 15/15 [00:00<00:00, 403.91it/s]


Batch_id:  13 Training loss:  397457.34375
Batch_id:  14 Training loss:  91087.734375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1164.02it/s]


Batch_id:  0 Eval loss:  7381.4619140625
Batch_id:  1 Eval loss:  264988.375
Batch_id:  2 Eval loss:  49921.7578125
Batch_id:  3 Eval loss:  136692.625
Batch_id:  4 Eval loss:  40775.11328125
Batch_id:  5 Eval loss:  169698.828125
Batch_id:  6 Eval loss:  29383.87890625
Epoch  49 Training loss:  170867.50118771114
Epoch  49 Evaluation loss:  109194.06875610352
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 428.48it/s]

Batch_id:  0 Training loss:  299688.5625
Batch_id:  1 Training loss:  199551.96875
Batch_id:  2 Training loss:  96554.9375
Batch_id:  3 Training loss:  185896.484375
Batch_id:  4 Training loss:  51353.05078125
Batch_id:  5 Training loss:  180927.765625
Batch_id:  6 Training loss:  197003.375
Batch_id:  7 Training loss:  41957.0078125
Batch_id:  8 Training loss:  319070.21875
Batch_id:  9 Training loss:  90422.765625
Batch_id:  10 Training loss:  95321.3203125
Batch_id:  11 Training loss:  175512.140625
Batch_id:  12 Training loss:  96571.5390625
Batch_id:  13 Training loss:  259338.921875
Batch_id:  14 Training loss:  271811.125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 2125.08it/s]


Batch_id:  0 Eval loss:  115850.9375
Batch_id:  1 Eval loss:  44502.26953125
Batch_id:  2 Eval loss:  235226.109375
Batch_id:  3 Eval loss:  50455.25390625
Batch_id:  4 Eval loss:  86557.6015625
Batch_id:  5 Eval loss:  132406.1875
Batch_id:  6 Eval loss:  39836.98046875
Epoch  50 Training loss:  173039.26916173985
Epoch  50 Evaluation loss:  110130.52185058594
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 440.97it/s]

Batch_id:  0 Training loss:  77667.5546875
Batch_id:  1 Training loss:  77424.578125
Batch_id:  2 Training loss:  493980.59375
Batch_id:  3 Training loss:  324428.46875
Batch_id:  4 Training loss:  87524.28125
Batch_id:  5 Training loss:  273924.9375
Batch_id:  6 Training loss:  139099.796875
Batch_id:  7 Training loss:  162734.296875
Batch_id:  8 Training loss:  179254.828125
Batch_id:  9 Training loss:  79663.0390625
Batch_id:  10 Training loss:  106961.25
Batch_id:  11 Training loss:  111237.8046875
Batch_id:  12 Training loss:  223185.5
Batch_id:  13 Training loss:  47891.421875
Batch_id:  14 Training loss:  152423.90625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1608.95it/s]


Batch_id:  0 Eval loss:  45876.20703125
Batch_id:  1 Eval loss:  134780.625
Batch_id:  2 Eval loss:  59594.74609375
Batch_id:  3 Eval loss:  255127.15625
Batch_id:  4 Eval loss:  55980.15234375
Batch_id:  5 Eval loss:  113558.328125
Batch_id:  6 Eval loss:  39472.2734375
Epoch  51 Training loss:  171446.0985008446
Epoch  51 Evaluation loss:  110060.85754394531
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  72677.7734375
Batch_id:  1 Training loss:  140815.25
Batch_id:  2 Training loss:  299434.0
Batch_id:  3 Training loss:  322568.375
Batch_id:  4 Training loss:  55668.04296875
Batch_id:  5 Training loss:  248118.953125
Batch_id:  6 Training loss:  69192.1484375
Batch_id:  7 Training loss:  164011.34375
Batch_id:  8 Training loss:  191727.578125
Batch_id:  9 Training loss:  256338.046875
Batch_id:  10 Training loss:  223101.109375
Batch_id:  11 Training loss:  131832.09375
Batch_id:  12 Training loss:  136493.53125
Batch_id:  13 Training loss:  71028.078125
Batch_id:  14 Training loss:  154655.296875


100%|██████████| 15/15 [00:00<00:00, 213.82it/s]


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1162.27it/s]


Batch_id:  0 Eval loss:  50639.1875
Batch_id:  1 Eval loss:  71963.8671875
Batch_id:  2 Eval loss:  33462.35546875
Batch_id:  3 Eval loss:  42384.16796875
Batch_id:  4 Eval loss:  123710.6640625
Batch_id:  5 Eval loss:  240415.453125
Batch_id:  6 Eval loss:  295735.125
Epoch  52 Training loss:  171463.623046875
Epoch  52 Evaluation loss:  134111.06567382812
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  65858.5390625


100%|██████████| 15/15 [00:00<00:00, 323.72it/s]

Batch_id:  1 Training loss:  211794.453125
Batch_id:  2 Training loss:  284740.5
Batch_id:  3 Training loss:  242503.484375
Batch_id:  4 Training loss:  78808.2421875
Batch_id:  5 Training loss:  96227.2109375
Batch_id:  6 Training loss:  204756.125
Batch_id:  7 Training loss:  148643.4375
Batch_id:  8 Training loss:  345691.03125
Batch_id:  9 Training loss:  112540.8515625
Batch_id:  10 Training loss:  73975.140625
Batch_id:  11 Training loss:  219998.65625
Batch_id:  12 Training loss:  296652.125
Batch_id:  13 Training loss:  87163.671875
Batch_id:  14 Training loss:  45884.3046875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1000.17it/s]

Batch_id:  0 Eval loss:  39533.3828125
Batch_id:  1 Eval loss:  142452.234375
Batch_id:  2 Eval loss:  308850.625
Batch_id:  3 Eval loss:  53586.38671875
Batch_id:  4 Eval loss:  76071.5546875
Batch_id:  5 Eval loss:  53807.57421875
Batch_id:  6 Eval loss:  15563.025390625


Epoch  53 Training loss:  169948.49820523648
Epoch  53 Evaluation loss:  107791.37237548828
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  399380.75
Batch_id:  1 Training loss:  183520.234375
Batch_id:  2 Training loss:  78507.859375
Batch_id:  3 Training loss:  148102.40625
Batch_id:  4 Training loss:  49250.40625
Batch_id:  5 Training loss:  79814.9140625
Batch_id:  6 Training loss:  237120.359375
Batch_id:  7 Training loss:  177459.375
Batch_id:  8 Training loss:  86551.140625
Batch_id:  9 Training loss:  86563.2890625
Batch_id:  10 Training loss:  110415.578125
Batch_id:  11 Training loss:  279914.3125


100%|██████████| 15/15 [00:00<00:00, 319.76it/s]

Batch_id:  12 Training loss:  60372.36328125
Batch_id:  13 Training loss:  452982.09375
Batch_id:  14 Training loss:  97321.6484375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 960.93it/s]


Batch_id:  0 Eval loss:  218944.0
Batch_id:  1 Eval loss:  75139.1875
Batch_id:  2 Eval loss:  25214.052734375
Batch_id:  3 Eval loss:  94156.265625
Batch_id:  4 Eval loss:  62188.4140625
Batch_id:  5 Eval loss:  169547.0
Batch_id:  6 Eval loss:  87896.5390625
Epoch  54 Training loss:  170761.9412478885
Epoch  54 Evaluation loss:  114544.6029663086
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 332.35it/s]

Batch_id:  0 Training loss:  63579.1015625
Batch_id:  1 Training loss:  74013.2421875
Batch_id:  2 Training loss:  57671.15234375
Batch_id:  3 Training loss:  48008.46484375
Batch_id:  4 Training loss:  226488.28125
Batch_id:  5 Training loss:  477920.8125
Batch_id:  6 Training loss:  214855.203125
Batch_id:  7 Training loss:  196159.921875
Batch_id:  8 Training loss:  180266.5625
Batch_id:  9 Training loss:  171681.453125
Batch_id:  10 Training loss:  177764.109375
Batch_id:  11 Training loss:  202646.796875
Batch_id:  12 Training loss:  199907.625
Batch_id:  13 Training loss:  101834.9140625
Batch_id:  14 Training loss:  150856.375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 999.29it/s]


Batch_id:  0 Eval loss:  26682.494140625
Batch_id:  1 Eval loss:  56499.07421875
Batch_id:  2 Eval loss:  94232.2109375
Batch_id:  3 Eval loss:  83018.75
Batch_id:  4 Eval loss:  322790.75
Batch_id:  5 Eval loss:  75778.9296875
Batch_id:  6 Eval loss:  52658.39453125
Epoch  55 Training loss:  171868.51456925675
Epoch  55 Evaluation loss:  111196.9692993164
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  438164.90625
Batch_id:  1 Training loss:  144758.359375
Batch_id:  2 Training loss:  185594.359375
Batch_id:  3 Training loss:  131097.484375
Batch_id:  4 Training loss:  86113.5625
Batch_id:  5 Training loss:  83908.703125
Batch_id:  6 Training loss:  71801.7109375
Batch_id:  7 Training loss:  42606.57421875
Batch_id:  8 Training loss:  249378.40625
Batch_id:  9 Training loss:  66756.0625
Batch_id:  10 Training loss:  80830.5546875
Batch_id:  11 Training loss:  419619.21875
Batch_id:  12 Training loss:  281308.59375
Batch_id:  13 Training loss:  191817.359375
Batch_id:  14 Training loss:  41901.1640625


100%|██████████| 15/15 [00:00<00:00, 301.42it/s]


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1395.31it/s]


Batch_id:  0 Eval loss:  76380.109375
Batch_id:  1 Eval loss:  52222.296875
Batch_id:  2 Eval loss:  215846.609375
Batch_id:  3 Eval loss:  129526.0
Batch_id:  4 Eval loss:  26744.619140625
Batch_id:  5 Eval loss:  79778.390625
Batch_id:  6 Eval loss:  248764.28125
Epoch  56 Training loss:  169976.82564400337
Epoch  56 Evaluation loss:  129572.23541259766
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  106155.7421875
Batch_id:  1 Training loss:  194368.734375
Batch_id:  2 Training loss:  102738.2734375
Batch_id:  3 Training loss:  224781.421875
Batch_id:  4 Training loss:  42226.39453125
Batch_id:  5 Training loss:  174315.875
Batch_id:  6 Training loss:  152786.375
Batch_id:  7 Training loss:  321236.875
Batch_id:  8 Training loss:  78442.4921875
Batch_id:  9 Training loss:  29901.95703125
Batch_id:  10 Training loss:  155305.703125
Batch_id:  11 Training loss:  239246.546875
Batch_id:  12 Training loss:  91878.5
Batch_id:  13 Training loss:  363975.625


100%|██████████| 15/15 [00:00<00:00, 411.06it/s]

Batch_id:  14 Training loss:  284164.09375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1155.00it/s]


Batch_id:  0 Eval loss:  24592.509765625
Batch_id:  1 Eval loss:  163915.53125
Batch_id:  2 Eval loss:  228665.53125
Batch_id:  3 Eval loss:  119375.453125
Batch_id:  4 Eval loss:  61124.83984375
Batch_id:  5 Eval loss:  63000.48828125
Batch_id:  6 Eval loss:  47654.1953125
Epoch 00058: reducing learning rate of group 0 to 3.1250e-04.
Epoch  57 Training loss:  173075.98711993243
Epoch  57 Evaluation loss:  110676.33575439453
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  49806.984375
Batch_id:  1 Training loss:  298586.34375
Batch_id:  2 Training loss:  34633.31640625
Batch_id:  3 Training loss:  228775.828125
Batch_id:  4 Training loss:  208721.71875
Batch_id:  5 Training loss:  136308.609375
Batch_id:  6 Training loss:  289808.34375
Batch_id:  7 Training loss:  155257.921875
Batch_id:  8 Training loss:  299085.46875
Batch_id:  9 Training loss:  211898.234375
Batch_id:  10 Training loss:  168457.125
Batch_id:  11 Training loss:  65969.765625


100%|██████████| 15/15 [00:00<00:00, 367.95it/s]


Batch_id:  12 Training loss:  31144.150390625
Batch_id:  13 Training loss:  271119.40625
Batch_id:  14 Training loss:  67151.5546875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1392.93it/s]


Batch_id:  0 Eval loss:  33680.01171875
Batch_id:  1 Eval loss:  44517.89453125
Batch_id:  2 Eval loss:  97669.25
Batch_id:  3 Eval loss:  213715.703125
Batch_id:  4 Eval loss:  222919.953125
Batch_id:  5 Eval loss:  43339.625
Batch_id:  6 Eval loss:  59980.2421875
Epoch  58 Training loss:  170048.97104624155
Epoch  58 Evaluation loss:  111847.29370117188
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 408.96it/s]

Batch_id:  0 Training loss:  259313.03125
Batch_id:  1 Training loss:  170119.484375
Batch_id:  2 Training loss:  52864.2890625
Batch_id:  3 Training loss:  202328.546875
Batch_id:  4 Training loss:  42172.9296875
Batch_id:  5 Training loss:  167603.28125
Batch_id:  6 Training loss:  67716.6015625
Batch_id:  7 Training loss:  92883.4453125
Batch_id:  8 Training loss:  435498.90625
Batch_id:  9 Training loss:  320910.1875
Batch_id:  10 Training loss:  83906.8125
Batch_id:  11 Training loss:  66352.7890625
Batch_id:  12 Training loss:  221118.703125
Batch_id:  13 Training loss:  134071.953125
Batch_id:  14 Training loss:  231730.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1389.57it/s]


Batch_id:  0 Eval loss:  57051.98828125
Batch_id:  1 Eval loss:  111764.8984375
Batch_id:  2 Eval loss:  48962.7578125
Batch_id:  3 Eval loss:  163482.890625
Batch_id:  4 Eval loss:  212891.109375
Batch_id:  5 Eval loss:  42242.34765625
Batch_id:  6 Eval loss:  108274.109375
Epoch  59 Training loss:  172202.1510768581
Epoch  59 Evaluation loss:  116354.70336914062
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 444.87it/s]


Batch_id:  0 Training loss:  137758.828125
Batch_id:  1 Training loss:  238360.203125
Batch_id:  2 Training loss:  43550.01171875
Batch_id:  3 Training loss:  179032.484375
Batch_id:  4 Training loss:  230679.171875
Batch_id:  5 Training loss:  118965.578125
Batch_id:  6 Training loss:  91321.609375
Batch_id:  7 Training loss:  150251.765625
Batch_id:  8 Training loss:  57858.86328125
Batch_id:  9 Training loss:  92426.375
Batch_id:  10 Training loss:  179619.921875
Batch_id:  11 Training loss:  225372.171875
Batch_id:  12 Training loss:  379409.59375
Batch_id:  13 Training loss:  261486.234375
Batch_id:  14 Training loss:  146575.578125
Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  46920.82421875


100%|██████████| 7/7 [00:00<00:00, 1752.84it/s]


Batch_id:  1 Eval loss:  177151.421875
Batch_id:  2 Eval loss:  74887.8046875
Batch_id:  3 Eval loss:  278687.0625
Batch_id:  4 Eval loss:  44829.15234375
Batch_id:  5 Eval loss:  48567.96875
Batch_id:  6 Eval loss:  21433.404296875
Epoch  60 Training loss:  171126.24260979728
Epoch  60 Evaluation loss:  108199.63104248047
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 400.42it/s]

Batch_id:  0 Training loss:  231617.671875
Batch_id:  1 Training loss:  80952.3046875
Batch_id:  2 Training loss:  254070.609375
Batch_id:  3 Training loss:  286509.90625
Batch_id:  4 Training loss:  78452.640625
Batch_id:  5 Training loss:  98953.1484375
Batch_id:  6 Training loss:  391813.1875
Batch_id:  7 Training loss:  73681.1796875
Batch_id:  8 Training loss:  52415.33984375
Batch_id:  9 Training loss:  165020.28125
Batch_id:  10 Training loss:  67094.1796875
Batch_id:  11 Training loss:  239180.40625
Batch_id:  12 Training loss:  293254.1875
Batch_id:  13 Training loss:  128178.5859375
Batch_id:  14 Training loss:  76255.953125


Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  66250.9609375


100%|██████████| 7/7 [00:00<00:00, 697.47it/s]


Batch_id:  1 Eval loss:  45962.44921875
Batch_id:  2 Eval loss:  75399.34375
Batch_id:  3 Eval loss:  116989.4921875
Batch_id:  4 Eval loss:  163431.53125
Batch_id:  5 Eval loss:  187364.578125
Batch_id:  6 Eval loss:  60174.6484375
Epoch  61 Training loss:  170097.9447318412
Epoch  61 Evaluation loss:  111808.28186035156
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 327.57it/s]

Batch_id:  0 Training loss:  148666.734375
Batch_id:  1 Training loss:  264742.65625
Batch_id:  2 Training loss:  297226.84375
Batch_id:  3 Training loss:  56238.09375
Batch_id:  4 Training loss:  92939.1796875
Batch_id:  5 Training loss:  47586.21484375
Batch_id:  6 Training loss:  179188.375
Batch_id:  7 Training loss:  85745.6796875
Batch_id:  8 Training loss:  186803.625
Batch_id:  9 Training loss:  209164.703125
Batch_id:  10 Training loss:  82903.0234375
Batch_id:  11 Training loss:  175201.015625
Batch_id:  12 Training loss:  330616.34375
Batch_id:  13 Training loss:  148586.46875
Batch_id:  14 Training loss:  247065.140625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1000.07it/s]

Batch_id:  0 Eval loss:  75089.5859375
Batch_id:  1 Eval loss:  203335.671875
Batch_id:  2 Eval loss:  64220.67578125
Batch_id:  3 Eval loss:  48807.87109375
Batch_id:  4 Eval loss:  212897.203125
Batch_id:  5 Eval loss:  68102.5390625
Batch_id:  6 Eval loss:  17205.056640625


Epoch  62 Training loss:  172477.97957136823
Epoch  62 Evaluation loss:  107759.1567993164
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 436.67it/s]

Batch_id:  0 Training loss:  58531.51953125
Batch_id:  1 Training loss:  129358.1015625
Batch_id:  2 Training loss:  76567.4375
Batch_id:  3 Training loss:  105346.75
Batch_id:  4 Training loss:  484801.40625
Batch_id:  5 Training loss:  98188.8359375
Batch_id:  6 Training loss:  253242.40625
Batch_id:  7 Training loss:  111260.625
Batch_id:  8 Training loss:  188319.15625
Batch_id:  9 Training loss:  267680.34375
Batch_id:  10 Training loss:  180550.640625
Batch_id:  11 Training loss:  79709.5546875
Batch_id:  12 Training loss:  48080.6640625
Batch_id:  13 Training loss:  366684.5625
Batch_id:  14 Training loss:  67614.9765625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1398.50it/s]


Batch_id:  0 Eval loss:  27586.732421875
Batch_id:  1 Eval loss:  254054.828125
Batch_id:  2 Eval loss:  172160.84375
Batch_id:  3 Eval loss:  38444.73828125
Batch_id:  4 Eval loss:  76853.515625
Batch_id:  5 Eval loss:  57110.87109375
Batch_id:  6 Eval loss:  133111.125
Epoch  63 Training loss:  169995.74192356417
Epoch  63 Evaluation loss:  118644.16473388672
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 396.37it/s]

Batch_id:  0 Training loss:  95157.8125
Batch_id:  1 Training loss:  214969.15625
Batch_id:  2 Training loss:  56411.26171875
Batch_id:  3 Training loss:  232500.671875
Batch_id:  4 Training loss:  167189.453125
Batch_id:  5 Training loss:  163822.828125
Batch_id:  6 Training loss:  216728.65625
Batch_id:  7 Training loss:  65283.77734375
Batch_id:  8 Training loss:  376235.9375
Batch_id:  9 Training loss:  390070.21875
Batch_id:  10 Training loss:  111246.6484375
Batch_id:  11 Training loss:  137261.734375
Batch_id:  12 Training loss:  94666.5390625
Batch_id:  13 Training loss:  61374.90625
Batch_id:  14 Training loss:  151409.078125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1747.11it/s]


Batch_id:  0 Eval loss:  111452.0546875
Batch_id:  1 Eval loss:  158662.921875
Batch_id:  2 Eval loss:  34021.12109375
Batch_id:  3 Eval loss:  64413.89453125
Batch_id:  4 Eval loss:  242442.578125
Batch_id:  5 Eval loss:  48467.69921875
Batch_id:  6 Eval loss:  49395.58203125
Epoch  64 Training loss:  171238.42430320944
Epoch  64 Evaluation loss:  110758.72680664062
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 365.44it/s]

Batch_id:  0 Training loss:  57007.25
Batch_id:  1 Training loss:  344272.84375
Batch_id:  2 Training loss:  66067.1875
Batch_id:  3 Training loss:  238603.65625
Batch_id:  4 Training loss:  39958.52734375
Batch_id:  5 Training loss:  182722.0
Batch_id:  6 Training loss:  359332.4375
Batch_id:  7 Training loss:  231828.578125
Batch_id:  8 Training loss:  171803.25
Batch_id:  9 Training loss:  98620.390625
Batch_id:  10 Training loss:  158904.609375
Batch_id:  11 Training loss:  49595.5
Batch_id:  12 Training loss:  279518.53125
Batch_id:  13 Training loss:  131503.359375
Batch_id:  14 Training loss:  115444.4921875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1426.98it/s]


Batch_id:  0 Eval loss:  56649.78125
Batch_id:  1 Eval loss:  96261.7109375
Batch_id:  2 Eval loss:  25878.427734375
Batch_id:  3 Eval loss:  53728.07421875
Batch_id:  4 Eval loss:  98686.7890625
Batch_id:  5 Eval loss:  308161.78125
Batch_id:  6 Eval loss:  99724.265625
Epoch  65 Training loss:  170620.4468433277
Epoch  65 Evaluation loss:  115482.94219970703
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 471.44it/s]

Batch_id:  0 Training loss:  56870.21484375
Batch_id:  1 Training loss:  348701.09375
Batch_id:  2 Training loss:  405134.84375
Batch_id:  3 Training loss:  58650.5703125
Batch_id:  4 Training loss:  52381.6484375
Batch_id:  5 Training loss:  211979.609375
Batch_id:  6 Training loss:  326675.84375
Batch_id:  7 Training loss:  79692.203125
Batch_id:  8 Training loss:  68213.4296875
Batch_id:  9 Training loss:  134688.75
Batch_id:  10 Training loss:  193832.546875
Batch_id:  11 Training loss:  53212.92578125
Batch_id:  12 Training loss:  94739.0234375
Batch_id:  13 Training loss:  121509.8125
Batch_id:  14 Training loss:  370297.03125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1750.23it/s]


Batch_id:  0 Eval loss:  146624.453125
Batch_id:  1 Eval loss:  42104.203125
Batch_id:  2 Eval loss:  126189.6484375
Batch_id:  3 Eval loss:  59844.98828125
Batch_id:  4 Eval loss:  235113.375
Batch_id:  5 Eval loss:  57974.1015625
Batch_id:  6 Eval loss:  28197.87890625
Epoch  66 Training loss:  174093.21262668917
Epoch  66 Evaluation loss:  108757.60131835938
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  91187.0703125


100%|██████████| 15/15 [00:00<00:00, 415.93it/s]

Batch_id:  1 Training loss:  157409.0
Batch_id:  2 Training loss:  194795.375
Batch_id:  3 Training loss:  280157.0625
Batch_id:  4 Training loss:  87195.453125
Batch_id:  5 Training loss:  77452.953125
Batch_id:  6 Training loss:  163348.390625
Batch_id:  7 Training loss:  169117.203125
Batch_id:  8 Training loss:  186428.3125
Batch_id:  9 Training loss:  114027.25
Batch_id:  10 Training loss:  113900.3984375
Batch_id:  11 Training loss:  188166.984375
Batch_id:  12 Training loss:  284680.96875
Batch_id:  13 Training loss:  303600.84375
Batch_id:  14 Training loss:  111133.6796875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1160.94it/s]


Batch_id:  0 Eval loss:  152678.703125
Batch_id:  1 Eval loss:  68942.890625
Batch_id:  2 Eval loss:  165097.890625
Batch_id:  3 Eval loss:  21448.96875
Batch_id:  4 Eval loss:  232117.0
Batch_id:  5 Eval loss:  35331.44921875
Batch_id:  6 Eval loss:  8342.1953125
Epoch  67 Training loss:  170446.00981841216
Epoch  67 Evaluation loss:  106868.60900878906
Best Model saved!
Best Loss value:  107525.48942565918
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 330.37it/s]

Batch_id:  0 Training loss:  58736.4140625
Batch_id:  1 Training loss:  177309.28125
Batch_id:  2 Training loss:  77396.6171875
Batch_id:  3 Training loss:  112750.3515625
Batch_id:  4 Training loss:  513923.25
Batch_id:  5 Training loss:  66142.0859375
Batch_id:  6 Training loss:  140446.078125
Batch_id:  7 Training loss:  184373.234375
Batch_id:  8 Training loss:  79951.3828125
Batch_id:  9 Training loss:  78602.3359375
Batch_id:  10 Training loss:  387231.6875
Batch_id:  11 Training loss:  192373.734375
Batch_id:  12 Training loss:  196543.234375
Batch_id:  13 Training loss:  180996.40625
Batch_id:  14 Training loss:  68146.1640625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1164.44it/s]


Batch_id:  0 Eval loss:  125034.4921875
Batch_id:  1 Eval loss:  42173.27734375
Batch_id:  2 Eval loss:  128844.9140625
Batch_id:  3 Eval loss:  30113.58203125
Batch_id:  4 Eval loss:  228745.0
Batch_id:  5 Eval loss:  92727.609375
Batch_id:  6 Eval loss:  78245.1015625
Epoch 00069: reducing learning rate of group 0 to 1.5625e-04.
Epoch  68 Training loss:  169927.17958192568
Epoch  68 Evaluation loss:  113419.37133789062
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 354.45it/s]

Batch_id:  0 Training loss:  540356.1875
Batch_id:  1 Training loss:  55455.71484375
Batch_id:  2 Training loss:  90506.5234375
Batch_id:  3 Training loss:  248604.453125
Batch_id:  4 Training loss:  66995.5859375
Batch_id:  5 Training loss:  157968.46875
Batch_id:  6 Training loss:  61429.07421875
Batch_id:  7 Training loss:  307672.125
Batch_id:  8 Training loss:  53429.57421875
Batch_id:  9 Training loss:  82436.1015625
Batch_id:  10 Training loss:  154480.546875
Batch_id:  11 Training loss:  154576.625
Batch_id:  12 Training loss:  99602.8671875
Batch_id:  13 Training loss:  247913.328125
Batch_id:  14 Training loss:  223443.96875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 850.08it/s]


Batch_id:  0 Eval loss:  47375.52734375
Batch_id:  1 Eval loss:  55132.5
Batch_id:  2 Eval loss:  50368.10546875
Batch_id:  3 Eval loss:  12601.716796875
Batch_id:  4 Eval loss:  228278.03125
Batch_id:  5 Eval loss:  176261.359375
Batch_id:  6 Eval loss:  272068.4375
Epoch  69 Training loss:  171950.75300886823
Epoch  69 Evaluation loss:  131575.8871459961
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 429.56it/s]

Batch_id:  0 Training loss:  375582.8125
Batch_id:  1 Training loss:  67980.4609375
Batch_id:  2 Training loss:  149350.484375
Batch_id:  3 Training loss:  239356.921875
Batch_id:  4 Training loss:  266596.375
Batch_id:  5 Training loss:  85164.7421875
Batch_id:  6 Training loss:  45639.24609375
Batch_id:  7 Training loss:  394053.65625
Batch_id:  8 Training loss:  64430.23046875
Batch_id:  9 Training loss:  375255.53125
Batch_id:  10 Training loss:  51031.8359375
Batch_id:  11 Training loss:  46176.59375
Batch_id:  12 Training loss:  218956.5
Batch_id:  13 Training loss:  85879.1640625
Batch_id:  14 Training loss:  45600.640625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1733.18it/s]


Batch_id:  0 Eval loss:  36324.6484375
Batch_id:  1 Eval loss:  80377.5390625
Batch_id:  2 Eval loss:  87316.265625
Batch_id:  3 Eval loss:  106109.3515625
Batch_id:  4 Eval loss:  304816.6875
Batch_id:  5 Eval loss:  59349.1015625
Batch_id:  6 Eval loss:  11666.44140625
Epoch  70 Training loss:  169665.8915751689
Epoch  70 Evaluation loss:  107181.25549316406
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 430.80it/s]

Batch_id:  0 Training loss:  382910.375
Batch_id:  1 Training loss:  48552.48046875
Batch_id:  2 Training loss:  80202.2578125
Batch_id:  3 Training loss:  131074.8125
Batch_id:  4 Training loss:  59442.05078125
Batch_id:  5 Training loss:  284139.28125
Batch_id:  6 Training loss:  33353.0625
Batch_id:  7 Training loss:  452460.0
Batch_id:  8 Training loss:  278129.71875
Batch_id:  9 Training loss:  62612.125
Batch_id:  10 Training loss:  61997.24609375
Batch_id:  11 Training loss:  201739.15625
Batch_id:  12 Training loss:  107479.9765625
Batch_id:  13 Training loss:  64563.98828125
Batch_id:  14 Training loss:  313900.3125
Eval in session: 



100%|██████████| 7/7 [00:00<00:00, 1404.39it/s]


Batch_id:  0 Eval loss:  61344.0
Batch_id:  1 Eval loss:  59960.45703125
Batch_id:  2 Eval loss:  42475.19921875
Batch_id:  3 Eval loss:  192267.25
Batch_id:  4 Eval loss:  82435.1953125
Batch_id:  5 Eval loss:  127562.6796875
Batch_id:  6 Eval loss:  282263.53125
Epoch  71 Training loss:  173145.7326858108
Epoch  71 Evaluation loss:  132548.173828125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 428.12it/s]

Batch_id:  0 Training loss:  260861.921875
Batch_id:  1 Training loss:  122770.4609375
Batch_id:  2 Training loss:  237262.703125
Batch_id:  3 Training loss:  213879.5
Batch_id:  4 Training loss:  176944.84375
Batch_id:  5 Training loss:  152525.765625
Batch_id:  6 Training loss:  312831.46875
Batch_id:  7 Training loss:  233072.328125
Batch_id:  8 Training loss:  81577.21875
Batch_id:  9 Training loss:  330177.65625
Batch_id:  10 Training loss:  53791.9765625
Batch_id:  11 Training loss:  82540.578125
Batch_id:  12 Training loss:  100535.5234375
Batch_id:  13 Training loss:  79731.828125
Batch_id:  14 Training loss:  76181.765625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1700.85it/s]

Batch_id:  0 Eval loss:  53509.8125
Batch_id:  1 Eval loss:  51557.36328125
Batch_id:  2 Eval loss:  250183.671875
Batch_id:  3 Eval loss:  121270.1171875
Batch_id:  4 Eval loss:  160876.328125
Batch_id:  5 Eval loss:  33579.7734375
Batch_id:  6 Eval loss:  19556.162109375


Epoch  72 Training loss:  169911.18507179053
Epoch  72 Evaluation loss:  107895.8169555664
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 277.71it/s]

Batch_id:  0 Training loss:  186133.484375
Batch_id:  1 Training loss:  65131.3125
Batch_id:  2 Training loss:  482351.90625
Batch_id:  3 Training loss:  69211.9140625
Batch_id:  4 Training loss:  151132.390625
Batch_id:  5 Training loss:  70799.2109375
Batch_id:  6 Training loss:  119830.4375
Batch_id:  7 Training loss:  112613.0234375
Batch_id:  8 Training loss:  163622.875
Batch_id:  9 Training loss:  112744.5390625
Batch_id:  10 Training loss:  246102.875
Batch_id:  11 Training loss:  80775.3359375
Batch_id:  12 Training loss:  272404.84375
Batch_id:  13 Training loss:  166476.859375
Batch_id:  14 Training loss:  250719.6875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 861.30it/s]

Batch_id:  0 Eval loss:  287997.09375
Batch_id:  1 Eval loss:  63827.13671875
Batch_id:  2 Eval loss:  145188.75
Batch_id:  3 Eval loss:  29243.619140625
Batch_id:  4 Eval loss:  45253.66015625
Batch_id:  5 Eval loss:  73590.75
Batch_id:  6 Eval loss:  83860.3125


Epoch  73 Training loss:  172300.72265625
Epoch  73 Evaluation loss:  113900.2066040039
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 326.44it/s]

Batch_id:  0 Training loss:  62303.86328125
Batch_id:  1 Training loss:  190756.953125
Batch_id:  2 Training loss:  110067.078125
Batch_id:  3 Training loss:  101924.078125
Batch_id:  4 Training loss:  70780.984375
Batch_id:  5 Training loss:  55581.9765625
Batch_id:  6 Training loss:  353806.09375
Batch_id:  7 Training loss:  298174.96875
Batch_id:  8 Training loss:  66188.0859375
Batch_id:  9 Training loss:  72163.9296875
Batch_id:  10 Training loss:  199186.421875
Batch_id:  11 Training loss:  121455.453125
Batch_id:  12 Training loss:  491880.21875
Batch_id:  13 Training loss:  179871.296875
Batch_id:  14 Training loss:  159540.046875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1187.28it/s]


Batch_id:  0 Eval loss:  95201.9609375
Batch_id:  1 Eval loss:  62634.90234375
Batch_id:  2 Eval loss:  65884.546875
Batch_id:  3 Eval loss:  283012.0625
Batch_id:  4 Eval loss:  93202.203125
Batch_id:  5 Eval loss:  49886.359375
Batch_id:  6 Eval loss:  71864.078125
Epoch  74 Training loss:  171194.69251478038
Epoch  74 Evaluation loss:  112763.45520019531
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  428394.46875
Batch_id:  1 Training loss:  298802.59375
Batch_id:  2 Training loss:  39960.46875
Batch_id:  3 Training loss:  146988.375
Batch_id:  4 Training loss:  154354.375
Batch_id:  5 Training loss:  149730.875
Batch_id:  6 Training loss:  88861.4375
Batch_id:  7 Training loss:  163817.484375
Batch_id:  8 Training loss:  93811.3046875
Batch_id:  9 Training loss:  161390.671875
Batch_id:  10 Training loss:  100939.4609375
Batch_id:  11 Training loss:  347644.84375
Batch_id:  12 Training loss:  203534.71875
Batch_id:  13 Training loss:  83616.734375


100%|██████████| 15/15 [00:00<00:00, 356.97it/s]


Batch_id:  14 Training loss:  47885.8125
Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  90488.7421875


100%|██████████| 7/7 [00:00<00:00, 1166.66it/s]


Batch_id:  1 Eval loss:  127505.765625
Batch_id:  2 Eval loss:  21850.208984375
Batch_id:  3 Eval loss:  238752.875
Batch_id:  4 Eval loss:  93104.9765625
Batch_id:  5 Eval loss:  92982.7265625
Batch_id:  6 Eval loss:  34823.703125
Epoch  75 Training loss:  169576.5962837838
Epoch  75 Evaluation loss:  109298.28094482422
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  36452.60546875
Batch_id:  1 Training loss:  261515.796875
Batch_id:  2 Training loss:  118702.375
Batch_id:  3 Training loss:  296358.90625
Batch_id:  4 Training loss:  84519.3984375
Batch_id:  5 Training loss:  39432.8203125
Batch_id:  6 Training loss:  287470.0
Batch_id:  7 Training loss:  181361.84375
Batch_id:  8 Training loss:  280278.40625
Batch_id:  9 Training loss:  202488.953125
Batch_id:  10 Training loss:  218855.421875
Batch_id:  11 Training loss:  179104.453125


100%|██████████| 15/15 [00:00<00:00, 352.54it/s]


Batch_id:  12 Training loss:  77680.734375
Batch_id:  13 Training loss:  161968.265625
Batch_id:  14 Training loss:  91535.875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1167.22it/s]


Batch_id:  0 Eval loss:  131149.84375
Batch_id:  1 Eval loss:  51560.6640625
Batch_id:  2 Eval loss:  58514.86328125
Batch_id:  3 Eval loss:  152894.1875
Batch_id:  4 Eval loss:  216652.546875
Batch_id:  5 Eval loss:  59012.23828125
Batch_id:  6 Eval loss:  22132.46875
Epoch  76 Training loss:  170116.61185599663
Epoch  76 Evaluation loss:  108112.001953125
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  493987.09375


100%|██████████| 15/15 [00:00<00:00, 428.65it/s]

Batch_id:  1 Training loss:  63967.375
Batch_id:  2 Training loss:  97235.90625
Batch_id:  3 Training loss:  204385.171875
Batch_id:  4 Training loss:  134052.34375
Batch_id:  5 Training loss:  151154.703125
Batch_id:  6 Training loss:  67058.2734375
Batch_id:  7 Training loss:  60922.8125
Batch_id:  8 Training loss:  133559.8125
Batch_id:  9 Training loss:  404566.21875
Batch_id:  10 Training loss:  265201.21875
Batch_id:  11 Training loss:  100154.109375
Batch_id:  12 Training loss:  86209.1796875
Batch_id:  13 Training loss:  84395.5234375
Batch_id:  14 Training loss:  190450.0625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1750.02it/s]


Batch_id:  0 Eval loss:  140539.1875
Batch_id:  1 Eval loss:  62040.65234375
Batch_id:  2 Eval loss:  242672.109375
Batch_id:  3 Eval loss:  47273.2578125
Batch_id:  4 Eval loss:  45077.1875
Batch_id:  5 Eval loss:  137804.984375
Batch_id:  6 Eval loss:  7679.3154296875
Epoch  77 Training loss:  171439.17599239864
Epoch  77 Evaluation loss:  106732.29598999023
Best Model saved!
Best Loss value:  106868.60900878906
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 418.30it/s]

Batch_id:  0 Training loss:  46141.87109375
Batch_id:  1 Training loss:  63762.83984375
Batch_id:  2 Training loss:  74674.4765625
Batch_id:  3 Training loss:  237097.796875
Batch_id:  4 Training loss:  549172.125
Batch_id:  5 Training loss:  165369.4375
Batch_id:  6 Training loss:  377871.5625
Batch_id:  7 Training loss:  100550.7734375
Batch_id:  8 Training loss:  284408.5625
Batch_id:  9 Training loss:  105622.8515625
Batch_id:  10 Training loss:  121514.3515625
Batch_id:  11 Training loss:  44809.92578125
Batch_id:  12 Training loss:  91051.6171875
Batch_id:  13 Training loss:  116489.2734375
Batch_id:  14 Training loss:  151677.078125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1770.60it/s]


Batch_id:  0 Eval loss:  30390.94140625
Batch_id:  1 Eval loss:  145218.25
Batch_id:  2 Eval loss:  81566.4921875
Batch_id:  3 Eval loss:  124058.4765625
Batch_id:  4 Eval loss:  51348.6875
Batch_id:  5 Eval loss:  62881.36328125
Batch_id:  6 Eval loss:  457637.5
Epoch  78 Training loss:  170960.4420924831
Epoch  78 Evaluation loss:  148922.14233398438
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 428.39it/s]

Batch_id:  0 Training loss:  159356.625
Batch_id:  1 Training loss:  105406.2265625
Batch_id:  2 Training loss:  72547.234375
Batch_id:  3 Training loss:  145621.90625
Batch_id:  4 Training loss:  123616.328125
Batch_id:  5 Training loss:  308564.34375
Batch_id:  6 Training loss:  207013.875
Batch_id:  7 Training loss:  96571.6171875
Batch_id:  8 Training loss:  305436.5625
Batch_id:  9 Training loss:  179983.75
Batch_id:  10 Training loss:  110258.2109375
Batch_id:  11 Training loss:  71882.8359375
Batch_id:  12 Training loss:  381900.59375
Batch_id:  13 Training loss:  154894.109375
Batch_id:  14 Training loss:  95800.140625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1765.70it/s]

Batch_id:  0 Eval loss:  40638.34375
Batch_id:  1 Eval loss:  45965.640625
Batch_id:  2 Eval loss:  274977.46875
Batch_id:  3 Eval loss:  176399.578125
Batch_id:  4 Eval loss:  56462.2265625
Batch_id:  5 Eval loss:  74456.6953125
Batch_id:  6 Eval loss:  23705.451171875


Epoch 00080: reducing learning rate of group 0 to 7.8125e-05.
Epoch  79 Training loss:  170192.86211993243
Epoch  79 Evaluation loss:  108219.59442138672
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 447.62it/s]

Batch_id:  0 Training loss:  120272.125
Batch_id:  1 Training loss:  221922.03125
Batch_id:  2 Training loss:  173682.859375
Batch_id:  3 Training loss:  177613.671875
Batch_id:  4 Training loss:  230534.921875
Batch_id:  5 Training loss:  78016.734375
Batch_id:  6 Training loss:  113652.1796875
Batch_id:  7 Training loss:  141197.90625
Batch_id:  8 Training loss:  180552.171875
Batch_id:  9 Training loss:  270624.84375
Batch_id:  10 Training loss:  274163.34375
Batch_id:  11 Training loss:  39659.7265625
Batch_id:  12 Training loss:  287277.71875
Batch_id:  13 Training loss:  124429.2890625
Batch_id:  14 Training loss:  81049.0859375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 2332.02it/s]


Batch_id:  0 Eval loss:  232703.625
Batch_id:  1 Eval loss:  143078.625
Batch_id:  2 Eval loss:  42124.13671875
Batch_id:  3 Eval loss:  165514.515625
Batch_id:  4 Eval loss:  48227.77734375
Batch_id:  5 Eval loss:  37633.6328125
Batch_id:  6 Eval loss:  22774.2890625
Epoch  80 Training loss:  169908.68982263512
Epoch  80 Evaluation loss:  108133.84399414062
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  111546.625


100%|██████████| 15/15 [00:00<00:00, 415.13it/s]

Batch_id:  1 Training loss:  155772.921875
Batch_id:  2 Training loss:  66320.078125
Batch_id:  3 Training loss:  84226.6015625
Batch_id:  4 Training loss:  150336.234375
Batch_id:  5 Training loss:  108840.7890625
Batch_id:  6 Training loss:  167286.96875
Batch_id:  7 Training loss:  215838.203125
Batch_id:  8 Training loss:  214562.859375
Batch_id:  9 Training loss:  246632.796875
Batch_id:  10 Training loss:  514658.59375
Batch_id:  11 Training loss:  195352.5
Batch_id:  12 Training loss:  143826.109375
Batch_id:  13 Training loss:  64008.1015625
Batch_id:  14 Training loss:  73752.265625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1400.17it/s]


Batch_id:  0 Eval loss:  315777.40625
Batch_id:  1 Eval loss:  31004.609375
Batch_id:  2 Eval loss:  45718.33984375
Batch_id:  3 Eval loss:  135368.828125
Batch_id:  4 Eval loss:  80923.3828125
Batch_id:  5 Eval loss:  37205.57421875
Batch_id:  6 Eval loss:  80993.078125
Epoch  81 Training loss:  169794.70597550675
Epoch  81 Evaluation loss:  113592.3779296875
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 410.27it/s]

Batch_id:  0 Training loss:  63947.90234375
Batch_id:  1 Training loss:  218074.796875
Batch_id:  2 Training loss:  52408.40234375
Batch_id:  3 Training loss:  282077.84375
Batch_id:  4 Training loss:  110757.453125
Batch_id:  5 Training loss:  70955.5859375
Batch_id:  6 Training loss:  442131.21875
Batch_id:  7 Training loss:  102284.1171875
Batch_id:  8 Training loss:  157325.875
Batch_id:  9 Training loss:  46226.68359375
Batch_id:  10 Training loss:  167738.984375
Batch_id:  11 Training loss:  191555.1875
Batch_id:  12 Training loss:  170256.96875
Batch_id:  13 Training loss:  252273.203125
Batch_id:  14 Training loss:  213055.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1752.22it/s]


Batch_id:  0 Eval loss:  46406.55859375
Batch_id:  1 Eval loss:  101353.5078125
Batch_id:  2 Eval loss:  56531.90234375
Batch_id:  3 Eval loss:  67268.375
Batch_id:  4 Eval loss:  149063.484375
Batch_id:  5 Eval loss:  234416.828125
Batch_id:  6 Eval loss:  58223.03515625
Epoch  82 Training loss:  171693.92551731417
Epoch  82 Evaluation loss:  111447.45178222656
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  71373.2578125
Batch_id:  1 Training loss:  154539.625
Batch_id:  2 Training loss:  429661.71875
Batch_id:  3 Training loss:  63565.28125
Batch_id:  4 Training loss:  182518.140625
Batch_id:  5 Training loss:  106356.6875
Batch_id:  6 Training loss:  150576.546875
Batch_id:  7 Training loss:  77088.953125
Batch_id:  8 Training loss:  339074.96875
Batch_id:  9 Training loss:  61188.73828125
Batch_id:  10 Training loss:  399867.90625
Batch_id:  11 Training loss:  67407.046875
Batch_id:  12 Training loss:  128645.625
Batch_id:  13 Training loss:  96965.578125


100%|██████████| 15/15 [00:00<00:00, 434.23it/s]


Batch_id:  14 Training loss:  211514.359375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1394.71it/s]

Batch_id:  0 Eval loss:  88754.921875
Batch_id:  1 Eval loss:  53314.74609375
Batch_id:  2 Eval loss:  162759.484375
Batch_id:  3 Eval loss:  53964.84375
Batch_id:  4 Eval loss:  74526.8828125
Batch_id:  5 Eval loss:  220810.078125
Batch_id:  6 Eval loss:  60501.8125


Epoch  83 Training loss:  171644.89416173985
Epoch  83 Evaluation loss:  111661.37023925781
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  58571.48828125
Batch_id:  1 Training loss:  704184.4375
Batch_id:  2 Training loss:  164474.515625
Batch_id:  3 Training loss:  69580.7109375
Batch_id:  4 Training loss:  85618.625
Batch_id:  5 Training loss:  173915.375
Batch_id:  6 Training loss:  110944.25
Batch_id:  7 Training loss:  70587.9375
Batch_id:  8 Training loss:  38303.4453125
Batch_id:  9 Training loss:  241236.40625
Batch_id:  10 Training loss:  125100.0546875
Batch_id:  11 Training loss:  167561.15625
Batch_id:  12 Training loss:  199529.921875
Batch_id:  13 Training loss:  203804.21875


100%|██████████| 15/15 [00:00<00:00, 401.48it/s]


Batch_id:  14 Training loss:  106253.625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1366.22it/s]

Batch_id:  0 Eval loss:  39219.60546875
Batch_id:  1 Eval loss:  154773.484375
Batch_id:  2 Eval loss:  203872.828125
Batch_id:  3 Eval loss:  129488.1015625
Batch_id:  4 Eval loss:  100587.9140625
Batch_id:  5 Eval loss:  42517.95703125
Batch_id:  6 Eval loss:  19640.1171875


Epoch  84 Training loss:  170247.71405194257
Epoch  84 Evaluation loss:  107828.12622070312
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  297688.65625
Batch_id:  1 Training loss:  62158.65234375
Batch_id:  2 Training loss:  345736.34375
Batch_id:  3 Training loss:  39048.28125
Batch_id:  4 Training loss:  179494.53125
Batch_id:  5 Training loss:  62612.17578125
Batch_id:  6 Training loss:  172043.609375
Batch_id:  7 Training loss:  136818.46875
Batch_id:  8 Training loss:  181795.421875
Batch_id:  9 Training loss:  74258.7578125
Batch_id:  10 Training loss:  155845.234375
Batch_id:  11 Training loss:  127770.3984375
Batch_id:  12 Training loss:  295807.4375
Batch_id:  13 Training loss:  227654.953125


100%|██████████| 15/15 [00:00<00:00, 364.61it/s]


Batch_id:  14 Training loss:  174750.40625
Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  74794.3984375


100%|██████████| 7/7 [00:00<00:00, 771.64it/s]


Batch_id:  1 Eval loss:  71230.1875
Batch_id:  2 Eval loss:  163184.40625
Batch_id:  3 Eval loss:  26434.05078125
Batch_id:  4 Eval loss:  249565.984375
Batch_id:  5 Eval loss:  44498.02734375
Batch_id:  6 Eval loss:  121478.6796875
Epoch  85 Training loss:  171181.3059543919
Epoch  85 Evaluation loss:  117372.77099609375
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 366.52it/s]

Batch_id:  0 Training loss:  33502.4453125
Batch_id:  1 Training loss:  134241.984375
Batch_id:  2 Training loss:  197223.953125
Batch_id:  3 Training loss:  465967.25
Batch_id:  4 Training loss:  73772.7890625
Batch_id:  5 Training loss:  246810.28125
Batch_id:  6 Training loss:  60885.7890625
Batch_id:  7 Training loss:  202074.234375
Batch_id:  8 Training loss:  187819.859375
Batch_id:  9 Training loss:  116751.578125
Batch_id:  10 Training loss:  159855.359375
Batch_id:  11 Training loss:  69248.6015625
Batch_id:  12 Training loss:  318007.21875
Batch_id:  13 Training loss:  72880.734375
Batch_id:  14 Training loss:  198912.9375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 996.58it/s]


Batch_id:  0 Eval loss:  98263.1640625
Batch_id:  1 Eval loss:  58381.82421875
Batch_id:  2 Eval loss:  53108.6015625
Batch_id:  3 Eval loss:  115085.078125
Batch_id:  4 Eval loss:  230052.78125
Batch_id:  5 Eval loss:  92633.40625
Batch_id:  6 Eval loss:  76966.1796875
Epoch  86 Training loss:  171483.44700168917
Epoch  86 Evaluation loss:  113201.72424316406
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  75715.7109375
Batch_id:  1 Training loss:  98165.9296875
Batch_id:  2 Training loss:  59244.5
Batch_id:  3 Training loss:  244693.0
Batch_id:  4 Training loss:  233148.828125
Batch_id:  5 Training loss:  439088.90625
Batch_id:  6 Training loss:  243183.078125
Batch_id:  7 Training loss:  235952.421875
Batch_id:  8 Training loss:  80798.75
Batch_id:  9 Training loss:  50564.32421875
Batch_id:  10 Training loss:  57443.05078125


100%|██████████| 15/15 [00:00<00:00, 290.35it/s]


Batch_id:  11 Training loss:  237257.75
Batch_id:  12 Training loss:  76151.3125
Batch_id:  13 Training loss:  168791.328125
Batch_id:  14 Training loss:  247668.875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1410.87it/s]


Batch_id:  0 Eval loss:  86321.78125
Batch_id:  1 Eval loss:  52820.89453125
Batch_id:  2 Eval loss:  227617.046875
Batch_id:  3 Eval loss:  59255.5390625
Batch_id:  4 Eval loss:  30724.982421875
Batch_id:  5 Eval loss:  200566.25
Batch_id:  6 Eval loss:  52447.546875
Epoch  87 Training loss:  172153.2274070946
Epoch  87 Evaluation loss:  110899.0689086914
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 375.32it/s]

Batch_id:  0 Training loss:  137907.9375
Batch_id:  1 Training loss:  115909.078125
Batch_id:  2 Training loss:  182112.578125
Batch_id:  3 Training loss:  342322.34375
Batch_id:  4 Training loss:  44671.83984375
Batch_id:  5 Training loss:  276498.46875
Batch_id:  6 Training loss:  344514.34375
Batch_id:  7 Training loss:  95141.7734375
Batch_id:  8 Training loss:  99674.5234375
Batch_id:  9 Training loss:  43667.90625
Batch_id:  10 Training loss:  116671.7109375
Batch_id:  11 Training loss:  345561.71875
Batch_id:  12 Training loss:  233042.15625
Batch_id:  13 Training loss:  75831.390625
Batch_id:  14 Training loss:  55598.2578125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1715.56it/s]


Batch_id:  0 Eval loss:  64760.65234375
Batch_id:  1 Eval loss:  50024.69140625
Batch_id:  2 Eval loss:  62086.52734375
Batch_id:  3 Eval loss:  54719.6015625
Batch_id:  4 Eval loss:  104780.59375
Batch_id:  5 Eval loss:  330357.15625
Batch_id:  6 Eval loss:  28937.884765625
Epoch  88 Training loss:  169535.5423880912
Epoch  88 Evaluation loss:  108697.98553466797
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 403.38it/s]

Batch_id:  0 Training loss:  371333.6875
Batch_id:  1 Training loss:  380492.59375
Batch_id:  2 Training loss:  48484.32421875
Batch_id:  3 Training loss:  113489.5234375
Batch_id:  4 Training loss:  113733.1640625
Batch_id:  5 Training loss:  56747.04296875
Batch_id:  6 Training loss:  215797.171875
Batch_id:  7 Training loss:  85423.0234375
Batch_id:  8 Training loss:  258350.546875
Batch_id:  9 Training loss:  98821.953125
Batch_id:  10 Training loss:  190254.109375
Batch_id:  11 Training loss:  294548.375
Batch_id:  12 Training loss:  117929.5
Batch_id:  13 Training loss:  117298.6875
Batch_id:  14 Training loss:  44656.015625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1766.98it/s]


Batch_id:  0 Eval loss:  45023.41796875
Batch_id:  1 Eval loss:  50713.7421875
Batch_id:  2 Eval loss:  269028.25
Batch_id:  3 Eval loss:  141599.90625
Batch_id:  4 Eval loss:  118009.8125
Batch_id:  5 Eval loss:  19408.91015625
Batch_id:  6 Eval loss:  86076.09375
Epoch  89 Training loss:  169416.19721283784
Epoch  89 Evaluation loss:  114040.64575195312
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 427.68it/s]

Batch_id:  0 Training loss:  95231.171875
Batch_id:  1 Training loss:  70935.3671875
Batch_id:  2 Training loss:  90029.34375
Batch_id:  3 Training loss:  81283.1796875
Batch_id:  4 Training loss:  45198.3203125
Batch_id:  5 Training loss:  92810.1015625
Batch_id:  6 Training loss:  296173.59375
Batch_id:  7 Training loss:  213350.53125
Batch_id:  8 Training loss:  242493.328125
Batch_id:  9 Training loss:  301260.34375
Batch_id:  10 Training loss:  303558.34375
Batch_id:  11 Training loss:  303112.40625
Batch_id:  12 Training loss:  218739.859375
Batch_id:  13 Training loss:  108702.6015625
Batch_id:  14 Training loss:  44501.921875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1738.93it/s]


Batch_id:  0 Eval loss:  49027.5859375
Batch_id:  1 Eval loss:  241118.359375
Batch_id:  2 Eval loss:  53866.21875
Batch_id:  3 Eval loss:  56918.66796875
Batch_id:  4 Eval loss:  145521.25
Batch_id:  5 Eval loss:  42213.88671875
Batch_id:  6 Eval loss:  223754.734375
Epoch 00091: reducing learning rate of group 0 to 3.9063e-05.
Epoch  90 Training loss:  169417.5955447635
Epoch  90 Evaluation loss:  126940.73486328125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 423.11it/s]

Batch_id:  0 Training loss:  47495.68359375
Batch_id:  1 Training loss:  236784.5
Batch_id:  2 Training loss:  55686.5
Batch_id:  3 Training loss:  155227.9375
Batch_id:  4 Training loss:  118281.390625
Batch_id:  5 Training loss:  248993.125
Batch_id:  6 Training loss:  68836.6640625
Batch_id:  7 Training loss:  80692.8359375
Batch_id:  8 Training loss:  45352.21484375
Batch_id:  9 Training loss:  384579.59375
Batch_id:  10 Training loss:  512992.0
Batch_id:  11 Training loss:  99504.8984375
Batch_id:  12 Training loss:  215107.609375
Batch_id:  13 Training loss:  175880.671875
Batch_id:  14 Training loss:  65217.140625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1766.98it/s]

Batch_id:  0 Eval loss:  159270.234375
Batch_id:  1 Eval loss:  45956.03515625
Batch_id:  2 Eval loss:  145096.015625
Batch_id:  3 Eval loss:  37732.7578125
Batch_id:  4 Eval loss:  16628.513671875
Batch_id:  5 Eval loss:  250376.046875
Batch_id:  6 Eval loss:  57822.1875


Epoch  91 Training loss:  169637.3490287162
Epoch  91 Evaluation loss:  111387.7798461914
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 442.36it/s]


Batch_id:  0 Training loss:  77097.3046875
Batch_id:  1 Training loss:  114735.9296875
Batch_id:  2 Training loss:  350690.09375
Batch_id:  3 Training loss:  42475.58203125
Batch_id:  4 Training loss:  310192.65625
Batch_id:  5 Training loss:  181377.40625
Batch_id:  6 Training loss:  184805.5
Batch_id:  7 Training loss:  128482.828125
Batch_id:  8 Training loss:  76448.4921875
Batch_id:  9 Training loss:  236532.296875
Batch_id:  10 Training loss:  72437.7421875
Batch_id:  11 Training loss:  103169.0078125
Batch_id:  12 Training loss:  317442.03125
Batch_id:  13 Training loss:  141871.796875
Batch_id:  14 Training loss:  200318.5625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1396.24it/s]


Batch_id:  0 Eval loss:  61542.26171875
Batch_id:  1 Eval loss:  168201.0
Batch_id:  2 Eval loss:  99079.671875
Batch_id:  3 Eval loss:  22984.7734375
Batch_id:  4 Eval loss:  39453.40234375
Batch_id:  5 Eval loss:  96759.5234375
Batch_id:  6 Eval loss:  475468.03125
Epoch  92 Training loss:  171491.704761402
Epoch  92 Evaluation loss:  150545.10375976562
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 442.76it/s]


Batch_id:  0 Training loss:  121716.5546875
Batch_id:  1 Training loss:  135791.578125
Batch_id:  2 Training loss:  132486.90625
Batch_id:  3 Training loss:  59120.4375
Batch_id:  4 Training loss:  31084.201171875
Batch_id:  5 Training loss:  59640.08984375
Batch_id:  6 Training loss:  487495.8125
Batch_id:  7 Training loss:  128654.2265625
Batch_id:  8 Training loss:  147717.0
Batch_id:  9 Training loss:  52021.171875
Batch_id:  10 Training loss:  126465.640625
Batch_id:  11 Training loss:  276021.84375
Batch_id:  12 Training loss:  424504.21875
Batch_id:  13 Training loss:  248917.28125
Batch_id:  14 Training loss:  82623.109375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1390.22it/s]


Batch_id:  0 Eval loss:  134342.796875
Batch_id:  1 Eval loss:  41095.83984375
Batch_id:  2 Eval loss:  105452.390625
Batch_id:  3 Eval loss:  132589.09375
Batch_id:  4 Eval loss:  24510.466796875
Batch_id:  5 Eval loss:  46260.37109375
Batch_id:  6 Eval loss:  484770.0625
Epoch  93 Training loss:  169882.43731524493
Epoch  93 Evaluation loss:  151409.5346069336
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 425.37it/s]

Batch_id:  0 Training loss:  162743.125
Batch_id:  1 Training loss:  274471.21875
Batch_id:  2 Training loss:  84868.3671875
Batch_id:  3 Training loss:  42213.22265625
Batch_id:  4 Training loss:  370276.71875
Batch_id:  5 Training loss:  104615.203125
Batch_id:  6 Training loss:  352374.34375
Batch_id:  7 Training loss:  125591.2890625
Batch_id:  8 Training loss:  303249.46875
Batch_id:  9 Training loss:  333968.96875
Batch_id:  10 Training loss:  77835.34375
Batch_id:  11 Training loss:  89518.5546875
Batch_id:  12 Training loss:  59759.58984375
Batch_id:  13 Training loss:  61693.53125
Batch_id:  14 Training loss:  68799.03125


Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  92522.3671875


100%|██████████| 7/7 [00:00<00:00, 1430.60it/s]


Batch_id:  1 Eval loss:  239987.0
Batch_id:  2 Eval loss:  58120.5
Batch_id:  3 Eval loss:  115174.453125
Batch_id:  4 Eval loss:  128228.0234375
Batch_id:  5 Eval loss:  29533.447265625
Batch_id:  6 Eval loss:  36570.6484375
Epoch  94 Training loss:  169728.24165962837
Epoch  94 Evaluation loss:  109396.31866455078
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  343718.96875
Batch_id:  1 Training loss:  96433.2734375
Batch_id:  2 Training loss:  60833.8515625
Batch_id:  3 Training loss:  219253.359375
Batch_id:  4 Training loss:  116793.25
Batch_id:  5 Training loss:  60879.75
Batch_id:  6 Training loss:  239347.28125
Batch_id:  7 Training loss:  211172.796875
Batch_id:  8 Training loss:  185785.03125
Batch_id:  9 Training loss:  93609.7890625
Batch_id:  10 Training loss:  201538.609375
Batch_id:  11 Training loss:  179425.609375
Batch_id:  12 Training loss:  299142.59375
Batch_id:  13 Training loss:  58719.5


100%|██████████| 15/15 [00:00<00:00, 230.38it/s]


Batch_id:  14 Training loss:  163528.03125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 777.88it/s]


Batch_id:  0 Eval loss:  133959.078125
Batch_id:  1 Eval loss:  50028.06640625
Batch_id:  2 Eval loss:  266639.59375
Batch_id:  3 Eval loss:  48945.13671875
Batch_id:  4 Eval loss:  122871.9140625
Batch_id:  5 Eval loss:  36031.0078125
Batch_id:  6 Eval loss:  49181.02734375
Epoch  95 Training loss:  170958.22265625
Epoch  95 Evaluation loss:  110571.22253417969
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 314.82it/s]

Batch_id:  0 Training loss:  341497.71875
Batch_id:  1 Training loss:  139271.328125
Batch_id:  2 Training loss:  61731.26171875
Batch_id:  3 Training loss:  185341.625
Batch_id:  4 Training loss:  64180.33203125
Batch_id:  5 Training loss:  400824.46875
Batch_id:  6 Training loss:  195871.625
Batch_id:  7 Training loss:  130809.8125
Batch_id:  8 Training loss:  82473.5546875
Batch_id:  9 Training loss:  253567.78125
Batch_id:  10 Training loss:  147833.953125
Batch_id:  11 Training loss:  204609.328125
Batch_id:  12 Training loss:  97645.4765625
Batch_id:  13 Training loss:  27291.935546875
Batch_id:  14 Training loss:  205689.03125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 980.11it/s]


Batch_id:  0 Eval loss:  87167.015625
Batch_id:  1 Eval loss:  131912.609375
Batch_id:  2 Eval loss:  57105.9140625
Batch_id:  3 Eval loss:  57075.90234375
Batch_id:  4 Eval loss:  289862.78125
Batch_id:  5 Eval loss:  32161.091796875
Batch_id:  6 Eval loss:  57162.51171875
Epoch  96 Training loss:  171529.67786634288
Epoch  96 Evaluation loss:  111319.97283935547
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 290.01it/s]

Batch_id:  0 Training loss:  215976.671875
Batch_id:  1 Training loss:  89194.8046875
Batch_id:  2 Training loss:  92608.4140625
Batch_id:  3 Training loss:  161847.65625
Batch_id:  4 Training loss:  97633.4140625
Batch_id:  5 Training loss:  126390.578125
Batch_id:  6 Training loss:  123168.4609375
Batch_id:  7 Training loss:  128702.0390625
Batch_id:  8 Training loss:  367383.6875
Batch_id:  9 Training loss:  97882.078125
Batch_id:  10 Training loss:  309851.6875
Batch_id:  11 Training loss:  433781.96875
Batch_id:  12 Training loss:  107782.9296875
Batch_id:  13 Training loss:  100446.453125
Batch_id:  14 Training loss:  55943.55078125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 875.17it/s]


Batch_id:  0 Eval loss:  60583.24609375
Batch_id:  1 Eval loss:  49171.8203125
Batch_id:  2 Eval loss:  358879.21875
Batch_id:  3 Eval loss:  33437.99609375
Batch_id:  4 Eval loss:  122775.25
Batch_id:  5 Eval loss:  45673.54296875
Batch_id:  6 Eval loss:  19042.125
Epoch  97 Training loss:  169499.62125211148
Epoch  97 Evaluation loss:  107744.24987792969
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  109805.2421875
Batch_id:  1 Training loss:  61985.21484375
Batch_id:  2 Training loss:  324908.40625
Batch_id:  3 Training loss:  130458.3125
Batch_id:  4 Training loss:  76690.6875
Batch_id:  5 Training loss:  208055.09375
Batch_id:  6 Training loss:  242135.75
Batch_id:  7 Training loss:  47256.31640625
Batch_id:  8 Training loss:  91363.1875


100%|██████████| 15/15 [00:00<00:00, 282.39it/s]

Batch_id:  9 Training loss:  151816.796875
Batch_id:  10 Training loss:  371899.96875
Batch_id:  11 Training loss:  203278.40625
Batch_id:  12 Training loss:  200418.171875
Batch_id:  13 Training loss:  243067.703125
Batch_id:  14 Training loss:  42685.140625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 636.44it/s]

Batch_id:  0 Eval loss:  226622.546875
Batch_id:  1 Eval loss:  71168.8515625
Batch_id:  2 Eval loss:  111060.8046875
Batch_id:  3 Eval loss:  106946.4296875
Batch_id:  4 Eval loss:  72648.4609375
Batch_id:  5 Eval loss:  58528.5390625
Batch_id:  6 Eval loss:  77849.265625


Epoch  98 Training loss:  169312.45935388512
Epoch  98 Evaluation loss:  113253.89038085938
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 286.80it/s]

Batch_id:  0 Training loss:  81970.5390625
Batch_id:  1 Training loss:  63888.11328125
Batch_id:  2 Training loss:  102060.8359375
Batch_id:  3 Training loss:  58435.44921875
Batch_id:  4 Training loss:  137753.828125
Batch_id:  5 Training loss:  280454.0
Batch_id:  6 Training loss:  297335.09375
Batch_id:  7 Training loss:  406514.6875
Batch_id:  8 Training loss:  144940.984375
Batch_id:  9 Training loss:  129951.828125
Batch_id:  10 Training loss:  53739.11328125
Batch_id:  11 Training loss:  300606.65625
Batch_id:  12 Training loss:  257792.65625
Batch_id:  13 Training loss:  46083.1015625
Batch_id:  14 Training loss:  170603.5625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1398.23it/s]


Batch_id:  0 Eval loss:  65869.6015625
Batch_id:  1 Eval loss:  71653.8125
Batch_id:  2 Eval loss:  61557.70703125
Batch_id:  3 Eval loss:  287586.71875
Batch_id:  4 Eval loss:  56714.1015625
Batch_id:  5 Eval loss:  101409.7265625
Batch_id:  6 Eval loss:  83200.453125
Epoch  99 Training loss:  171089.8952174831
Epoch  99 Evaluation loss:  113748.76892089844
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 360.55it/s]

Batch_id:  0 Training loss:  121471.0546875
Batch_id:  1 Training loss:  219174.359375
Batch_id:  2 Training loss:  209483.0
Batch_id:  3 Training loss:  74625.765625
Batch_id:  4 Training loss:  67340.03125
Batch_id:  5 Training loss:  120141.7890625
Batch_id:  6 Training loss:  174970.375
Batch_id:  7 Training loss:  72168.375
Batch_id:  8 Training loss:  315377.46875
Batch_id:  9 Training loss:  252170.953125
Batch_id:  10 Training loss:  255243.421875
Batch_id:  11 Training loss:  273944.84375
Batch_id:  12 Training loss:  150641.71875
Batch_id:  13 Training loss:  128050.5546875
Batch_id:  14 Training loss:  78106.53125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 776.33it/s]


Batch_id:  0 Eval loss:  74228.0546875
Batch_id:  1 Eval loss:  53341.58203125
Batch_id:  2 Eval loss:  223979.796875
Batch_id:  3 Eval loss:  11915.1318359375
Batch_id:  4 Eval loss:  59088.65234375
Batch_id:  5 Eval loss:  139028.015625
Batch_id:  6 Eval loss:  291217.625
Epoch  100 Training loss:  169791.23258023648
Epoch  100 Evaluation loss:  133249.82162475586
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 361.89it/s]

Batch_id:  0 Training loss:  39725.10546875
Batch_id:  1 Training loss:  334485.40625
Batch_id:  2 Training loss:  148413.796875
Batch_id:  3 Training loss:  295341.84375
Batch_id:  4 Training loss:  51594.94921875
Batch_id:  5 Training loss:  43323.1640625
Batch_id:  6 Training loss:  259762.90625
Batch_id:  7 Training loss:  52977.51953125
Batch_id:  8 Training loss:  129703.375
Batch_id:  9 Training loss:  261272.234375
Batch_id:  10 Training loss:  204516.796875
Batch_id:  11 Training loss:  225845.046875
Batch_id:  12 Training loss:  67333.890625
Batch_id:  13 Training loss:  154382.203125
Batch_id:  14 Training loss:  286046.40625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 883.52it/s]

Batch_id:  0 Eval loss:  81712.109375
Batch_id:  1 Eval loss:  303804.59375
Batch_id:  2 Eval loss:  42074.76171875
Batch_id:  3 Eval loss:  55524.4375
Batch_id:  4 Eval loss:  63570.71484375
Batch_id:  5 Eval loss:  30559.443359375
Batch_id:  6 Eval loss:  252010.640625


Epoch 00102: reducing learning rate of group 0 to 1.9531e-05.
Epoch  101 Training loss:  172616.53003589527
Epoch  101 Evaluation loss:  129571.35955810547
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 358.74it/s]

Batch_id:  0 Training loss:  68153.453125
Batch_id:  1 Training loss:  368983.46875
Batch_id:  2 Training loss:  221843.046875
Batch_id:  3 Training loss:  45764.4140625
Batch_id:  4 Training loss:  52840.30078125
Batch_id:  5 Training loss:  276045.09375
Batch_id:  6 Training loss:  339212.0625
Batch_id:  7 Training loss:  86501.75
Batch_id:  8 Training loss:  107210.0234375
Batch_id:  9 Training loss:  188228.03125
Batch_id:  10 Training loss:  128852.5546875
Batch_id:  11 Training loss:  91720.7890625
Batch_id:  12 Training loss:  163698.03125
Batch_id:  13 Training loss:  198052.65625
Batch_id:  14 Training loss:  200210.8125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1404.66it/s]


Batch_id:  0 Eval loss:  51077.89453125
Batch_id:  1 Eval loss:  109520.7890625
Batch_id:  2 Eval loss:  41687.7265625
Batch_id:  3 Eval loss:  244379.578125
Batch_id:  4 Eval loss:  102438.7421875
Batch_id:  5 Eval loss:  121472.6640625
Batch_id:  6 Eval loss:  18710.166015625
Epoch  102 Training loss:  171440.30326224663
Epoch  102 Evaluation loss:  107701.18133544922
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  308579.8125
Batch_id:  1 Training loss:  86036.7890625
Batch_id:  2 Training loss:  275730.65625
Batch_id:  3 Training loss:  53094.96484375
Batch_id:  4 Training loss:  174552.03125
Batch_id:  5 Training loss:  80202.1875
Batch_id:  6 Training loss:  145827.96875
Batch_id:  7 Training loss:  167990.21875
Batch_id:  8 Training loss:  258118.25


100%|██████████| 15/15 [00:00<00:00, 383.34it/s]


Batch_id:  9 Training loss:  134474.796875
Batch_id:  10 Training loss:  92886.046875
Batch_id:  11 Training loss:  386283.625
Batch_id:  12 Training loss:  92641.75
Batch_id:  13 Training loss:  186250.234375
Batch_id:  14 Training loss:  68133.625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1397.44it/s]


Batch_id:  0 Eval loss:  26230.1875
Batch_id:  1 Eval loss:  271373.34375
Batch_id:  2 Eval loss:  151689.484375
Batch_id:  3 Eval loss:  112744.5859375
Batch_id:  4 Eval loss:  90551.6875
Batch_id:  5 Eval loss:  18609.021484375
Batch_id:  6 Eval loss:  17172.37109375
Epoch  103 Training loss:  169648.84844805743
Epoch  103 Evaluation loss:  107557.91900634766
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  145504.578125
Batch_id:  1 Training loss:  61172.08984375
Batch_id:  2 Training loss:  410162.59375
Batch_id:  3 Training loss:  58804.2890625
Batch_id:  4 Training loss:  252431.15625
Batch_id:  5 Training loss:  254165.984375
Batch_id:  6 Training loss:  45365.63671875
Batch_id:  7 Training loss:  53979.9140625
Batch_id:  8 Training loss:  129971.4765625
Batch_id:  9 Training loss:  73860.2734375
Batch_id:  10 Training loss:  380649.96875
Batch_id:  11 Training loss:  114669.1015625
Batch_id:  12 Training loss:  214871.359375
Batch_id:  13 Training loss:  193258.578125


100%|██████████| 15/15 [00:00<00:00, 347.62it/s]


Batch_id:  14 Training loss:  135549.65625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1741.61it/s]


Batch_id:  0 Eval loss:  166097.171875
Batch_id:  1 Eval loss:  115281.9296875
Batch_id:  2 Eval loss:  237149.953125
Batch_id:  3 Eval loss:  65937.203125
Batch_id:  4 Eval loss:  61728.24609375
Batch_id:  5 Eval loss:  24193.419921875
Batch_id:  6 Eval loss:  19142.7578125
Epoch  104 Training loss:  170568.69298986485
Epoch  104 Evaluation loss:  107739.16900634766
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  321831.6875
Batch_id:  1 Training loss:  95201.7421875
Batch_id:  2 Training loss:  184320.53125
Batch_id:  3 Training loss:  93681.6171875
Batch_id:  4 Training loss:  50714.62109375
Batch_id:  5 Training loss:  376816.21875
Batch_id:  6 Training loss:  155825.453125
Batch_id:  7 Training loss:  471612.40625
Batch_id:  8 Training loss:  119723.75
Batch_id:  9 Training loss:  49233.94921875
Batch_id:  10 Training loss:  165218.421875
Batch_id:  11 Training loss:  87201.4140625
Batch_id:  12 Training loss:  47128.4375
Batch_id:  13 Training loss:  117802.1484375


100%|██████████| 15/15 [00:00<00:00, 390.08it/s]


Batch_id:  14 Training loss:  201152.71875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1371.33it/s]


Batch_id:  0 Eval loss:  109836.3671875
Batch_id:  1 Eval loss:  60977.71484375
Batch_id:  2 Eval loss:  48506.68359375
Batch_id:  3 Eval loss:  53858.5625
Batch_id:  4 Eval loss:  345949.84375
Batch_id:  5 Eval loss:  41418.71875
Batch_id:  6 Eval loss:  43775.69921875
Epoch  105 Training loss:  171450.34575591216
Epoch  105 Evaluation loss:  110050.56091308594
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 393.43it/s]

Batch_id:  0 Training loss:  144469.734375
Batch_id:  1 Training loss:  83166.3515625
Batch_id:  2 Training loss:  176528.015625
Batch_id:  3 Training loss:  212741.359375
Batch_id:  4 Training loss:  76318.5
Batch_id:  5 Training loss:  42641.12109375
Batch_id:  6 Training loss:  339521.8125
Batch_id:  7 Training loss:  360176.0
Batch_id:  8 Training loss:  246624.5
Batch_id:  9 Training loss:  151488.140625
Batch_id:  10 Training loss:  167448.15625
Batch_id:  11 Training loss:  208652.703125
Batch_id:  12 Training loss:  56678.91796875
Batch_id:  13 Training loss:  50428.41796875
Batch_id:  14 Training loss:  225284.640625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1751.80it/s]


Batch_id:  0 Eval loss:  65661.0546875
Batch_id:  1 Eval loss:  64468.52734375
Batch_id:  2 Eval loss:  136558.75
Batch_id:  3 Eval loss:  205193.625
Batch_id:  4 Eval loss:  134425.421875
Batch_id:  5 Eval loss:  65243.6953125
Batch_id:  6 Eval loss:  16270.728515625
Epoch  106 Training loss:  171768.13318201012
Epoch  106 Evaluation loss:  107472.1566772461
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 425.25it/s]

Batch_id:  0 Training loss:  230382.703125
Batch_id:  1 Training loss:  181789.234375
Batch_id:  2 Training loss:  45501.98046875
Batch_id:  3 Training loss:  40125.6015625
Batch_id:  4 Training loss:  205684.3125
Batch_id:  5 Training loss:  72902.7109375
Batch_id:  6 Training loss:  104654.4296875
Batch_id:  7 Training loss:  59532.88671875
Batch_id:  8 Training loss:  238380.921875
Batch_id:  9 Training loss:  284621.09375
Batch_id:  10 Training loss:  390031.21875
Batch_id:  11 Training loss:  72906.0859375
Batch_id:  12 Training loss:  234361.453125
Batch_id:  13 Training loss:  70535.1875
Batch_id:  14 Training loss:  332202.1875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1351.13it/s]


Batch_id:  0 Eval loss:  161730.875
Batch_id:  1 Eval loss:  113021.6015625
Batch_id:  2 Eval loss:  187214.421875
Batch_id:  3 Eval loss:  69443.6875
Batch_id:  4 Eval loss:  51811.1640625
Batch_id:  5 Eval loss:  82933.2421875
Batch_id:  6 Eval loss:  29724.904296875
Epoch  107 Training loss:  173217.02755489864
Epoch  107 Evaluation loss:  108731.2338256836
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  87479.2421875


100%|██████████| 15/15 [00:00<00:00, 296.31it/s]

Batch_id:  1 Training loss:  378465.875
Batch_id:  2 Training loss:  333235.96875
Batch_id:  3 Training loss:  214084.671875
Batch_id:  4 Training loss:  59786.28125
Batch_id:  5 Training loss:  72197.7578125
Batch_id:  6 Training loss:  98148.8046875
Batch_id:  7 Training loss:  116132.015625
Batch_id:  8 Training loss:  76565.578125
Batch_id:  9 Training loss:  333224.8125
Batch_id:  10 Training loss:  227664.171875
Batch_id:  11 Training loss:  57958.33984375
Batch_id:  12 Training loss:  97497.453125
Batch_id:  13 Training loss:  118181.515625
Batch_id:  14 Training loss:  283068.5625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 733.86it/s]


Batch_id:  0 Eval loss:  206211.234375
Batch_id:  1 Eval loss:  136806.484375
Batch_id:  2 Eval loss:  43240.49609375
Batch_id:  3 Eval loss:  146966.90625
Batch_id:  4 Eval loss:  40654.55859375
Batch_id:  5 Eval loss:  67221.7734375
Batch_id:  6 Eval loss:  92361.140625
Epoch  108 Training loss:  172546.69262035473
Epoch  108 Evaluation loss:  114603.5302734375
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  149538.03125
Batch_id:  1 Training loss:  292548.0625
Batch_id:  2 Training loss:  88761.1796875
Batch_id:  3 Training loss:  135395.9375
Batch_id:  4 Training loss:  227992.703125
Batch_id:  5 Training loss:  141533.9375
Batch_id:  6 Training loss:  195154.140625
Batch_id:  7 Training loss:  69303.765625
Batch_id:  8 Training loss:  270341.75
Batch_id:  9 Training loss:  29057.375


100%|██████████| 15/15 [00:00<00:00, 293.89it/s]


Batch_id:  10 Training loss:  159934.234375
Batch_id:  11 Training loss:  217436.03125
Batch_id:  12 Training loss:  300458.3125
Batch_id:  13 Training loss:  64571.5703125
Batch_id:  14 Training loss:  193825.96875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1546.65it/s]


Batch_id:  0 Eval loss:  111079.515625
Batch_id:  1 Eval loss:  62878.98046875
Batch_id:  2 Eval loss:  147888.921875
Batch_id:  3 Eval loss:  81561.7578125
Batch_id:  4 Eval loss:  46499.1796875
Batch_id:  5 Eval loss:  225332.203125
Batch_id:  6 Eval loss:  6999.30224609375
Epoch  109 Training loss:  171341.4189189189
Epoch  109 Evaluation loss:  106599.97825622559
Best Model saved!
Best Loss value:  106732.29598999023
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  82501.875
Batch_id:  1 Training loss:  315460.65625
Batch_id:  2 Training loss:  183281.828125
Batch_id:  3 Training loss:  56666.71484375
Batch_id:  4 Training loss:  330468.96875
Batch_id:  5 Training loss:  213182.609375
Batch_id:  6 Training loss:  315163.5
Batch_id:  7 Training loss:  77643.578125
Batch_id:  8 Training loss:  235742.171875
Batch_id:  9 Training loss:  100129.1484375
Batch_id:  10 Training loss:  148058.75
Batch_id:  11 Training loss:  130782.578125
Batch_id:  12 Training loss:  55838.26171875
Batch_id:  13 Training loss:  207051.03125


100%|██████████| 15/15 [00:00<00:00, 364.73it/s]


Batch_id:  14 Training loss:  56291.88671875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 998.68it/s]

Batch_id:  0 Eval loss:  216229.125
Batch_id:  1 Eval loss:  134499.90625
Batch_id:  2 Eval loss:  72167.4765625
Batch_id:  3 Eval loss:  15113.8310546875
Batch_id:  4 Eval loss:  74972.2265625
Batch_id:  5 Eval loss:  128791.25
Batch_id:  6 Eval loss:  90676.875


Epoch  110 Training loss:  169477.26747255068
Epoch  110 Evaluation loss:  114445.42037963867
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 448.14it/s]

Batch_id:  0 Training loss:  47631.21484375
Batch_id:  1 Training loss:  273064.46875
Batch_id:  2 Training loss:  200115.984375
Batch_id:  3 Training loss:  55215.9375
Batch_id:  4 Training loss:  132304.921875
Batch_id:  5 Training loss:  105343.6796875
Batch_id:  6 Training loss:  68093.171875
Batch_id:  7 Training loss:  89387.5625
Batch_id:  8 Training loss:  256702.703125
Batch_id:  9 Training loss:  334596.5
Batch_id:  10 Training loss:  232029.609375
Batch_id:  11 Training loss:  244029.296875
Batch_id:  12 Training loss:  87105.5546875
Batch_id:  13 Training loss:  154343.328125
Batch_id:  14 Training loss:  271381.75


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1716.57it/s]


Batch_id:  0 Eval loss:  81928.734375
Batch_id:  1 Eval loss:  24403.818359375
Batch_id:  2 Eval loss:  307116.625
Batch_id:  3 Eval loss:  48551.55859375
Batch_id:  4 Eval loss:  130216.515625
Batch_id:  5 Eval loss:  70907.8125
Batch_id:  6 Eval loss:  37263.1015625
Epoch  111 Training loss:  172388.22186444254
Epoch  111 Evaluation loss:  109435.6509399414
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 445.54it/s]

Batch_id:  0 Training loss:  75776.6796875
Batch_id:  1 Training loss:  607798.4375
Batch_id:  2 Training loss:  184996.21875
Batch_id:  3 Training loss:  275943.5625
Batch_id:  4 Training loss:  35256.3203125
Batch_id:  5 Training loss:  94821.078125
Batch_id:  6 Training loss:  138745.890625
Batch_id:  7 Training loss:  194904.671875
Batch_id:  8 Training loss:  339980.15625
Batch_id:  9 Training loss:  128626.1796875
Batch_id:  10 Training loss:  37688.18359375
Batch_id:  11 Training loss:  167389.40625
Batch_id:  12 Training loss:  97194.203125
Batch_id:  13 Training loss:  73360.0625
Batch_id:  14 Training loss:  55877.71875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1385.37it/s]


Batch_id:  0 Eval loss:  70775.25
Batch_id:  1 Eval loss:  70903.171875
Batch_id:  2 Eval loss:  49713.76171875
Batch_id:  3 Eval loss:  228409.203125
Batch_id:  4 Eval loss:  27223.92578125
Batch_id:  5 Eval loss:  136437.28125
Batch_id:  6 Eval loss:  236461.59375
Epoch 00113: reducing learning rate of group 0 to 9.7656e-06.
Epoch  112 Training loss:  169483.70064400337
Epoch  112 Evaluation loss:  128113.154296875
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 416.02it/s]

Batch_id:  0 Training loss:  70980.484375
Batch_id:  1 Training loss:  226566.453125
Batch_id:  2 Training loss:  215462.546875
Batch_id:  3 Training loss:  76077.375
Batch_id:  4 Training loss:  220546.796875
Batch_id:  5 Training loss:  301142.8125
Batch_id:  6 Training loss:  71138.828125
Batch_id:  7 Training loss:  318004.34375
Batch_id:  8 Training loss:  76745.6171875
Batch_id:  9 Training loss:  283503.0625
Batch_id:  10 Training loss:  200246.609375
Batch_id:  11 Training loss:  26380.8125
Batch_id:  12 Training loss:  195707.203125
Batch_id:  13 Training loss:  83758.140625
Batch_id:  14 Training loss:  163418.171875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1753.58it/s]


Batch_id:  0 Eval loss:  263632.25
Batch_id:  1 Eval loss:  35582.875
Batch_id:  2 Eval loss:  76060.984375
Batch_id:  3 Eval loss:  59825.54296875
Batch_id:  4 Eval loss:  91776.453125
Batch_id:  5 Eval loss:  144603.859375
Batch_id:  6 Eval loss:  16416.37890625
Epoch  113 Training loss:  170924.27417652027
Epoch  113 Evaluation loss:  107484.1162109375
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 471.40it/s]

Batch_id:  0 Training loss:  96863.203125
Batch_id:  1 Training loss:  132288.640625
Batch_id:  2 Training loss:  240314.046875
Batch_id:  3 Training loss:  369113.34375
Batch_id:  4 Training loss:  222078.828125
Batch_id:  5 Training loss:  55635.5078125
Batch_id:  6 Training loss:  188719.671875
Batch_id:  7 Training loss:  227831.15625
Batch_id:  8 Training loss:  280192.21875
Batch_id:  9 Training loss:  82008.9609375
Batch_id:  10 Training loss:  182742.421875
Batch_id:  11 Training loss:  73808.640625
Batch_id:  12 Training loss:  77909.6015625
Batch_id:  13 Training loss:  43059.4375
Batch_id:  14 Training loss:  280473.40625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1746.38it/s]


Batch_id:  0 Eval loss:  142625.125
Batch_id:  1 Eval loss:  64300.125
Batch_id:  2 Eval loss:  61176.23046875
Batch_id:  3 Eval loss:  229966.796875
Batch_id:  4 Eval loss:  49073.4921875
Batch_id:  5 Eval loss:  123515.0859375
Batch_id:  6 Eval loss:  18466.91796875
Epoch  114 Training loss:  172502.64094172296
Epoch  114 Evaluation loss:  107675.58959960938
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 440.64it/s]

Batch_id:  0 Training loss:  62010.94921875
Batch_id:  1 Training loss:  357052.34375
Batch_id:  2 Training loss:  173953.359375
Batch_id:  3 Training loss:  168533.078125
Batch_id:  4 Training loss:  29010.94140625
Batch_id:  5 Training loss:  133345.90625
Batch_id:  6 Training loss:  149289.796875
Batch_id:  7 Training loss:  279827.0625
Batch_id:  8 Training loss:  215052.046875
Batch_id:  9 Training loss:  218931.859375
Batch_id:  10 Training loss:  239547.953125
Batch_id:  11 Training loss:  240425.734375
Batch_id:  12 Training loss:  161192.15625
Batch_id:  13 Training loss:  56085.75
Batch_id:  14 Training loss:  15859.27734375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1705.60it/s]


Batch_id:  0 Eval loss:  193808.828125
Batch_id:  1 Eval loss:  49448.10546875
Batch_id:  2 Eval loss:  195047.75
Batch_id:  3 Eval loss:  46529.64453125
Batch_id:  4 Eval loss:  65960.7109375
Batch_id:  5 Eval loss:  77511.5859375
Batch_id:  6 Eval loss:  124316.90625
Epoch  115 Training loss:  168926.90640836148
Epoch  115 Evaluation loss:  117597.4267578125
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  53635.52734375


100%|██████████| 15/15 [00:00<00:00, 407.19it/s]

Batch_id:  1 Training loss:  141805.796875
Batch_id:  2 Training loss:  120750.3671875
Batch_id:  3 Training loss:  101549.8515625
Batch_id:  4 Training loss:  114613.875
Batch_id:  5 Training loss:  60471.125
Batch_id:  6 Training loss:  295691.21875
Batch_id:  7 Training loss:  278024.0
Batch_id:  8 Training loss:  149368.890625
Batch_id:  9 Training loss:  206841.21875
Batch_id:  10 Training loss:  192268.984375
Batch_id:  11 Training loss:  481573.71875
Batch_id:  12 Training loss:  92729.7890625
Batch_id:  13 Training loss:  185840.515625
Batch_id:  14 Training loss:  27219.142578125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 953.99it/s]


Batch_id:  0 Eval loss:  130513.578125
Batch_id:  1 Eval loss:  85018.6171875
Batch_id:  2 Eval loss:  215982.828125
Batch_id:  3 Eval loss:  125036.8515625
Batch_id:  4 Eval loss:  49403.7265625
Batch_id:  5 Eval loss:  49199.46875
Batch_id:  6 Eval loss:  57184.390625
Epoch  116 Training loss:  169080.00145164694
Epoch  116 Evaluation loss:  111303.04077148438
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  398967.6875


100%|██████████| 15/15 [00:00<00:00, 227.56it/s]

Batch_id:  1 Training loss:  124953.2265625
Batch_id:  2 Training loss:  63757.625
Batch_id:  3 Training loss:  79830.875
Batch_id:  4 Training loss:  138167.796875
Batch_id:  5 Training loss:  162922.90625
Batch_id:  6 Training loss:  110670.5390625
Batch_id:  7 Training loss:  160962.421875
Batch_id:  8 Training loss:  221458.25
Batch_id:  9 Training loss:  92683.3515625
Batch_id:  10 Training loss:  192712.1875
Batch_id:  11 Training loss:  44963.37890625
Batch_id:  12 Training loss:  68559.3203125
Batch_id:  13 Training loss:  193493.921875
Batch_id:  14 Training loss:  553535.1875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 874.20it/s]


Batch_id:  0 Eval loss:  81358.578125
Batch_id:  1 Eval loss:  87224.5390625
Batch_id:  2 Eval loss:  26575.857421875
Batch_id:  3 Eval loss:  131120.484375
Batch_id:  4 Eval loss:  204504.109375
Batch_id:  5 Eval loss:  128775.8046875
Batch_id:  6 Eval loss:  46167.47265625
Epoch  117 Training loss:  176191.802417652
Epoch  117 Evaluation loss:  110269.81964111328
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  197892.265625
Batch_id:  1 Training loss:  65192.55859375
Batch_id:  2 Training loss:  332346.15625
Batch_id:  3 Training loss:  58382.96484375
Batch_id:  4 Training loss:  100317.6875
Batch_id:  5 Training loss:  91639.9375
Batch_id:  6 Training loss:  129508.2265625
Batch_id:  7 Training loss:  429655.75
Batch_id:  8 Training loss:  150016.109375
Batch_id:  9 Training loss:  81631.40625
Batch_id:  10 Training loss:  74553.8046875
Batch_id:  11 Training loss:  97740.4921875
Batch_id:  12 Training loss:  89689.5390625
Batch_id:  13 Training loss:  260890.734375


100%|██████████| 15/15 [00:00<00:00, 286.31it/s]


Batch_id:  14 Training loss:  421796.9375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1401.17it/s]


Batch_id:  0 Eval loss:  31397.125
Batch_id:  1 Eval loss:  56393.90234375
Batch_id:  2 Eval loss:  159329.65625
Batch_id:  3 Eval loss:  229840.703125
Batch_id:  4 Eval loss:  99754.0859375
Batch_id:  5 Eval loss:  76700.3671875
Batch_id:  6 Eval loss:  61530.1328125
Epoch  118 Training loss:  174409.09258868243
Epoch  118 Evaluation loss:  111710.30822753906
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  66854.484375


100%|██████████| 15/15 [00:00<00:00, 414.47it/s]

Batch_id:  1 Training loss:  37761.44140625
Batch_id:  2 Training loss:  158361.28125
Batch_id:  3 Training loss:  87213.7734375
Batch_id:  4 Training loss:  292191.5625
Batch_id:  5 Training loss:  377398.96875
Batch_id:  6 Training loss:  240285.046875
Batch_id:  7 Training loss:  295536.96875
Batch_id:  8 Training loss:  226346.828125
Batch_id:  9 Training loss:  52402.3359375
Batch_id:  10 Training loss:  72099.3671875
Batch_id:  11 Training loss:  259537.25
Batch_id:  12 Training loss:  169303.171875
Batch_id:  13 Training loss:  80461.796875
Batch_id:  14 Training loss:  101626.9921875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1725.54it/s]


Batch_id:  0 Eval loss:  129998.8515625
Batch_id:  1 Eval loss:  35496.29296875
Batch_id:  2 Eval loss:  35589.0390625
Batch_id:  3 Eval loss:  73764.8984375
Batch_id:  4 Eval loss:  350685.5625
Batch_id:  5 Eval loss:  48809.9921875
Batch_id:  6 Eval loss:  9225.421875
Epoch  119 Training loss:  170093.32902238175
Epoch  119 Evaluation loss:  106807.82165527344
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  140351.28125
Batch_id:  1 Training loss:  38745.54296875
Batch_id:  2 Training loss:  171596.640625
Batch_id:  3 Training loss:  99160.2890625
Batch_id:  4 Training loss:  179442.75
Batch_id:  5 Training loss:  335393.71875
Batch_id:  6 Training loss:  187831.015625


100%|██████████| 15/15 [00:00<00:00, 385.19it/s]


Batch_id:  7 Training loss:  119119.2265625
Batch_id:  8 Training loss:  50411.62109375
Batch_id:  9 Training loss:  171193.921875
Batch_id:  10 Training loss:  101919.8359375
Batch_id:  11 Training loss:  84360.921875
Batch_id:  12 Training loss:  338108.59375
Batch_id:  13 Training loss:  244359.421875
Batch_id:  14 Training loss:  293625.28125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1715.16it/s]


Batch_id:  0 Eval loss:  344492.90625
Batch_id:  1 Eval loss:  114671.1171875
Batch_id:  2 Eval loss:  57561.29296875
Batch_id:  3 Eval loss:  72256.7109375
Batch_id:  4 Eval loss:  21948.46875
Batch_id:  5 Eval loss:  64401.19921875
Batch_id:  6 Eval loss:  6741.265625
Epoch  120 Training loss:  172677.03125
Epoch  120 Evaluation loss:  106573.90014648438
Best Model saved!
Best Loss value:  106599.97825622559
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  332412.96875
Batch_id:  1 Training loss:  139027.75
Batch_id:  2 Training loss:  115970.625
Batch_id:  3 Training loss:  209517.296875
Batch_id:  4 Training loss:  139380.609375
Batch_id:  5 Training loss:  93039.9140625
Batch_id:  6 Training loss:  55313.11328125
Batch_id:  7 Training loss:  299657.40625
Batch_id:  8 Training loss:  47395.43359375
Batch_id:  9 Training loss:  248637.359375
Batch_id:  10 Training loss:  87290.1875
Batch_id:  11 Training loss:  252988.953125
Batch_id:  12 Training loss:  255794.296875
Batch_id:  13 Training loss:  182871.734375


100%|██████████| 15/15 [00:00<00:00, 425.00it/s]


Batch_id:  14 Training loss:  47028.34375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1718.17it/s]


Batch_id:  0 Eval loss:  46518.60546875
Batch_id:  1 Eval loss:  66922.6484375
Batch_id:  2 Eval loss:  155634.359375
Batch_id:  3 Eval loss:  236747.859375
Batch_id:  4 Eval loss:  67707.8125
Batch_id:  5 Eval loss:  97185.8984375
Batch_id:  6 Eval loss:  18258.056640625
Epoch  121 Training loss:  169346.35082347973
Epoch  121 Evaluation loss:  107652.3812866211
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 444.68it/s]


Batch_id:  0 Training loss:  256753.828125
Batch_id:  1 Training loss:  58570.6640625
Batch_id:  2 Training loss:  143144.421875
Batch_id:  3 Training loss:  232546.296875
Batch_id:  4 Training loss:  60959.5390625
Batch_id:  5 Training loss:  86759.4296875
Batch_id:  6 Training loss:  289929.28125
Batch_id:  7 Training loss:  202575.953125
Batch_id:  8 Training loss:  363957.5625
Batch_id:  9 Training loss:  144471.03125
Batch_id:  10 Training loss:  71510.4296875
Batch_id:  11 Training loss:  195550.390625
Batch_id:  12 Training loss:  106544.0625
Batch_id:  13 Training loss:  164309.328125
Batch_id:  14 Training loss:  149167.59375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1393.39it/s]

Batch_id:  0 Eval loss:  57978.19921875
Batch_id:  1 Eval loss:  95550.421875
Batch_id:  2 Eval loss:  33972.4375
Batch_id:  3 Eval loss:  108401.6640625
Batch_id:  4 Eval loss:  328119.59375
Batch_id:  5 Eval loss:  43496.04296875
Batch_id:  6 Eval loss:  26262.810546875


Epoch  122 Training loss:  170726.33868243243
Epoch  122 Evaluation loss:  108403.30780029297
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 442.05it/s]

Batch_id:  0 Training loss:  80364.5390625
Batch_id:  1 Training loss:  122130.1015625
Batch_id:  2 Training loss:  62317.8828125
Batch_id:  3 Training loss:  141112.859375
Batch_id:  4 Training loss:  194028.0
Batch_id:  5 Training loss:  89545.2421875
Batch_id:  6 Training loss:  258538.03125
Batch_id:  7 Training loss:  221288.671875
Batch_id:  8 Training loss:  90153.296875
Batch_id:  9 Training loss:  195496.75
Batch_id:  10 Training loss:  275130.9375
Batch_id:  11 Training loss:  271026.71875
Batch_id:  12 Training loss:  158818.40625
Batch_id:  13 Training loss:  243477.828125
Batch_id:  14 Training loss:  116788.375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1745.86it/s]


Batch_id:  0 Eval loss:  39288.75
Batch_id:  1 Eval loss:  79114.4375
Batch_id:  2 Eval loss:  116695.3359375
Batch_id:  3 Eval loss:  23775.125
Batch_id:  4 Eval loss:  220884.484375
Batch_id:  5 Eval loss:  164331.171875
Batch_id:  6 Eval loss:  84829.4296875
Epoch 00124: reducing learning rate of group 0 to 4.8828e-06.
Epoch  123 Training loss:  170284.97571790538
Epoch  123 Evaluation loss:  113893.55224609375
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  423202.21875


100%|██████████| 15/15 [00:00<00:00, 409.55it/s]

Batch_id:  1 Training loss:  57768.73828125
Batch_id:  2 Training loss:  215525.65625
Batch_id:  3 Training loss:  53002.96484375
Batch_id:  4 Training loss:  384838.96875
Batch_id:  5 Training loss:  177798.390625
Batch_id:  6 Training loss:  123111.25
Batch_id:  7 Training loss:  187158.421875
Batch_id:  8 Training loss:  83899.703125
Batch_id:  9 Training loss:  183153.71875
Batch_id:  10 Training loss:  99125.8046875
Batch_id:  11 Training loss:  72509.8359375
Batch_id:  12 Training loss:  171161.671875
Batch_id:  13 Training loss:  27834.572265625
Batch_id:  14 Training loss:  295988.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1396.57it/s]


Batch_id:  0 Eval loss:  47310.71484375
Batch_id:  1 Eval loss:  329685.65625
Batch_id:  2 Eval loss:  121243.1796875
Batch_id:  3 Eval loss:  54896.71484375
Batch_id:  4 Eval loss:  63771.40234375
Batch_id:  5 Eval loss:  56264.01171875
Batch_id:  6 Eval loss:  12124.4384765625
Epoch  124 Training loss:  172708.16155510978
Epoch  124 Evaluation loss:  107077.51846313477
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  96918.6484375
Batch_id:  1 Training loss:  49653.4921875
Batch_id:  2 Training loss:  89429.265625
Batch_id:  3 Training loss:  127370.5546875
Batch_id:  4 Training loss:  133598.421875
Batch_id:  5 Training loss:  303573.53125
Batch_id:  6 Training loss:  141673.28125
Batch_id:  7 Training loss:  341296.21875
Batch_id:  8 Training loss:  122288.6640625
Batch_id:  9 Training loss:  96890.0859375
Batch_id:  10 Training loss:  302783.5625
Batch_id:  11 Training loss:  106203.5234375
Batch_id:  12 Training loss:  133827.109375


100%|██████████| 15/15 [00:00<00:00, 291.90it/s]

Batch_id:  13 Training loss:  346156.25
Batch_id:  14 Training loss:  131561.6875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 869.62it/s]


Batch_id:  0 Eval loss:  70771.6171875
Batch_id:  1 Eval loss:  47313.43359375
Batch_id:  2 Eval loss:  102549.9453125
Batch_id:  3 Eval loss:  42520.3203125
Batch_id:  4 Eval loss:  309379.59375
Batch_id:  5 Eval loss:  61516.65625
Batch_id:  6 Eval loss:  109908.6171875
Epoch  125 Training loss:  170488.12816722973
Epoch  125 Evaluation loss:  116243.77868652344
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  94232.4765625
Batch_id:  1 Training loss:  225700.65625
Batch_id:  2 Training loss:  38127.26953125
Batch_id:  3 Training loss:  63933.875
Batch_id:  4 Training loss:  477655.21875
Batch_id:  5 Training loss:  43620.75
Batch_id:  6 Training loss:  31954.166015625
Batch_id:  7 Training loss:  124067.1484375
Batch_id:  8 Training loss:  170110.390625
Batch_id:  9 Training loss:  294719.96875
Batch_id:  10 Training loss:  305306.71875
Batch_id:  11 Training loss:  67747.9609375
Batch_id:  12 Training loss:  268116.71875


100%|██████████| 15/15 [00:00<00:00, 303.41it/s]


Batch_id:  13 Training loss:  224164.734375
Batch_id:  14 Training loss:  84230.4375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1162.55it/s]


Batch_id:  0 Eval loss:  280640.34375
Batch_id:  1 Eval loss:  44668.1953125
Batch_id:  2 Eval loss:  19742.767578125
Batch_id:  3 Eval loss:  29006.78515625
Batch_id:  4 Eval loss:  52730.4765625
Batch_id:  5 Eval loss:  112703.8515625
Batch_id:  6 Eval loss:  346302.8125
Epoch  126 Training loss:  169843.81690772803
Epoch  126 Evaluation loss:  138405.50506591797
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  279533.9375
Batch_id:  1 Training loss:  329158.3125
Batch_id:  2 Training loss:  182966.875
Batch_id:  3 Training loss:  99898.7109375
Batch_id:  4 Training loss:  122080.7734375
Batch_id:  5 Training loss:  101597.875
Batch_id:  6 Training loss:  149227.375
Batch_id:  7 Training loss:  61540.4140625
Batch_id:  8 Training loss:  263196.875
Batch_id:  9 Training loss:  205045.921875
Batch_id:  10 Training loss:  169501.75


100%|██████████| 15/15 [00:00<00:00, 273.65it/s]

Batch_id:  11 Training loss:  226339.546875
Batch_id:  12 Training loss:  159426.53125
Batch_id:  13 Training loss:  27657.53515625
Batch_id:  14 Training loss:  149595.84375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 883.62it/s]

Batch_id:  0 Eval loss:  48346.1015625
Batch_id:  1 Eval loss:  204283.890625
Batch_id:  2 Eval loss:  48445.125
Batch_id:  3 Eval loss:  38381.3828125
Batch_id:  4 Eval loss:  251721.578125
Batch_id:  5 Eval loss:  71263.65625
Batch_id:  6 Eval loss:  38937.0078125


Epoch  127 Training loss:  170727.58630701012
Epoch  127 Evaluation loss:  109590.42846679688
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 353.82it/s]

Batch_id:  0 Training loss:  36625.79296875
Batch_id:  1 Training loss:  60159.61328125
Batch_id:  2 Training loss:  617124.4375
Batch_id:  3 Training loss:  225421.921875
Batch_id:  4 Training loss:  254081.75
Batch_id:  5 Training loss:  84194.140625
Batch_id:  6 Training loss:  68742.265625
Batch_id:  7 Training loss:  96149.1796875
Batch_id:  8 Training loss:  94113.78125
Batch_id:  9 Training loss:  64943.82421875
Batch_id:  10 Training loss:  131589.875
Batch_id:  11 Training loss:  197141.40625
Batch_id:  12 Training loss:  201527.53125
Batch_id:  13 Training loss:  296233.125
Batch_id:  14 Training loss:  86040.8671875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 2330.35it/s]


Batch_id:  0 Eval loss:  33661.57421875
Batch_id:  1 Eval loss:  56184.69921875
Batch_id:  2 Eval loss:  73871.3203125
Batch_id:  3 Eval loss:  66890.7421875
Batch_id:  4 Eval loss:  38713.35546875
Batch_id:  5 Eval loss:  362392.09375
Batch_id:  6 Eval loss:  115755.5
Epoch  128 Training loss:  169870.9129539696
Epoch  128 Evaluation loss:  116792.07580566406
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  131907.671875
Batch_id:  1 Training loss:  70948.765625
Batch_id:  2 Training loss:  113945.1171875
Batch_id:  3 Training loss:  97202.828125
Batch_id:  4 Training loss:  68499.3046875
Batch_id:  5 Training loss:  310142.4375
Batch_id:  6 Training loss:  82527.1796875
Batch_id:  7 Training loss:  189613.78125
Batch_id:  8 Training loss:  356587.96875
Batch_id:  9 Training loss:  105787.5234375
Batch_id:  10 Training loss:  320368.28125


100%|██████████| 15/15 [00:00<00:00, 310.08it/s]

Batch_id:  11 Training loss:  80006.5234375
Batch_id:  12 Training loss:  34190.1640625
Batch_id:  13 Training loss:  360666.46875
Batch_id:  14 Training loss:  218070.5625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1396.77it/s]


Batch_id:  0 Eval loss:  74974.84375
Batch_id:  1 Eval loss:  56296.76171875
Batch_id:  2 Eval loss:  92419.5
Batch_id:  3 Eval loss:  194971.15625
Batch_id:  4 Eval loss:  130911.1796875
Batch_id:  5 Eval loss:  108577.0
Batch_id:  6 Eval loss:  49652.5078125
Epoch  129 Training loss:  171653.01203547296
Epoch  129 Evaluation loss:  110594.21081542969
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  310544.5625
Batch_id:  1 Training loss:  328176.5
Batch_id:  2 Training loss:  118022.828125
Batch_id:  3 Training loss:  148117.875
Batch_id:  4 Training loss:  141331.65625
Batch_id:  5 Training loss:  121827.8046875
Batch_id:  6 Training loss:  35901.14453125
Batch_id:  7 Training loss:  175885.34375
Batch_id:  8 Training loss:  290993.8125
Batch_id:  9 Training loss:  270826.0
Batch_id:  10 Training loss:  121591.9765625
Batch_id:  11 Training loss:  30860.396484375
Batch_id:  12 Training loss:  257634.796875
Batch_id:  13 Training loss:  86877.8984375


100%|██████████| 15/15 [00:00<00:00, 365.86it/s]


Batch_id:  14 Training loss:  72907.8046875
Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  36890.53125
Batch_id:  1 Eval loss:  124059.5625
Batch_id:  2 Eval loss:  59685.2578125
Batch_id:  3 Eval loss:  78772.2578125
Batch_id:  4 Eval loss:  226398.953125
Batch_id:  5 Eval loss:  141159.984375
Batch_id:  6 Eval loss:  27623.966796875


100%|██████████| 7/7 [00:00<00:00, 1397.10it/s]


Epoch  130 Training loss:  169695.97299936655
Epoch  130 Evaluation loss:  108529.76776123047
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  215073.5
Batch_id:  1 Training loss:  44942.375
Batch_id:  2 Training loss:  133618.171875
Batch_id:  3 Training loss:  276069.625
Batch_id:  4 Training loss:  291155.46875
Batch_id:  5 Training loss:  192782.828125
Batch_id:  6 Training loss:  151134.09375
Batch_id:  7 Training loss:  38841.12890625
Batch_id:  8 Training loss:  69328.7265625
Batch_id:  9 Training loss:  103732.515625
Batch_id:  10 Training loss:  280365.78125
Batch_id:  11 Training loss:  161237.359375


100%|██████████| 15/15 [00:00<00:00, 347.35it/s]


Batch_id:  12 Training loss:  390468.53125
Batch_id:  13 Training loss:  79677.7265625
Batch_id:  14 Training loss:  85485.578125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1373.06it/s]


Batch_id:  0 Eval loss:  69980.5234375
Batch_id:  1 Eval loss:  43138.68359375
Batch_id:  2 Eval loss:  88858.7109375
Batch_id:  3 Eval loss:  108029.4140625
Batch_id:  4 Eval loss:  276935.46875
Batch_id:  5 Eval loss:  65040.71484375
Batch_id:  6 Eval loss:  65070.03515625
Epoch  131 Training loss:  169859.01419974663
Epoch  131 Evaluation loss:  112039.61730957031
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  209702.53125


100%|██████████| 15/15 [00:00<00:00, 404.27it/s]

Batch_id:  1 Training loss:  290014.90625
Batch_id:  2 Training loss:  167744.546875
Batch_id:  3 Training loss:  142388.28125
Batch_id:  4 Training loss:  113699.5859375
Batch_id:  5 Training loss:  241189.703125
Batch_id:  6 Training loss:  214032.703125
Batch_id:  7 Training loss:  187675.0625
Batch_id:  8 Training loss:  128953.6796875
Batch_id:  9 Training loss:  65418.69921875
Batch_id:  10 Training loss:  172535.25
Batch_id:  11 Training loss:  78207.9609375
Batch_id:  12 Training loss:  39685.4375
Batch_id:  13 Training loss:  82247.8359375
Batch_id:  14 Training loss:  454187.5625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1708.37it/s]


Batch_id:  0 Eval loss:  36141.28515625
Batch_id:  1 Eval loss:  72536.9765625
Batch_id:  2 Eval loss:  227206.328125
Batch_id:  3 Eval loss:  50857.82421875
Batch_id:  4 Eval loss:  120033.5
Batch_id:  5 Eval loss:  135155.8125
Batch_id:  6 Eval loss:  90197.8203125
Epoch  132 Training loss:  174843.4963576858
Epoch  132 Evaluation loss:  114395.24169921875
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  117984.4296875
Batch_id:  1 Training loss:  41892.3125
Batch_id:  2 Training loss:  103381.859375
Batch_id:  3 Training loss:  428930.0625
Batch_id:  4 Training loss:  91881.1796875
Batch_id:  5 Training loss:  171726.546875
Batch_id:  6 Training loss:  108738.5859375
Batch_id:  7 Training loss:  185285.859375
Batch_id:  8 Training loss:  93698.9765625
Batch_id:  9 Training loss:  159320.078125
Batch_id:  10 Training loss:  178846.84375
Batch_id:  11 Training loss:  234258.609375
Batch_id:  12 Training loss:  369212.96875
Batch_id:  13 Training loss:  163348.859375


100%|██████████| 15/15 [00:00<00:00, 315.28it/s]


Batch_id:  14 Training loss:  60390.03515625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 537.63it/s]


Batch_id:  0 Eval loss:  75167.25
Batch_id:  1 Eval loss:  120933.7421875
Batch_id:  2 Eval loss:  89162.3671875
Batch_id:  3 Eval loss:  206684.8125
Batch_id:  4 Eval loss:  129110.828125
Batch_id:  5 Eval loss:  41835.4609375
Batch_id:  6 Eval loss:  37782.6171875
Epoch  133 Training loss:  169520.08155616553
Epoch  133 Evaluation loss:  109480.79345703125
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  174665.953125
Batch_id:  1 Training loss:  220210.28125
Batch_id:  2 Training loss:  63189.65625
Batch_id:  3 Training loss:  99460.7109375
Batch_id:  4 Training loss:  180212.046875
Batch_id:  5 Training loss:  47856.37109375
Batch_id:  6 Training loss:  86121.6875
Batch_id:  7 Training loss:  11373.5703125
Batch_id:  8 Training loss:  158272.828125
Batch_id:  9 Training loss:  343678.15625
Batch_id:  10 Training loss:  193974.328125
Batch_id:  11 Training loss:  324564.09375
Batch_id:  12 Training loss:  344503.8125


100%|██████████| 15/15 [00:00<00:00, 312.28it/s]


Batch_id:  13 Training loss:  143626.671875
Batch_id:  14 Training loss:  131452.90625
Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  311262.46875


100%|██████████| 7/7 [00:00<00:00, 684.00it/s]


Batch_id:  1 Eval loss:  27171.654296875
Batch_id:  2 Eval loss:  110979.5859375
Batch_id:  3 Eval loss:  65419.07421875
Batch_id:  4 Eval loss:  65718.9375
Batch_id:  5 Eval loss:  77164.1171875
Batch_id:  6 Eval loss:  50714.40234375
Epoch 00135: reducing learning rate of group 0 to 2.4414e-06.
Epoch  134 Training loss:  170483.9915012669
Epoch  134 Evaluation loss:  110692.2250366211
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  256324.703125
Batch_id:  1 Training loss:  284809.375
Batch_id:  2 Training loss:  48685.8046875
Batch_id:  3 Training loss:  289027.71875
Batch_id:  4 Training loss:  168631.203125
Batch_id:  5 Training loss:  174242.625
Batch_id:  6 Training loss:  183759.921875
Batch_id:  7 Training loss:  58337.79296875
Batch_id:  8 Training loss:  44027.65234375
Batch_id:  9 Training loss:  56452.83203125
Batch_id:  10 Training loss:  168657.671875
Batch_id:  11 Training loss:  165977.796875


100%|██████████| 15/15 [00:00<00:00, 287.62it/s]

Batch_id:  12 Training loss:  396883.34375
Batch_id:  13 Training loss:  77206.8671875
Batch_id:  14 Training loss:  154750.90625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 700.02it/s]


Batch_id:  0 Eval loss:  216272.109375
Batch_id:  1 Eval loss:  113307.8359375
Batch_id:  2 Eval loss:  77842.96875
Batch_id:  3 Eval loss:  40831.8515625
Batch_id:  4 Eval loss:  201583.3125
Batch_id:  5 Eval loss:  20738.189453125
Batch_id:  6 Eval loss:  18566.572265625
Epoch  135 Training loss:  170795.69019214527
Epoch  135 Evaluation loss:  107678.56872558594
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 396.83it/s]

Batch_id:  0 Training loss:  79162.1015625
Batch_id:  1 Training loss:  197422.109375
Batch_id:  2 Training loss:  242669.859375
Batch_id:  3 Training loss:  279118.15625
Batch_id:  4 Training loss:  163484.71875
Batch_id:  5 Training loss:  135153.671875
Batch_id:  6 Training loss:  181100.5
Batch_id:  7 Training loss:  43357.29296875
Batch_id:  8 Training loss:  149448.578125
Batch_id:  9 Training loss:  103534.640625
Batch_id:  10 Training loss:  305013.5
Batch_id:  11 Training loss:  83414.671875
Batch_id:  12 Training loss:  97635.1484375
Batch_id:  13 Training loss:  319428.96875
Batch_id:  14 Training loss:  146069.734375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1394.05it/s]


Batch_id:  0 Eval loss:  236507.609375
Batch_id:  1 Eval loss:  53050.2890625
Batch_id:  2 Eval loss:  103228.7890625
Batch_id:  3 Eval loss:  63885.75
Batch_id:  4 Eval loss:  71737.03125
Batch_id:  5 Eval loss:  140248.359375
Batch_id:  6 Eval loss:  23361.189453125
Epoch  136 Training loss:  170676.59813133444
Epoch  136 Evaluation loss:  108127.97149658203
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  107175.8359375
Batch_id:  1 Training loss:  120361.1796875
Batch_id:  2 Training loss:  209713.359375
Batch_id:  3 Training loss:  179681.71875
Batch_id:  4 Training loss:  268120.84375
Batch_id:  5 Training loss:  320101.21875
Batch_id:  6 Training loss:  307337.8125
Batch_id:  7 Training loss:  84997.640625
Batch_id:  8 Training loss:  77414.5390625
Batch_id:  9 Training loss:  233728.25
Batch_id:  10 Training loss:  141548.109375
Batch_id:  11 Training loss:  134584.78125
Batch_id:  12 Training loss:  35228.16796875
Batch_id:  13 Training loss:  179435.578125


100%|██████████| 15/15 [00:00<00:00, 382.69it/s]


Batch_id:  14 Training loss:  121706.1015625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1409.38it/s]

Batch_id:  0 Eval loss:  70151.9765625
Batch_id:  1 Eval loss:  82581.5390625
Batch_id:  2 Eval loss:  41812.5390625
Batch_id:  3 Eval loss:  125935.9765625
Batch_id:  4 Eval loss:  147939.609375
Batch_id:  5 Eval loss:  199132.546875
Batch_id:  6 Eval loss:  26118.716796875


Epoch  137 Training loss:  170346.96869721284
Epoch  137 Evaluation loss:  108386.39129638672
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  55781.5078125
Batch_id:  1 Training loss:  143231.953125
Batch_id:  2 Training loss:  59689.9375
Batch_id:  3 Training loss:  239837.03125
Batch_id:  4 Training loss:  73455.9296875
Batch_id:  5 Training loss:  125043.375
Batch_id:  6 Training loss:  181878.828125
Batch_id:  7 Training loss:  105637.5625
Batch_id:  8 Training loss:  275535.15625
Batch_id:  9 Training loss:  189494.203125
Batch_id:  10 Training loss:  291207.96875
Batch_id:  11 Training loss:  304155.4375
Batch_id:  12 Training loss:  66454.0859375


100%|██████████| 15/15 [00:00<00:00, 372.72it/s]


Batch_id:  13 Training loss:  215799.40625
Batch_id:  14 Training loss:  211990.3125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1390.22it/s]


Batch_id:  0 Eval loss:  59575.1640625
Batch_id:  1 Eval loss:  147342.25
Batch_id:  2 Eval loss:  87623.515625
Batch_id:  3 Eval loss:  42446.9453125
Batch_id:  4 Eval loss:  195117.703125
Batch_id:  5 Eval loss:  140333.703125
Batch_id:  6 Eval loss:  13905.4443359375
Epoch  138 Training loss:  171567.07400760133
Epoch  138 Evaluation loss:  107241.36337280273
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  53599.55859375
Batch_id:  1 Training loss:  323859.59375
Batch_id:  2 Training loss:  166771.875
Batch_id:  3 Training loss:  131870.671875
Batch_id:  4 Training loss:  174691.5
Batch_id:  5 Training loss:  182252.984375
Batch_id:  6 Training loss:  140302.78125
Batch_id:  7 Training loss:  138989.859375
Batch_id:  8 Training loss:  60473.01953125
Batch_id:  9 Training loss:  324226.875
Batch_id:  10 Training loss:  74873.390625
Batch_id:  11 Training loss:  382195.09375


100%|██████████| 15/15 [00:00<00:00, 365.79it/s]


Batch_id:  12 Training loss:  86659.9921875
Batch_id:  13 Training loss:  170641.734375
Batch_id:  14 Training loss:  106747.5390625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1381.59it/s]


Batch_id:  0 Eval loss:  248864.0
Batch_id:  1 Eval loss:  33497.27734375
Batch_id:  2 Eval loss:  67454.5078125
Batch_id:  3 Eval loss:  128523.0234375
Batch_id:  4 Eval loss:  33773.85546875
Batch_id:  5 Eval loss:  121654.875
Batch_id:  6 Eval loss:  110579.2421875
Epoch  139 Training loss:  170145.70734797296
Epoch  139 Evaluation loss:  116304.1845703125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 307.42it/s]

Batch_id:  0 Training loss:  104412.1484375
Batch_id:  1 Training loss:  248426.328125
Batch_id:  2 Training loss:  182922.78125
Batch_id:  3 Training loss:  65533.625
Batch_id:  4 Training loss:  67359.3359375
Batch_id:  5 Training loss:  122660.9609375
Batch_id:  6 Training loss:  54452.05859375
Batch_id:  7 Training loss:  227281.046875
Batch_id:  8 Training loss:  96195.3828125
Batch_id:  9 Training loss:  107938.1796875
Batch_id:  10 Training loss:  364049.46875
Batch_id:  11 Training loss:  280975.25
Batch_id:  12 Training loss:  84499.265625
Batch_id:  13 Training loss:  434981.40625
Batch_id:  14 Training loss:  68888.65625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 586.64it/s]


Batch_id:  0 Eval loss:  315991.34375
Batch_id:  1 Eval loss:  42591.38671875
Batch_id:  2 Eval loss:  116023.1171875
Batch_id:  3 Eval loss:  76634.546875
Batch_id:  4 Eval loss:  57155.01171875
Batch_id:  5 Eval loss:  21451.46875
Batch_id:  6 Eval loss:  120380.171875
Epoch  140 Training loss:  169633.50638724663
Epoch  140 Evaluation loss:  117222.97607421875
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  79965.078125
Batch_id:  1 Training loss:  255441.859375
Batch_id:  2 Training loss:  134416.71875
Batch_id:  3 Training loss:  57049.80078125
Batch_id:  4 Training loss:  386448.59375
Batch_id:  5 Training loss:  143430.703125
Batch_id:  6 Training loss:  171229.046875
Batch_id:  7 Training loss:  353853.09375
Batch_id:  8 Training loss:  180651.390625
Batch_id:  9 Training loss:  46024.02734375
Batch_id:  10 Training loss:  297480.0
Batch_id:  11 Training loss:  35418.3984375


100%|██████████| 15/15 [00:00<00:00, 292.24it/s]


Batch_id:  12 Training loss:  189562.078125
Batch_id:  13 Training loss:  69103.78125
Batch_id:  14 Training loss:  120897.09375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1708.37it/s]


Batch_id:  0 Eval loss:  71469.3359375
Batch_id:  1 Eval loss:  44235.58984375
Batch_id:  2 Eval loss:  70818.9609375
Batch_id:  3 Eval loss:  228657.125
Batch_id:  4 Eval loss:  151215.796875
Batch_id:  5 Eval loss:  109970.140625
Batch_id:  6 Eval loss:  4082.722900390625
Epoch  141 Training loss:  170335.9232474662
Epoch  141 Evaluation loss:  106320.26126861572
Best Model saved!
Best Loss value:  106573.90014648438
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 367.65it/s]

Batch_id:  0 Training loss:  116992.078125
Batch_id:  1 Training loss:  247988.796875
Batch_id:  2 Training loss:  163870.78125
Batch_id:  3 Training loss:  289656.09375
Batch_id:  4 Training loss:  405370.5
Batch_id:  5 Training loss:  181596.53125
Batch_id:  6 Training loss:  96425.375
Batch_id:  7 Training loss:  251804.453125
Batch_id:  8 Training loss:  178094.671875
Batch_id:  9 Training loss:  62765.75
Batch_id:  10 Training loss:  172015.203125
Batch_id:  11 Training loss:  123140.3984375
Batch_id:  12 Training loss:  52696.46484375
Batch_id:  13 Training loss:  34379.86328125
Batch_id:  14 Training loss:  150002.90625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1405.40it/s]


Batch_id:  0 Eval loss:  192204.546875
Batch_id:  1 Eval loss:  37334.83203125
Batch_id:  2 Eval loss:  117748.1171875
Batch_id:  3 Eval loss:  72024.3359375
Batch_id:  4 Eval loss:  55311.64453125
Batch_id:  5 Eval loss:  52659.15234375
Batch_id:  6 Eval loss:  376793.84375
Epoch  142 Training loss:  170729.72075591216
Epoch  142 Evaluation loss:  141261.94885253906
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  184643.046875


100%|██████████| 15/15 [00:00<00:00, 338.67it/s]


Batch_id:  1 Training loss:  156854.796875
Batch_id:  2 Training loss:  289430.34375
Batch_id:  3 Training loss:  265225.53125
Batch_id:  4 Training loss:  70530.5625
Batch_id:  5 Training loss:  58262.6875
Batch_id:  6 Training loss:  180307.796875
Batch_id:  7 Training loss:  137301.921875
Batch_id:  8 Training loss:  70597.4921875
Batch_id:  9 Training loss:  111440.9609375
Batch_id:  10 Training loss:  278123.34375
Batch_id:  11 Training loss:  246440.859375
Batch_id:  12 Training loss:  150965.296875
Batch_id:  13 Training loss:  226256.703125
Batch_id:  14 Training loss:  88015.046875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1372.48it/s]

Batch_id:  0 Eval loss:  80957.9609375
Batch_id:  1 Eval loss:  82021.625
Batch_id:  2 Eval loss:  55377.25
Batch_id:  3 Eval loss:  218387.296875
Batch_id:  4 Eval loss:  28338.75390625
Batch_id:  5 Eval loss:  176826.921875
Batch_id:  6 Eval loss:  90218.734375


Epoch  143 Training loss:  169891.6480152027
Epoch  143 Evaluation loss:  114395.08483886719
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  141918.9375
Batch_id:  1 Training loss:  315058.84375
Batch_id:  2 Training loss:  413471.90625
Batch_id:  3 Training loss:  53477.91796875
Batch_id:  4 Training loss:  121908.3984375
Batch_id:  5 Training loss:  40780.95703125
Batch_id:  6 Training loss:  89537.7109375
Batch_id:  7 Training loss:  92757.140625
Batch_id:  8 Training loss:  109627.5234375
Batch_id:  9 Training loss:  102423.4140625


100%|██████████| 15/15 [00:00<00:00, 312.61it/s]


Batch_id:  10 Training loss:  286539.40625
Batch_id:  11 Training loss:  366968.53125
Batch_id:  12 Training loss:  236650.796875
Batch_id:  13 Training loss:  89647.0546875
Batch_id:  14 Training loss:  45031.3203125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1163.01it/s]


Batch_id:  0 Eval loss:  20775.396484375
Batch_id:  1 Eval loss:  232549.40625
Batch_id:  2 Eval loss:  149385.125
Batch_id:  3 Eval loss:  105143.2109375
Batch_id:  4 Eval loss:  65685.3515625
Batch_id:  5 Eval loss:  22720.888671875
Batch_id:  6 Eval loss:  204341.75
Epoch  144 Training loss:  169310.80130912163
Epoch  144 Evaluation loss:  125093.92639160156
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  91604.765625


100%|██████████| 15/15 [00:00<00:00, 325.91it/s]

Batch_id:  1 Training loss:  78567.9921875
Batch_id:  2 Training loss:  72682.7109375
Batch_id:  3 Training loss:  88814.8984375
Batch_id:  4 Training loss:  159726.125
Batch_id:  5 Training loss:  148359.625
Batch_id:  6 Training loss:  232176.984375
Batch_id:  7 Training loss:  127489.625
Batch_id:  8 Training loss:  87433.390625
Batch_id:  9 Training loss:  223226.921875
Batch_id:  10 Training loss:  315037.59375
Batch_id:  11 Training loss:  136312.734375
Batch_id:  12 Training loss:  162177.453125
Batch_id:  13 Training loss:  157090.609375
Batch_id:  14 Training loss:  520134.875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1403.05it/s]


Batch_id:  0 Eval loss:  81512.9765625
Batch_id:  1 Eval loss:  303621.59375
Batch_id:  2 Eval loss:  122920.328125
Batch_id:  3 Eval loss:  60323.48046875
Batch_id:  4 Eval loss:  27404.232421875
Batch_id:  5 Eval loss:  42174.73828125
Batch_id:  6 Eval loss:  100090.875
Epoch 00146: reducing learning rate of group 0 to 1.2207e-06.
Epoch  145 Training loss:  175732.18274915538
Epoch  145 Evaluation loss:  115320.03509521484
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 427.41it/s]

Batch_id:  0 Training loss:  78838.1171875
Batch_id:  1 Training loss:  321102.4375
Batch_id:  2 Training loss:  51390.83203125
Batch_id:  3 Training loss:  388062.09375
Batch_id:  4 Training loss:  69621.890625
Batch_id:  5 Training loss:  332141.21875
Batch_id:  6 Training loss:  104647.9140625
Batch_id:  7 Training loss:  61818.82421875
Batch_id:  8 Training loss:  95061.9375
Batch_id:  9 Training loss:  287983.0
Batch_id:  10 Training loss:  182667.046875
Batch_id:  11 Training loss:  160038.078125
Batch_id:  12 Training loss:  199557.578125
Batch_id:  13 Training loss:  115658.5390625
Batch_id:  14 Training loss:  60238.6484375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1765.28it/s]


Batch_id:  0 Eval loss:  103668.390625
Batch_id:  1 Eval loss:  200056.546875
Batch_id:  2 Eval loss:  31948.087890625
Batch_id:  3 Eval loss:  158625.5
Batch_id:  4 Eval loss:  102674.6875
Batch_id:  5 Eval loss:  30465.34765625
Batch_id:  6 Eval loss:  126388.1484375
Epoch  146 Training loss:  169515.41596283784
Epoch  146 Evaluation loss:  117785.4232788086
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  395176.59375
Batch_id:  1 Training loss:  159295.046875
Batch_id:  2 Training loss:  67602.4140625
Batch_id:  3 Training loss:  274288.40625
Batch_id:  4 Training loss:  204988.390625
Batch_id:  5 Training loss:  69817.84375
Batch_id:  6 Training loss:  137178.484375
Batch_id:  7 Training loss:  188151.40625
Batch_id:  8 Training loss:  236181.296875
Batch_id:  9 Training loss:  148691.359375
Batch_id:  10 Training loss:  97181.0390625
Batch_id:  11 Training loss:  146785.625
Batch_id:  12 Training loss:  110848.7734375


100%|██████████| 15/15 [00:00<00:00, 415.18it/s]


Batch_id:  13 Training loss:  104992.8984375
Batch_id:  14 Training loss:  194506.03125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1833.17it/s]


Batch_id:  0 Eval loss:  49157.43359375
Batch_id:  1 Eval loss:  79044.6640625
Batch_id:  2 Eval loss:  56028.15625
Batch_id:  3 Eval loss:  12722.1845703125
Batch_id:  4 Eval loss:  146425.546875
Batch_id:  5 Eval loss:  284033.59375
Batch_id:  6 Eval loss:  126456.765625
Epoch  147 Training loss:  171330.10874155405
Epoch  147 Evaluation loss:  117791.92886352539
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  196784.421875
Batch_id:  1 Training loss:  195443.078125
Batch_id:  2 Training loss:  174765.34375
Batch_id:  3 Training loss:  70227.7421875
Batch_id:  4 Training loss:  372627.71875
Batch_id:  5 Training loss:  232289.125
Batch_id:  6 Training loss:  36473.51171875
Batch_id:  7 Training loss:  177864.609375
Batch_id:  8 Training loss:  60205.1953125
Batch_id:  9 Training loss:  168611.15625
Batch_id:  10 Training loss:  121862.578125
Batch_id:  11 Training loss:  68070.9921875
Batch_id:  12 Training loss:  182193.90625
Batch_id:  13 Training loss:  270251.59375


100%|██████████| 15/15 [00:00<00:00, 412.63it/s]


Batch_id:  14 Training loss:  211382.484375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 983.79it/s]


Batch_id:  0 Eval loss:  84994.5546875
Batch_id:  1 Eval loss:  85600.5625
Batch_id:  2 Eval loss:  53623.71484375
Batch_id:  3 Eval loss:  101742.484375
Batch_id:  4 Eval loss:  221827.171875
Batch_id:  5 Eval loss:  120202.25
Batch_id:  6 Eval loss:  25008.203125
Epoch  148 Training loss:  171557.666015625
Epoch  148 Evaluation loss:  108281.08459472656
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 447.96it/s]

Batch_id:  0 Training loss:  245869.28125
Batch_id:  1 Training loss:  221418.484375
Batch_id:  2 Training loss:  323530.0
Batch_id:  3 Training loss:  86263.515625
Batch_id:  4 Training loss:  131466.921875
Batch_id:  5 Training loss:  203538.21875
Batch_id:  6 Training loss:  62245.01953125
Batch_id:  7 Training loss:  262416.9375
Batch_id:  8 Training loss:  84842.4609375
Batch_id:  9 Training loss:  108934.8125
Batch_id:  10 Training loss:  191827.484375
Batch_id:  11 Training loss:  158210.4375
Batch_id:  12 Training loss:  184226.0
Batch_id:  13 Training loss:  150020.25
Batch_id:  14 Training loss:  102454.6015625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1758.51it/s]


Batch_id:  0 Eval loss:  89725.9375
Batch_id:  1 Eval loss:  219962.625
Batch_id:  2 Eval loss:  125985.140625
Batch_id:  3 Eval loss:  139939.734375
Batch_id:  4 Eval loss:  43861.3515625
Batch_id:  5 Eval loss:  43336.44921875
Batch_id:  6 Eval loss:  37956.6875
Epoch  149 Training loss:  170085.4341744088
Epoch  149 Evaluation loss:  109494.98840332031
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 428.36it/s]

Batch_id:  0 Training loss:  130110.9296875
Batch_id:  1 Training loss:  71252.390625
Batch_id:  2 Training loss:  249103.109375
Batch_id:  3 Training loss:  196839.625
Batch_id:  4 Training loss:  91699.5859375
Batch_id:  5 Training loss:  265183.90625
Batch_id:  6 Training loss:  244505.75
Batch_id:  7 Training loss:  418903.75
Batch_id:  8 Training loss:  271415.28125
Batch_id:  9 Training loss:  61014.03125
Batch_id:  10 Training loss:  108554.7734375
Batch_id:  11 Training loss:  40238.51171875
Batch_id:  12 Training loss:  91266.6640625
Batch_id:  13 Training loss:  216334.78125
Batch_id:  14 Training loss:  50447.0390625
Eval in session: 



100%|██████████| 7/7 [00:00<00:00, 1399.90it/s]


Batch_id:  0 Eval loss:  96940.8046875
Batch_id:  1 Eval loss:  70259.765625
Batch_id:  2 Eval loss:  72169.9921875
Batch_id:  3 Eval loss:  78544.8515625
Batch_id:  4 Eval loss:  303589.34375
Batch_id:  5 Eval loss:  42468.23046875
Batch_id:  6 Eval loss:  35052.671875
Epoch  150 Training loss:  169383.11681798985
Epoch  150 Evaluation loss:  109222.75939941406
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 416.09it/s]

Batch_id:  0 Training loss:  48173.33984375
Batch_id:  1 Training loss:  166770.921875
Batch_id:  2 Training loss:  264722.09375
Batch_id:  3 Training loss:  318056.875
Batch_id:  4 Training loss:  92195.390625
Batch_id:  5 Training loss:  102362.9140625
Batch_id:  6 Training loss:  220481.453125
Batch_id:  7 Training loss:  48586.92578125
Batch_id:  8 Training loss:  165825.875
Batch_id:  9 Training loss:  107028.4296875
Batch_id:  10 Training loss:  136014.53125
Batch_id:  11 Training loss:  123056.6484375
Batch_id:  12 Training loss:  327973.34375
Batch_id:  13 Training loss:  354064.9375
Batch_id:  14 Training loss:  26832.70703125


Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  141719.484375


100%|██████████| 7/7 [00:00<00:00, 1747.94it/s]


Batch_id:  1 Eval loss:  80503.7890625
Batch_id:  2 Eval loss:  92433.5546875
Batch_id:  3 Eval loss:  122942.5390625
Batch_id:  4 Eval loss:  202929.171875
Batch_id:  5 Eval loss:  26354.84765625
Batch_id:  6 Eval loss:  27775.39453125
Epoch  151 Training loss:  169063.94504856417
Epoch  151 Evaluation loss:  108540.4345703125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 386.96it/s]

Batch_id:  0 Training loss:  281781.96875
Batch_id:  1 Training loss:  30733.326171875
Batch_id:  2 Training loss:  162342.796875
Batch_id:  3 Training loss:  90309.7734375
Batch_id:  4 Training loss:  122301.2421875
Batch_id:  5 Training loss:  61510.77734375
Batch_id:  6 Training loss:  203629.734375
Batch_id:  7 Training loss:  155553.421875
Batch_id:  8 Training loss:  96485.8828125
Batch_id:  9 Training loss:  222094.625
Batch_id:  10 Training loss:  170217.734375
Batch_id:  11 Training loss:  75758.8828125
Batch_id:  12 Training loss:  172137.875
Batch_id:  13 Training loss:  113649.953125
Batch_id:  14 Training loss:  672831.5


Eval in session: 


  0%|          | 0/7 [00:00<?, ?it/s]

Batch_id:  0 Eval loss:  222311.875


100%|██████████| 7/7 [00:00<00:00, 875.64it/s]


Batch_id:  1 Eval loss:  161118.0625
Batch_id:  2 Eval loss:  146760.546875
Batch_id:  3 Eval loss:  36410.4140625
Batch_id:  4 Eval loss:  49169.96484375
Batch_id:  5 Eval loss:  57514.80859375
Batch_id:  6 Eval loss:  11768.359375
Epoch  152 Training loss:  177793.20906355573
Epoch  152 Evaluation loss:  107039.6923828125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 362.20it/s]

Batch_id:  0 Training loss:  221339.453125
Batch_id:  1 Training loss:  325826.21875
Batch_id:  2 Training loss:  113047.7265625
Batch_id:  3 Training loss:  50784.33984375
Batch_id:  4 Training loss:  64346.65234375
Batch_id:  5 Training loss:  91224.1171875
Batch_id:  6 Training loss:  88687.0625
Batch_id:  7 Training loss:  76511.6875
Batch_id:  8 Training loss:  152829.75
Batch_id:  9 Training loss:  186213.359375
Batch_id:  10 Training loss:  266783.3125
Batch_id:  11 Training loss:  276021.875
Batch_id:  12 Training loss:  195597.6875
Batch_id:  13 Training loss:  358599.09375
Batch_id:  14 Training loss:  36206.96875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 989.52it/s]


Batch_id:  0 Eval loss:  50327.21875
Batch_id:  1 Eval loss:  40584.078125
Batch_id:  2 Eval loss:  406758.75
Batch_id:  3 Eval loss:  79734.734375
Batch_id:  4 Eval loss:  32667.0
Batch_id:  5 Eval loss:  48732.07421875
Batch_id:  6 Eval loss:  47972.703125
Epoch  153 Training loss:  169190.4935599662
Epoch  153 Evaluation loss:  110433.83728027344
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  40059.90625
Batch_id:  1 Training loss:  206472.59375
Batch_id:  2 Training loss:  139011.140625
Batch_id:  3 Training loss:  387518.8125
Batch_id:  4 Training loss:  79878.2890625
Batch_id:  5 Training loss:  88602.9609375
Batch_id:  6 Training loss:  44883.66015625
Batch_id:  7 Training loss:  75683.9921875
Batch_id:  8 Training loss:  99710.2578125


100%|██████████| 15/15 [00:00<00:00, 287.00it/s]


Batch_id:  9 Training loss:  111707.6171875
Batch_id:  10 Training loss:  210779.984375
Batch_id:  11 Training loss:  261193.921875
Batch_id:  12 Training loss:  235686.359375
Batch_id:  13 Training loss:  455190.46875
Batch_id:  14 Training loss:  75489.75
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1743.68it/s]


Batch_id:  0 Eval loss:  69424.9765625
Batch_id:  1 Eval loss:  62565.5390625
Batch_id:  2 Eval loss:  132476.546875
Batch_id:  3 Eval loss:  116136.8984375
Batch_id:  4 Eval loss:  61129.125
Batch_id:  5 Eval loss:  221304.25
Batch_id:  6 Eval loss:  37387.984375
Epoch  154 Training loss:  169720.92667863175
Epoch  154 Evaluation loss:  109441.45629882812
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  324547.5


100%|██████████| 15/15 [00:00<00:00, 427.61it/s]

Batch_id:  1 Training loss:  74214.6875
Batch_id:  2 Training loss:  266102.96875
Batch_id:  3 Training loss:  274029.59375
Batch_id:  4 Training loss:  117925.6875
Batch_id:  5 Training loss:  83728.5390625
Batch_id:  6 Training loss:  86153.7265625
Batch_id:  7 Training loss:  169769.09375
Batch_id:  8 Training loss:  107149.453125
Batch_id:  9 Training loss:  150293.859375
Batch_id:  10 Training loss:  105238.5546875
Batch_id:  11 Training loss:  396428.8125
Batch_id:  12 Training loss:  138234.390625
Batch_id:  13 Training loss:  133383.796875
Batch_id:  14 Training loss:  86963.078125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1741.10it/s]


Batch_id:  0 Eval loss:  44167.2265625
Batch_id:  1 Eval loss:  290343.3125
Batch_id:  2 Eval loss:  39203.3125
Batch_id:  3 Eval loss:  123518.5234375
Batch_id:  4 Eval loss:  92017.9296875
Batch_id:  5 Eval loss:  53263.76953125
Batch_id:  6 Eval loss:  88695.125
Epoch  155 Training loss:  169875.92852618243
Epoch  155 Evaluation loss:  114251.43737792969
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 406.76it/s]

Batch_id:  0 Training loss:  109306.4375
Batch_id:  1 Training loss:  180459.421875
Batch_id:  2 Training loss:  228178.109375
Batch_id:  3 Training loss:  122850.4921875
Batch_id:  4 Training loss:  123283.5546875
Batch_id:  5 Training loss:  619234.5
Batch_id:  6 Training loss:  103506.0390625
Batch_id:  7 Training loss:  168838.734375
Batch_id:  8 Training loss:  76252.5390625
Batch_id:  9 Training loss:  275256.3125
Batch_id:  10 Training loss:  257452.546875
Batch_id:  11 Training loss:  65720.1875
Batch_id:  12 Training loss:  52849.38671875
Batch_id:  13 Training loss:  81440.2734375
Batch_id:  14 Training loss:  40176.9765625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1747.94it/s]


Batch_id:  0 Eval loss:  210806.75
Batch_id:  1 Eval loss:  63332.23828125
Batch_id:  2 Eval loss:  79172.1171875
Batch_id:  3 Eval loss:  47092.91796875
Batch_id:  4 Eval loss:  117753.1484375
Batch_id:  5 Eval loss:  136337.9375
Batch_id:  6 Eval loss:  58743.015625
Epoch 00157: reducing learning rate of group 0 to 6.1035e-07.
Epoch  156 Training loss:  169243.61565667228
Epoch  156 Evaluation loss:  111443.45703125
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 454.80it/s]

Batch_id:  0 Training loss:  188733.515625
Batch_id:  1 Training loss:  65002.375
Batch_id:  2 Training loss:  106730.8671875
Batch_id:  3 Training loss:  512760.5
Batch_id:  4 Training loss:  123271.4140625
Batch_id:  5 Training loss:  69955.40625
Batch_id:  6 Training loss:  293907.4375
Batch_id:  7 Training loss:  123235.8671875
Batch_id:  8 Training loss:  152432.265625
Batch_id:  9 Training loss:  58066.41796875
Batch_id:  10 Training loss:  195010.265625
Batch_id:  11 Training loss:  258423.03125
Batch_id:  12 Training loss:  222424.609375
Batch_id:  13 Training loss:  69837.59375
Batch_id:  14 Training loss:  71218.53125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1743.16it/s]

Batch_id:  0 Eval loss:  72420.8515625
Batch_id:  1 Eval loss:  148344.078125
Batch_id:  2 Eval loss:  62793.01171875
Batch_id:  3 Eval loss:  142078.046875
Batch_id:  4 Eval loss:  209994.421875
Batch_id:  5 Eval loss:  15555.1201171875
Batch_id:  6 Eval loss:  67015.828125


Epoch  157 Training loss:  169662.8444362331
Epoch  157 Evaluation loss:  112218.96224975586
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 372.97it/s]

Batch_id:  0 Training loss:  98333.2734375
Batch_id:  1 Training loss:  94249.7734375
Batch_id:  2 Training loss:  163926.96875
Batch_id:  3 Training loss:  177807.484375
Batch_id:  4 Training loss:  125054.3125
Batch_id:  5 Training loss:  321587.4375
Batch_id:  6 Training loss:  247782.15625
Batch_id:  7 Training loss:  44646.07421875
Batch_id:  8 Training loss:  354927.21875
Batch_id:  9 Training loss:  80765.5390625
Batch_id:  10 Training loss:  82017.203125
Batch_id:  11 Training loss:  80739.2421875
Batch_id:  12 Training loss:  122776.953125
Batch_id:  13 Training loss:  332293.875
Batch_id:  14 Training loss:  212323.515625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1486.97it/s]

Batch_id:  0 Eval loss:  295618.21875
Batch_id:  1 Eval loss:  87419.6015625
Batch_id:  2 Eval loss:  139378.40625
Batch_id:  3 Eval loss:  44830.6328125
Batch_id:  4 Eval loss:  42576.66796875
Batch_id:  5 Eval loss:  53207.52734375
Batch_id:  6 Eval loss:  37401.21875


Epoch  158 Training loss:  171569.66400971284
Epoch  158 Evaluation loss:  109442.54272460938
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  101792.3046875
Batch_id:  1 Training loss:  230066.953125
Batch_id:  2 Training loss:  41611.70703125
Batch_id:  3 Training loss:  150919.328125
Batch_id:  4 Training loss:  52052.015625
Batch_id:  5 Training loss:  31043.95703125
Batch_id:  6 Training loss:  150520.015625
Batch_id:  7 Training loss:  404460.53125
Batch_id:  8 Training loss:  141117.359375
Batch_id:  9 Training loss:  366436.96875
Batch_id:  10 Training loss:  116256.375
Batch_id:  11 Training loss:  185592.875
Batch_id:  12 Training loss:  326806.09375
Batch_id:  13 Training loss:  78273.078125


100%|██████████| 15/15 [00:00<00:00, 441.31it/s]


Batch_id:  14 Training loss:  149770.5
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1755.67it/s]


Batch_id:  0 Eval loss:  25450.03515625
Batch_id:  1 Eval loss:  111805.6875
Batch_id:  2 Eval loss:  153620.828125
Batch_id:  3 Eval loss:  72437.9921875
Batch_id:  4 Eval loss:  66160.0625
Batch_id:  5 Eval loss:  233839.171875
Batch_id:  6 Eval loss:  36694.09375
Epoch  159 Training loss:  170724.32854729728
Epoch  159 Evaluation loss:  109376.22985839844
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  97284.953125


100%|██████████| 15/15 [00:00<00:00, 405.12it/s]


Batch_id:  1 Training loss:  350250.4375
Batch_id:  2 Training loss:  134596.28125
Batch_id:  3 Training loss:  85896.9921875
Batch_id:  4 Training loss:  158262.9375
Batch_id:  5 Training loss:  265962.53125
Batch_id:  6 Training loss:  102417.2578125
Batch_id:  7 Training loss:  34683.9921875
Batch_id:  8 Training loss:  130196.25
Batch_id:  9 Training loss:  185927.109375
Batch_id:  10 Training loss:  55133.13671875
Batch_id:  11 Training loss:  38127.94921875
Batch_id:  12 Training loss:  117619.75
Batch_id:  13 Training loss:  320805.9375
Batch_id:  14 Training loss:  524502.375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1166.06it/s]


Batch_id:  0 Eval loss:  61311.5
Batch_id:  1 Eval loss:  82907.3671875
Batch_id:  2 Eval loss:  203997.078125
Batch_id:  3 Eval loss:  167028.046875
Batch_id:  4 Eval loss:  106373.0
Batch_id:  5 Eval loss:  38622.85546875
Batch_id:  6 Eval loss:  44378.41015625
Epoch  160 Training loss:  175788.37098817568
Epoch  160 Evaluation loss:  110096.60278320312
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  320334.375
Batch_id:  1 Training loss:  143997.5
Batch_id:  2 Training loss:  186938.609375
Batch_id:  3 Training loss:  71643.5234375
Batch_id:  4 Training loss:  83404.25
Batch_id:  5 Training loss:  78798.2265625
Batch_id:  6 Training loss:  173465.859375
Batch_id:  7 Training loss:  252782.53125
Batch_id:  8 Training loss:  162818.953125
Batch_id:  9 Training loss:  291814.71875
Batch_id:  10 Training loss:  192396.359375
Batch_id:  11 Training loss:  288329.40625
Batch_id:  12 Training loss:  89920.1796875


100%|██████████| 15/15 [00:00<00:00, 215.05it/s]


Batch_id:  13 Training loss:  39020.15234375
Batch_id:  14 Training loss:  151375.4375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 368.04it/s]

Batch_id:  0 Eval loss:  42205.92578125
Batch_id:  1 Eval loss:  82668.25
Batch_id:  2 Eval loss:  245878.609375
Batch_id:  3 Eval loss:  195468.046875
Batch_id:  4 Eval loss:  36589.6171875
Batch_id:  5 Eval loss:  47956.49609375
Batch_id:  6 Eval loss:  68061.203125


Epoch  161 Training loss:  170745.95148859796
Epoch  161 Evaluation loss:  112316.89819335938
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  77094.765625


100%|██████████| 15/15 [00:00<00:00, 321.57it/s]

Batch_id:  1 Training loss:  302978.03125
Batch_id:  2 Training loss:  104142.4296875
Batch_id:  3 Training loss:  410819.34375
Batch_id:  4 Training loss:  269812.34375
Batch_id:  5 Training loss:  64579.8203125
Batch_id:  6 Training loss:  188220.25
Batch_id:  7 Training loss:  107135.7890625
Batch_id:  8 Training loss:  239201.5
Batch_id:  9 Training loss:  98394.984375
Batch_id:  10 Training loss:  26507.92578125
Batch_id:  11 Training loss:  61753.89453125
Batch_id:  12 Training loss:  145420.421875
Batch_id:  13 Training loss:  268246.71875
Batch_id:  14 Training loss:  165574.3125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1163.28it/s]


Batch_id:  0 Eval loss:  50150.43359375
Batch_id:  1 Eval loss:  213411.125
Batch_id:  2 Eval loss:  21179.99609375
Batch_id:  3 Eval loss:  121714.078125
Batch_id:  4 Eval loss:  133868.578125
Batch_id:  5 Eval loss:  100620.1328125
Batch_id:  6 Eval loss:  92617.5078125
Epoch  162 Training loss:  170938.00886824323
Epoch  162 Evaluation loss:  114619.03930664062
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 393.69it/s]

Batch_id:  0 Training loss:  391494.09375
Batch_id:  1 Training loss:  104574.8984375
Batch_id:  2 Training loss:  289098.46875
Batch_id:  3 Training loss:  67146.1640625
Batch_id:  4 Training loss:  50857.13671875
Batch_id:  5 Training loss:  62707.55078125
Batch_id:  6 Training loss:  360901.84375
Batch_id:  7 Training loss:  90486.2421875
Batch_id:  8 Training loss:  196373.171875
Batch_id:  9 Training loss:  303171.65625
Batch_id:  10 Training loss:  37296.37890625
Batch_id:  11 Training loss:  109612.6796875
Batch_id:  12 Training loss:  169215.046875
Batch_id:  13 Training loss:  67943.9296875
Batch_id:  14 Training loss:  244854.9375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1630.48it/s]


Batch_id:  0 Eval loss:  42990.34765625
Batch_id:  1 Eval loss:  157617.109375
Batch_id:  2 Eval loss:  45293.21484375
Batch_id:  3 Eval loss:  279162.5625
Batch_id:  4 Eval loss:  57017.625
Batch_id:  5 Eval loss:  57677.3125
Batch_id:  6 Eval loss:  95582.7890625
Epoch  163 Training loss:  172009.06751478038
Epoch  163 Evaluation loss:  114897.02514648438
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 386.38it/s]

Batch_id:  0 Training loss:  164276.859375
Batch_id:  1 Training loss:  133787.109375
Batch_id:  2 Training loss:  540526.625
Batch_id:  3 Training loss:  76835.0078125
Batch_id:  4 Training loss:  163952.5
Batch_id:  5 Training loss:  145113.359375
Batch_id:  6 Training loss:  41731.5703125
Batch_id:  7 Training loss:  263402.46875
Batch_id:  8 Training loss:  182789.40625
Batch_id:  9 Training loss:  123362.5234375
Batch_id:  10 Training loss:  48860.6953125
Batch_id:  11 Training loss:  51071.71875
Batch_id:  12 Training loss:  308731.125
Batch_id:  13 Training loss:  191146.421875
Batch_id:  14 Training loss:  76471.53125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1174.26it/s]


Batch_id:  0 Eval loss:  70198.453125
Batch_id:  1 Eval loss:  26286.92578125
Batch_id:  2 Eval loss:  150795.421875
Batch_id:  3 Eval loss:  270987.21875
Batch_id:  4 Eval loss:  81195.171875
Batch_id:  5 Eval loss:  67643.3125
Batch_id:  6 Eval loss:  27211.8671875
Epoch  164 Training loss:  169733.7109375
Epoch  164 Evaluation loss:  108487.24548339844
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 405.76it/s]

Batch_id:  0 Training loss:  205375.546875
Batch_id:  1 Training loss:  45074.96875
Batch_id:  2 Training loss:  41191.609375
Batch_id:  3 Training loss:  373252.34375
Batch_id:  4 Training loss:  83841.8671875
Batch_id:  5 Training loss:  88972.2421875
Batch_id:  6 Training loss:  210475.546875
Batch_id:  7 Training loss:  35170.234375
Batch_id:  8 Training loss:  208091.484375
Batch_id:  9 Training loss:  310678.09375
Batch_id:  10 Training loss:  248992.453125
Batch_id:  11 Training loss:  119343.75
Batch_id:  12 Training loss:  320702.21875
Batch_id:  13 Training loss:  178732.875
Batch_id:  14 Training loss:  33587.01953125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1166.10it/s]


Batch_id:  0 Eval loss:  245219.78125
Batch_id:  1 Eval loss:  47288.42578125
Batch_id:  2 Eval loss:  117677.1484375
Batch_id:  3 Eval loss:  182974.25
Batch_id:  4 Eval loss:  38285.49609375
Batch_id:  5 Eval loss:  27476.037109375
Batch_id:  6 Eval loss:  47674.20703125
Epoch  165 Training loss:  169154.2063450169
Epoch  165 Evaluation loss:  110405.52276611328
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 356.92it/s]

Batch_id:  0 Training loss:  261659.734375
Batch_id:  1 Training loss:  74471.6328125
Batch_id:  2 Training loss:  55050.32421875
Batch_id:  3 Training loss:  185984.734375
Batch_id:  4 Training loss:  98238.6640625
Batch_id:  5 Training loss:  277331.21875
Batch_id:  6 Training loss:  395220.4375
Batch_id:  7 Training loss:  63750.58984375
Batch_id:  8 Training loss:  216142.453125
Batch_id:  9 Training loss:  347567.28125
Batch_id:  10 Training loss:  163507.546875
Batch_id:  11 Training loss:  45514.9375
Batch_id:  12 Training loss:  82819.875
Batch_id:  13 Training loss:  143033.15625
Batch_id:  14 Training loss:  108090.671875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 699.43it/s]


Batch_id:  0 Eval loss:  117239.5390625
Batch_id:  1 Eval loss:  55627.9375
Batch_id:  2 Eval loss:  29962.201171875
Batch_id:  3 Eval loss:  245504.109375
Batch_id:  4 Eval loss:  131226.09375
Batch_id:  5 Eval loss:  71733.7734375
Batch_id:  6 Eval loss:  66742.140625
Epoch  166 Training loss:  170161.030933277
Epoch  166 Evaluation loss:  112193.09295654297
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  114771.5
Batch_id:  1 Training loss:  430817.84375
Batch_id:  2 Training loss:  55176.61328125
Batch_id:  3 Training loss:  131037.9765625
Batch_id:  4 Training loss:  71246.09375
Batch_id:  5 Training loss:  67351.265625
Batch_id:  6 Training loss:  264660.59375
Batch_id:  7 Training loss:  39697.6328125
Batch_id:  8 Training loss:  68645.2265625
Batch_id:  9 Training loss:  191331.984375
Batch_id:  10 Training loss:  336638.78125
Batch_id:  11 Training loss:  183380.703125
Batch_id:  12 Training loss:  335509.15625
Batch_id:  13 Training loss:  143024.640625


100%|██████████| 15/15 [00:00<00:00, 296.56it/s]


Batch_id:  14 Training loss:  79339.5859375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 932.69it/s]


Batch_id:  0 Eval loss:  61677.01953125
Batch_id:  1 Eval loss:  165276.015625
Batch_id:  2 Eval loss:  27926.5625
Batch_id:  3 Eval loss:  199845.921875
Batch_id:  4 Eval loss:  88865.2109375
Batch_id:  5 Eval loss:  104489.3359375
Batch_id:  6 Eval loss:  74775.6796875
Epoch 00168: reducing learning rate of group 0 to 3.0518e-07.
Epoch  167 Training loss:  169772.2701119088
Epoch  167 Evaluation loss:  112946.21032714844
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  452259.40625
Batch_id:  1 Training loss:  112662.7265625
Batch_id:  2 Training loss:  62504.19921875
Batch_id:  3 Training loss:  388317.5625
Batch_id:  4 Training loss:  243674.609375
Batch_id:  5 Training loss:  70720.9765625
Batch_id:  6 Training loss:  31865.607421875
Batch_id:  7 Training loss:  96399.3984375
Batch_id:  8 Training loss:  190194.671875
Batch_id:  9 Training loss:  57164.51953125
Batch_id:  10 Training loss:  151874.4375


100%|██████████| 15/15 [00:00<00:00, 340.84it/s]


Batch_id:  11 Training loss:  154798.296875
Batch_id:  12 Training loss:  93930.375
Batch_id:  13 Training loss:  144403.359375
Batch_id:  14 Training loss:  307490.53125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1747.52it/s]


Batch_id:  0 Eval loss:  223841.203125
Batch_id:  1 Eval loss:  135342.734375
Batch_id:  2 Eval loss:  83317.6171875
Batch_id:  3 Eval loss:  91438.6484375
Batch_id:  4 Eval loss:  71592.1875
Batch_id:  5 Eval loss:  54373.02734375
Batch_id:  6 Eval loss:  45212.265625
Epoch  168 Training loss:  172855.45119826857
Epoch  168 Evaluation loss:  110174.63806152344
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  32716.572265625
Batch_id:  1 Training loss:  345700.84375
Batch_id:  2 Training loss:  51710.6484375
Batch_id:  3 Training loss:  75694.9921875
Batch_id:  4 Training loss:  484259.15625
Batch_id:  5 Training loss:  69655.8125
Batch_id:  6 Training loss:  165701.734375
Batch_id:  7 Training loss:  86070.25
Batch_id:  8 Training loss:  205946.625
Batch_id:  9 Training loss:  236254.546875
Batch_id:  10 Training loss:  157460.03125
Batch_id:  11 Training loss:  317852.8125
Batch_id:  12 Training loss:  77505.2421875
Batch_id:  13 Training loss:  72109.4609375


100%|██████████| 15/15 [00:00<00:00, 451.64it/s]


Batch_id:  14 Training loss:  147651.375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1765.49it/s]


Batch_id:  0 Eval loss:  115294.765625
Batch_id:  1 Eval loss:  97632.5859375
Batch_id:  2 Eval loss:  51044.18359375
Batch_id:  3 Eval loss:  26655.17578125
Batch_id:  4 Eval loss:  296955.875
Batch_id:  5 Eval loss:  56777.52734375
Batch_id:  6 Eval loss:  84075.171875
Epoch  169 Training loss:  170695.27726456925
Epoch  169 Evaluation loss:  113818.01330566406
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 458.37it/s]

Batch_id:  0 Training loss:  129304.4296875
Batch_id:  1 Training loss:  242164.734375
Batch_id:  2 Training loss:  127135.75
Batch_id:  3 Training loss:  527957.8125
Batch_id:  4 Training loss:  224290.78125
Batch_id:  5 Training loss:  71096.375
Batch_id:  6 Training loss:  171063.0
Batch_id:  7 Training loss:  238132.484375
Batch_id:  8 Training loss:  69930.3046875
Batch_id:  9 Training loss:  188382.921875
Batch_id:  10 Training loss:  101816.328125
Batch_id:  11 Training loss:  276410.9375
Batch_id:  12 Training loss:  31888.306640625
Batch_id:  13 Training loss:  57757.27734375
Batch_id:  14 Training loss:  49284.8203125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1771.03it/s]


Batch_id:  0 Eval loss:  131660.5
Batch_id:  1 Eval loss:  238593.921875
Batch_id:  2 Eval loss:  19549.431640625
Batch_id:  3 Eval loss:  73452.9609375
Batch_id:  4 Eval loss:  121876.6875
Batch_id:  5 Eval loss:  68636.015625
Batch_id:  6 Eval loss:  60551.140625
Epoch  170 Training loss:  169365.96376161318
Epoch  170 Evaluation loss:  111612.60284423828
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 465.59it/s]


Batch_id:  0 Training loss:  439625.71875
Batch_id:  1 Training loss:  68741.2421875
Batch_id:  2 Training loss:  240713.0
Batch_id:  3 Training loss:  203154.484375
Batch_id:  4 Training loss:  114389.6015625
Batch_id:  5 Training loss:  76988.15625
Batch_id:  6 Training loss:  189246.375
Batch_id:  7 Training loss:  155448.359375
Batch_id:  8 Training loss:  54778.76953125
Batch_id:  9 Training loss:  330462.71875
Batch_id:  10 Training loss:  56668.13671875
Batch_id:  11 Training loss:  58917.20703125
Batch_id:  12 Training loss:  182771.34375
Batch_id:  13 Training loss:  188072.6875
Batch_id:  14 Training loss:  170979.5625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1764.86it/s]

Batch_id:  0 Eval loss:  288462.09375
Batch_id:  1 Eval loss:  60877.2265625
Batch_id:  2 Eval loss:  23294.693359375
Batch_id:  3 Eval loss:  39047.33984375
Batch_id:  4 Eval loss:  77840.875
Batch_id:  5 Eval loss:  168350.796875
Batch_id:  6 Eval loss:  50292.265625


Epoch  171 Training loss:  171010.6326541385
Epoch  171 Evaluation loss:  110650.8267211914
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 471.60it/s]

Batch_id:  0 Training loss:  236695.03125
Batch_id:  1 Training loss:  183191.3125
Batch_id:  2 Training loss:  52295.14453125
Batch_id:  3 Training loss:  58010.4765625
Batch_id:  4 Training loss:  189166.859375
Batch_id:  5 Training loss:  81239.7421875
Batch_id:  6 Training loss:  70444.265625
Batch_id:  7 Training loss:  79135.984375
Batch_id:  8 Training loss:  355144.53125
Batch_id:  9 Training loss:  73842.1796875
Batch_id:  10 Training loss:  103933.1484375
Batch_id:  11 Training loss:  254062.65625
Batch_id:  12 Training loss:  220001.453125
Batch_id:  13 Training loss:  239692.703125
Batch_id:  14 Training loss:  374880.34375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1186.17it/s]


Batch_id:  0 Eval loss:  117834.5234375
Batch_id:  1 Eval loss:  245424.328125
Batch_id:  2 Eval loss:  15287.40625
Batch_id:  3 Eval loss:  121758.375
Batch_id:  4 Eval loss:  44340.76171875
Batch_id:  5 Eval loss:  87532.03125
Batch_id:  6 Eval loss:  114530.796875
Epoch  172 Training loss:  173765.93459670607
Epoch  172 Evaluation loss:  116673.15979003906
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  136767.359375


100%|██████████| 15/15 [00:00<00:00, 394.12it/s]

Batch_id:  1 Training loss:  73111.7109375
Batch_id:  2 Training loss:  109849.2265625
Batch_id:  3 Training loss:  208357.40625
Batch_id:  4 Training loss:  244623.15625
Batch_id:  5 Training loss:  50923.49609375
Batch_id:  6 Training loss:  74588.203125
Batch_id:  7 Training loss:  361899.65625
Batch_id:  8 Training loss:  200017.625
Batch_id:  9 Training loss:  139988.109375
Batch_id:  10 Training loss:  217027.953125
Batch_id:  11 Training loss:  109113.25
Batch_id:  12 Training loss:  173213.265625
Batch_id:  13 Training loss:  344412.65625
Batch_id:  14 Training loss:  66081.78125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 970.42it/s]


Batch_id:  0 Eval loss:  67757.9375
Batch_id:  1 Eval loss:  54990.73828125
Batch_id:  2 Eval loss:  24634.5625
Batch_id:  3 Eval loss:  87734.390625
Batch_id:  4 Eval loss:  227409.40625
Batch_id:  5 Eval loss:  36859.2265625
Batch_id:  6 Eval loss:  446508.84375
Epoch  173 Training loss:  169592.89563978038
Epoch  173 Evaluation loss:  147796.1102294922
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  175153.3125
Batch_id:  1 Training loss:  53944.13671875
Batch_id:  2 Training loss:  122527.0546875
Batch_id:  3 Training loss:  154664.734375
Batch_id:  4 Training loss:  18930.607421875
Batch_id:  5 Training loss:  55178.90234375
Batch_id:  6 Training loss:  86721.28125
Batch_id:  7 Training loss:  233701.75
Batch_id:  8 Training loss:  448557.09375
Batch_id:  9 Training loss:  231549.5
Batch_id:  10 Training loss:  141903.265625
Batch_id:  11 Training loss:  231083.484375


100%|██████████| 15/15 [00:00<00:00, 338.34it/s]


Batch_id:  12 Training loss:  116196.078125
Batch_id:  13 Training loss:  253027.921875
Batch_id:  14 Training loss:  217027.703125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1005.21it/s]


Batch_id:  0 Eval loss:  150254.234375
Batch_id:  1 Eval loss:  55981.23828125
Batch_id:  2 Eval loss:  79570.953125
Batch_id:  3 Eval loss:  123035.8671875
Batch_id:  4 Eval loss:  70241.7890625
Batch_id:  5 Eval loss:  181552.640625
Batch_id:  6 Eval loss:  43382.1796875
Epoch  174 Training loss:  171632.89366026182
Epoch  174 Evaluation loss:  110002.95349121094
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 293.49it/s]

Batch_id:  0 Training loss:  80336.0390625
Batch_id:  1 Training loss:  50204.79296875
Batch_id:  2 Training loss:  232420.5
Batch_id:  3 Training loss:  296933.8125
Batch_id:  4 Training loss:  93949.1640625
Batch_id:  5 Training loss:  245601.125
Batch_id:  6 Training loss:  34629.2421875
Batch_id:  7 Training loss:  104577.3046875
Batch_id:  8 Training loss:  163977.4375
Batch_id:  9 Training loss:  241259.28125
Batch_id:  10 Training loss:  444591.3125
Batch_id:  11 Training loss:  77747.0546875
Batch_id:  12 Training loss:  48728.8203125
Batch_id:  13 Training loss:  127615.3359375
Batch_id:  14 Training loss:  317733.25


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1000.17it/s]


Batch_id:  0 Eval loss:  70352.7109375
Batch_id:  1 Eval loss:  72424.9296875
Batch_id:  2 Eval loss:  107727.1640625
Batch_id:  3 Eval loss:  134117.296875
Batch_id:  4 Eval loss:  50923.94140625
Batch_id:  5 Eval loss:  236785.40625
Batch_id:  6 Eval loss:  14145.30859375
Epoch  175 Training loss:  172993.54544974663
Epoch  175 Evaluation loss:  107261.99340820312
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  227728.328125
Batch_id:  1 Training loss:  130176.625
Batch_id:  2 Training loss:  155175.515625
Batch_id:  3 Training loss:  262168.4375
Batch_id:  4 Training loss:  275485.09375
Batch_id:  5 Training loss:  99172.0546875
Batch_id:  6 Training loss:  298495.34375
Batch_id:  7 Training loss:  54202.24609375
Batch_id:  8 Training loss:  106606.703125
Batch_id:  9 Training loss:  171943.203125
Batch_id:  10 Training loss:  169130.046875
Batch_id:  11 Training loss:  141350.609375
Batch_id:  12 Training loss:  99702.0
Batch_id:  13 Training loss:  187833.1875


100%|██████████| 15/15 [00:00<00:00, 306.31it/s]


Batch_id:  14 Training loss:  146986.953125
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1718.68it/s]

Batch_id:  0 Eval loss:  100944.1875
Batch_id:  1 Eval loss:  84593.9609375
Batch_id:  2 Eval loss:  39895.89453125
Batch_id:  3 Eval loss:  92232.984375
Batch_id:  4 Eval loss:  113316.4140625
Batch_id:  5 Eval loss:  205899.140625
Batch_id:  6 Eval loss:  102767.9765625


Epoch  176 Training loss:  170686.23970650337
Epoch  176 Evaluation loss:  115570.39978027344
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 441.09it/s]

Batch_id:  0 Training loss:  93873.03125
Batch_id:  1 Training loss:  292521.65625
Batch_id:  2 Training loss:  63921.6015625
Batch_id:  3 Training loss:  107528.8046875
Batch_id:  4 Training loss:  377822.84375
Batch_id:  5 Training loss:  76721.59375
Batch_id:  6 Training loss:  249733.671875
Batch_id:  7 Training loss:  75431.6640625
Batch_id:  8 Training loss:  101908.875
Batch_id:  9 Training loss:  149494.0
Batch_id:  10 Training loss:  148352.109375
Batch_id:  11 Training loss:  174850.859375
Batch_id:  12 Training loss:  139419.609375
Batch_id:  13 Training loss:  331244.21875
Batch_id:  14 Training loss:  142417.65625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1714.56it/s]


Batch_id:  0 Eval loss:  208243.890625
Batch_id:  1 Eval loss:  77933.2265625
Batch_id:  2 Eval loss:  146196.671875
Batch_id:  3 Eval loss:  114587.6875
Batch_id:  4 Eval loss:  68592.453125
Batch_id:  5 Eval loss:  33416.14453125
Batch_id:  6 Eval loss:  72548.75
Epoch  177 Training loss:  170624.47265625
Epoch  177 Evaluation loss:  112737.31628417969
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 477.46it/s]

Batch_id:  0 Training loss:  368279.125
Batch_id:  1 Training loss:  215506.578125
Batch_id:  2 Training loss:  210877.75
Batch_id:  3 Training loss:  43522.07421875
Batch_id:  4 Training loss:  86789.046875
Batch_id:  5 Training loss:  79913.8359375
Batch_id:  6 Training loss:  34720.71484375
Batch_id:  7 Training loss:  312966.46875
Batch_id:  8 Training loss:  189417.359375
Batch_id:  9 Training loss:  386377.84375
Batch_id:  10 Training loss:  213388.453125
Batch_id:  11 Training loss:  172097.1875
Batch_id:  12 Training loss:  112110.0
Batch_id:  13 Training loss:  38934.85546875
Batch_id:  14 Training loss:  39823.37890625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1730.94it/s]


Batch_id:  0 Eval loss:  203430.640625
Batch_id:  1 Eval loss:  42300.046875
Batch_id:  2 Eval loss:  32494.478515625
Batch_id:  3 Eval loss:  88899.765625
Batch_id:  4 Eval loss:  72673.078125
Batch_id:  5 Eval loss:  213274.40625
Batch_id:  6 Eval loss:  62292.7578125
Epoch 00179: reducing learning rate of group 0 to 1.5259e-07.
Epoch  178 Training loss:  169238.15350506757
Epoch  178 Evaluation loss:  111775.80841064453
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 428.21it/s]

Batch_id:  0 Training loss:  204580.359375
Batch_id:  1 Training loss:  44648.41796875
Batch_id:  2 Training loss:  84205.71875
Batch_id:  3 Training loss:  263380.46875
Batch_id:  4 Training loss:  73337.8984375
Batch_id:  5 Training loss:  471647.96875
Batch_id:  6 Training loss:  33699.90234375
Batch_id:  7 Training loss:  194456.765625
Batch_id:  8 Training loss:  164356.109375
Batch_id:  9 Training loss:  37902.62109375
Batch_id:  10 Training loss:  271955.25
Batch_id:  11 Training loss:  340673.03125
Batch_id:  12 Training loss:  47377.0625
Batch_id:  13 Training loss:  57393.83203125
Batch_id:  14 Training loss:  258927.09375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1746.90it/s]


Batch_id:  0 Eval loss:  39691.6953125
Batch_id:  1 Eval loss:  87289.3359375
Batch_id:  2 Eval loss:  41512.3125
Batch_id:  3 Eval loss:  137216.9375
Batch_id:  4 Eval loss:  75919.4296875
Batch_id:  5 Eval loss:  116222.0546875
Batch_id:  6 Eval loss:  450344.375
Epoch  179 Training loss:  172198.81756756754
Epoch  179 Evaluation loss:  148155.64697265625
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 427.75it/s]


Batch_id:  0 Training loss:  57515.09375
Batch_id:  1 Training loss:  117482.8671875
Batch_id:  2 Training loss:  303401.03125
Batch_id:  3 Training loss:  86423.0625
Batch_id:  4 Training loss:  63402.67578125
Batch_id:  5 Training loss:  65885.2890625
Batch_id:  6 Training loss:  202908.296875
Batch_id:  7 Training loss:  108992.0
Batch_id:  8 Training loss:  105765.5234375
Batch_id:  9 Training loss:  44458.90234375
Batch_id:  10 Training loss:  233908.40625
Batch_id:  11 Training loss:  147526.625
Batch_id:  12 Training loss:  187159.796875
Batch_id:  13 Training loss:  604332.5
Batch_id:  14 Training loss:  209493.265625
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1747.11it/s]


Batch_id:  0 Eval loss:  43680.34765625
Batch_id:  1 Eval loss:  193871.546875
Batch_id:  2 Eval loss:  202015.125
Batch_id:  3 Eval loss:  47828.984375
Batch_id:  4 Eval loss:  24393.388671875
Batch_id:  5 Eval loss:  67357.609375
Batch_id:  6 Eval loss:  247106.28125
Epoch  180 Training loss:  171530.76594172296
Epoch  180 Evaluation loss:  129102.07550048828
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  98749.4140625


100%|██████████| 15/15 [00:00<00:00, 376.59it/s]

Batch_id:  1 Training loss:  122466.4140625
Batch_id:  2 Training loss:  290013.09375
Batch_id:  3 Training loss:  214750.28125
Batch_id:  4 Training loss:  50522.17578125
Batch_id:  5 Training loss:  344618.125
Batch_id:  6 Training loss:  71572.90625
Batch_id:  7 Training loss:  255627.296875
Batch_id:  8 Training loss:  229166.625
Batch_id:  9 Training loss:  116012.9140625
Batch_id:  10 Training loss:  365585.3125
Batch_id:  11 Training loss:  71556.8671875
Batch_id:  12 Training loss:  26416.732421875
Batch_id:  13 Training loss:  67636.7734375
Batch_id:  14 Training loss:  215077.890625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1182.78it/s]


Batch_id:  0 Eval loss:  240606.25
Batch_id:  1 Eval loss:  104318.390625
Batch_id:  2 Eval loss:  110772.3515625
Batch_id:  3 Eval loss:  80327.3828125
Batch_id:  4 Eval loss:  100009.59375
Batch_id:  5 Eval loss:  27500.39453125
Batch_id:  6 Eval loss:  36137.828125
Epoch  181 Training loss:  171606.2717747044
Epoch  181 Evaluation loss:  109323.77990722656
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  43588.0078125
Batch_id:  1 Training loss:  255583.796875
Batch_id:  2 Training loss:  268782.84375
Batch_id:  3 Training loss:  303205.96875
Batch_id:  4 Training loss:  72692.4296875
Batch_id:  5 Training loss:  187766.875
Batch_id:  6 Training loss:  270651.3125


100%|██████████| 15/15 [00:00<00:00, 176.66it/s]

Batch_id:  7 Training loss:  103824.703125
Batch_id:  8 Training loss:  66546.7421875
Batch_id:  9 Training loss:  288729.34375
Batch_id:  10 Training loss:  295925.59375
Batch_id:  11 Training loss:  146922.078125
Batch_id:  12 Training loss:  49887.68359375
Batch_id:  13 Training loss:  33986.15234375
Batch_id:  14 Training loss:  135829.140625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1005.17it/s]

Batch_id:  0 Eval loss:  106912.1796875
Batch_id:  1 Eval loss:  266307.15625
Batch_id:  2 Eval loss:  25509.123046875
Batch_id:  3 Eval loss:  62867.21484375
Batch_id:  4 Eval loss:  122322.828125
Batch_id:  5 Eval loss:  58913.04296875
Batch_id:  6 Eval loss:  87894.953125


Epoch  182 Training loss:  170535.31566722973
Epoch  182 Evaluation loss:  114176.01531982422
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 360.76it/s]

Batch_id:  0 Training loss:  82215.9765625
Batch_id:  1 Training loss:  270783.09375
Batch_id:  2 Training loss:  166374.28125
Batch_id:  3 Training loss:  117139.8671875
Batch_id:  4 Training loss:  98189.1484375
Batch_id:  5 Training loss:  426427.21875
Batch_id:  6 Training loss:  72651.515625
Batch_id:  7 Training loss:  60177.2265625
Batch_id:  8 Training loss:  152122.234375
Batch_id:  9 Training loss:  131157.3125
Batch_id:  10 Training loss:  162361.984375
Batch_id:  11 Training loss:  72172.3515625
Batch_id:  12 Training loss:  79112.21875
Batch_id:  13 Training loss:  448961.59375
Batch_id:  14 Training loss:  196138.578125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1258.04it/s]


Batch_id:  0 Eval loss:  57929.1640625
Batch_id:  1 Eval loss:  210930.40625
Batch_id:  2 Eval loss:  114738.25
Batch_id:  3 Eval loss:  41708.0234375
Batch_id:  4 Eval loss:  46288.5078125
Batch_id:  5 Eval loss:  102477.109375
Batch_id:  6 Eval loss:  259794.953125
Epoch  183 Training loss:  171350.31091638512
Epoch  183 Evaluation loss:  130291.62719726562
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 441.64it/s]

Batch_id:  0 Training loss:  177718.078125
Batch_id:  1 Training loss:  260928.078125
Batch_id:  2 Training loss:  89071.1328125
Batch_id:  3 Training loss:  114947.4296875
Batch_id:  4 Training loss:  179551.015625
Batch_id:  5 Training loss:  53734.8828125
Batch_id:  6 Training loss:  147588.859375
Batch_id:  7 Training loss:  90603.0234375
Batch_id:  8 Training loss:  119627.8671875
Batch_id:  9 Training loss:  334623.5625
Batch_id:  10 Training loss:  40695.84375
Batch_id:  11 Training loss:  295759.9375
Batch_id:  12 Training loss:  327161.75
Batch_id:  13 Training loss:  139213.65625
Batch_id:  14 Training loss:  156915.8125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1768.90it/s]

Batch_id:  0 Eval loss:  123184.0546875
Batch_id:  1 Eval loss:  289512.15625
Batch_id:  2 Eval loss:  41037.02734375
Batch_id:  3 Eval loss:  86879.3046875
Batch_id:  4 Eval loss:  34724.77734375
Batch_id:  5 Eval loss:  63935.28125
Batch_id:  6 Eval loss:  96791.8203125


Epoch  184 Training loss:  170820.33308699323
Epoch  184 Evaluation loss:  115010.06591796875
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 424.14it/s]


Batch_id:  0 Training loss:  194341.6875
Batch_id:  1 Training loss:  192102.84375
Batch_id:  2 Training loss:  121847.8046875
Batch_id:  3 Training loss:  348872.46875
Batch_id:  4 Training loss:  255756.0
Batch_id:  5 Training loss:  115566.015625
Batch_id:  6 Training loss:  138113.46875
Batch_id:  7 Training loss:  172836.046875
Batch_id:  8 Training loss:  125594.2421875
Batch_id:  9 Training loss:  265966.34375
Batch_id:  10 Training loss:  76173.7109375
Batch_id:  11 Training loss:  96390.7265625
Batch_id:  12 Training loss:  256642.40625
Batch_id:  13 Training loss:  97170.5703125
Batch_id:  14 Training loss:  49231.27734375
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1398.70it/s]


Batch_id:  0 Eval loss:  262966.59375
Batch_id:  1 Eval loss:  62890.4140625
Batch_id:  2 Eval loss:  28617.458984375
Batch_id:  3 Eval loss:  60644.08203125
Batch_id:  4 Eval loss:  65792.890625
Batch_id:  5 Eval loss:  109466.828125
Batch_id:  6 Eval loss:  219028.0
Epoch  185 Training loss:  169365.244140625
Epoch  185 Evaluation loss:  126469.72930908203
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 389.39it/s]

Batch_id:  0 Training loss:  183830.546875
Batch_id:  1 Training loss:  68983.3671875
Batch_id:  2 Training loss:  204879.109375
Batch_id:  3 Training loss:  88434.5
Batch_id:  4 Training loss:  361355.5
Batch_id:  5 Training loss:  63508.85546875
Batch_id:  6 Training loss:  186701.890625
Batch_id:  7 Training loss:  78676.4765625
Batch_id:  8 Training loss:  174069.15625
Batch_id:  9 Training loss:  274754.09375
Batch_id:  10 Training loss:  77286.7734375
Batch_id:  11 Training loss:  259391.25
Batch_id:  12 Training loss:  90242.234375
Batch_id:  13 Training loss:  330541.59375
Batch_id:  14 Training loss:  67625.6796875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1714.96it/s]


Batch_id:  0 Eval loss:  133749.53125
Batch_id:  1 Eval loss:  87481.578125
Batch_id:  2 Eval loss:  64838.1953125
Batch_id:  3 Eval loss:  222691.25
Batch_id:  4 Eval loss:  115372.2734375
Batch_id:  5 Eval loss:  39569.7265625
Batch_id:  6 Eval loss:  35716.875
Epoch  186 Training loss:  169613.58292863175
Epoch  186 Evaluation loss:  109284.28588867188
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  141644.484375


100%|██████████| 15/15 [00:00<00:00, 424.82it/s]

Batch_id:  1 Training loss:  371140.40625
Batch_id:  2 Training loss:  97365.1328125
Batch_id:  3 Training loss:  49160.68359375
Batch_id:  4 Training loss:  128199.203125
Batch_id:  5 Training loss:  59331.375
Batch_id:  6 Training loss:  321200.96875
Batch_id:  7 Training loss:  193474.71875
Batch_id:  8 Training loss:  226883.609375
Batch_id:  9 Training loss:  49524.12890625
Batch_id:  10 Training loss:  167912.046875
Batch_id:  11 Training loss:  358400.34375
Batch_id:  12 Training loss:  90684.1015625
Batch_id:  13 Training loss:  169400.96875
Batch_id:  14 Training loss:  90542.28125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1750.23it/s]


Batch_id:  0 Eval loss:  78418.953125
Batch_id:  1 Eval loss:  106947.4609375
Batch_id:  2 Eval loss:  31802.552734375
Batch_id:  3 Eval loss:  74250.7890625
Batch_id:  4 Eval loss:  304703.375
Batch_id:  5 Eval loss:  74963.703125
Batch_id:  6 Eval loss:  17256.0
Epoch  187 Training loss:  169923.27385979728
Epoch  187 Evaluation loss:  107553.5678100586
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 443.32it/s]

Batch_id:  0 Training loss:  489287.3125
Batch_id:  1 Training loss:  217634.75
Batch_id:  2 Training loss:  199857.8125
Batch_id:  3 Training loss:  63580.15625
Batch_id:  4 Training loss:  48099.04296875
Batch_id:  5 Training loss:  190681.953125
Batch_id:  6 Training loss:  97849.4296875
Batch_id:  7 Training loss:  200387.828125
Batch_id:  8 Training loss:  85578.6015625
Batch_id:  9 Training loss:  101019.390625
Batch_id:  10 Training loss:  75825.4296875
Batch_id:  11 Training loss:  255362.828125
Batch_id:  12 Training loss:  127170.4765625
Batch_id:  13 Training loss:  77515.4296875
Batch_id:  14 Training loss:  333633.4375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1398.90it/s]


Batch_id:  0 Eval loss:  64838.13671875
Batch_id:  1 Eval loss:  111271.328125
Batch_id:  2 Eval loss:  53026.5390625
Batch_id:  3 Eval loss:  242488.75
Batch_id:  4 Eval loss:  113322.8046875
Batch_id:  5 Eval loss:  49711.93359375
Batch_id:  6 Eval loss:  108324.484375
Epoch  188 Training loss:  173208.37019636823
Epoch  188 Evaluation loss:  116091.24633789062
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 454.04it/s]

Batch_id:  0 Training loss:  303917.71875
Batch_id:  1 Training loss:  76009.328125
Batch_id:  2 Training loss:  299736.0625
Batch_id:  3 Training loss:  345304.21875
Batch_id:  4 Training loss:  311622.53125
Batch_id:  5 Training loss:  281168.96875
Batch_id:  6 Training loss:  139061.375
Batch_id:  7 Training loss:  97915.296875
Batch_id:  8 Training loss:  99683.25
Batch_id:  9 Training loss:  46909.37109375
Batch_id:  10 Training loss:  74024.6484375
Batch_id:  11 Training loss:  105390.828125
Batch_id:  12 Training loss:  120453.6015625
Batch_id:  13 Training loss:  45330.30859375
Batch_id:  14 Training loss:  187785.625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1332.61it/s]

Batch_id:  0 Eval loss:  233784.0
Batch_id:  1 Eval loss:  118965.765625
Batch_id:  2 Eval loss:  86791.203125
Batch_id:  3 Eval loss:  35548.2890625
Batch_id:  4 Eval loss:  34919.04296875
Batch_id:  5 Eval loss:  65971.7265625
Batch_id:  6 Eval loss:  255022.984375


Epoch 00190: reducing learning rate of group 0 to 7.6294e-08.
Epoch  189 Training loss:  171237.37383868243
Epoch  189 Evaluation loss:  129844.22058105469
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 393.81it/s]

Batch_id:  0 Training loss:  84068.4609375
Batch_id:  1 Training loss:  85570.734375
Batch_id:  2 Training loss:  49170.04296875
Batch_id:  3 Training loss:  111243.2421875
Batch_id:  4 Training loss:  206098.484375
Batch_id:  5 Training loss:  107104.4765625
Batch_id:  6 Training loss:  406221.90625
Batch_id:  7 Training loss:  39640.8203125
Batch_id:  8 Training loss:  188121.734375
Batch_id:  9 Training loss:  42387.0859375
Batch_id:  10 Training loss:  168470.53125
Batch_id:  11 Training loss:  344441.75
Batch_id:  12 Training loss:  84860.28125
Batch_id:  13 Training loss:  512195.59375
Batch_id:  14 Training loss:  83950.578125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 999.49it/s]

Batch_id:  0 Eval loss:  39890.54296875
Batch_id:  1 Eval loss:  35796.46484375
Batch_id:  2 Eval loss:  33420.96875
Batch_id:  3 Eval loss:  109333.1875
Batch_id:  4 Eval loss:  93358.125
Batch_id:  5 Eval loss:  257676.828125
Batch_id:  6 Eval loss:  271282.625


Epoch  190 Training loss:  169834.17044974663
Epoch  190 Evaluation loss:  131368.55346679688
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  72643.4375
Batch_id:  1 Training loss:  133411.515625
Batch_id:  2 Training loss:  68811.890625
Batch_id:  3 Training loss:  154356.0625
Batch_id:  4 Training loss:  126466.2265625
Batch_id:  5 Training loss:  87749.0859375
Batch_id:  6 Training loss:  273313.3125
Batch_id:  7 Training loss:  105183.125
Batch_id:  8 Training loss:  90739.2421875
Batch_id:  9 Training loss:  265691.15625
Batch_id:  10 Training loss:  232587.75
Batch_id:  11 Training loss:  303505.84375
Batch_id:  12 Training loss:  66494.2421875


100%|██████████| 15/15 [00:00<00:00, 349.00it/s]

Batch_id:  13 Training loss:  432623.5
Batch_id:  14 Training loss:  103974.3046875


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1173.79it/s]


Batch_id:  0 Eval loss:  151458.984375
Batch_id:  1 Eval loss:  38662.08984375
Batch_id:  2 Eval loss:  41281.51171875
Batch_id:  3 Eval loss:  268051.90625
Batch_id:  4 Eval loss:  47926.82421875
Batch_id:  5 Eval loss:  126448.2109375
Batch_id:  6 Eval loss:  10398.96875
Epoch  191 Training loss:  170104.77671030405
Epoch  191 Evaluation loss:  106910.70251464844
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 323.58it/s]

Batch_id:  0 Training loss:  68199.4609375
Batch_id:  1 Training loss:  70074.0
Batch_id:  2 Training loss:  64766.375
Batch_id:  3 Training loss:  77055.9375
Batch_id:  4 Training loss:  165243.15625
Batch_id:  5 Training loss:  494178.5
Batch_id:  6 Training loss:  94655.9140625
Batch_id:  7 Training loss:  166507.984375
Batch_id:  8 Training loss:  169495.015625
Batch_id:  9 Training loss:  287195.34375
Batch_id:  10 Training loss:  298736.34375
Batch_id:  11 Training loss:  306389.40625
Batch_id:  12 Training loss:  76504.6484375
Batch_id:  13 Training loss:  102600.5703125
Batch_id:  14 Training loss:  68941.390625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1161.90it/s]


Batch_id:  0 Eval loss:  116400.6171875
Batch_id:  1 Eval loss:  146190.734375
Batch_id:  2 Eval loss:  34502.4921875
Batch_id:  3 Eval loss:  42125.765625
Batch_id:  4 Eval loss:  101993.9296875
Batch_id:  5 Eval loss:  206588.90625
Batch_id:  6 Eval loss:  75466.6953125
Epoch  192 Training loss:  169631.35451858107
Epoch  192 Evaluation loss:  113010.80322265625
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 340.61it/s]

Batch_id:  0 Training loss:  77948.4140625
Batch_id:  1 Training loss:  484938.09375
Batch_id:  2 Training loss:  151686.1875
Batch_id:  3 Training loss:  57251.53125
Batch_id:  4 Training loss:  106358.9296875
Batch_id:  5 Training loss:  50362.79296875
Batch_id:  6 Training loss:  259595.953125
Batch_id:  7 Training loss:  55514.71875
Batch_id:  8 Training loss:  61470.125
Batch_id:  9 Training loss:  329285.96875
Batch_id:  10 Training loss:  52098.71484375
Batch_id:  11 Training loss:  184659.265625
Batch_id:  12 Training loss:  116901.2890625
Batch_id:  13 Training loss:  260630.359375
Batch_id:  14 Training loss:  310066.3125


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1174.03it/s]


Batch_id:  0 Eval loss:  29697.083984375
Batch_id:  1 Eval loss:  44217.6171875
Batch_id:  2 Eval loss:  49876.4765625
Batch_id:  3 Eval loss:  135913.109375
Batch_id:  4 Eval loss:  151976.296875
Batch_id:  5 Eval loss:  242322.546875
Batch_id:  6 Eval loss:  59964.86328125
Epoch  193 Training loss:  172889.77407094595
Epoch  193 Evaluation loss:  111557.49908447266
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  120047.1796875


100%|██████████| 15/15 [00:00<00:00, 404.95it/s]

Batch_id:  1 Training loss:  166226.0
Batch_id:  2 Training loss:  302123.34375
Batch_id:  3 Training loss:  55483.3203125
Batch_id:  4 Training loss:  119276.953125
Batch_id:  5 Training loss:  52410.4765625
Batch_id:  6 Training loss:  165591.25
Batch_id:  7 Training loss:  294973.3125
Batch_id:  8 Training loss:  207821.421875
Batch_id:  9 Training loss:  66202.1015625
Batch_id:  10 Training loss:  304641.96875
Batch_id:  11 Training loss:  160011.15625
Batch_id:  12 Training loss:  251702.171875
Batch_id:  13 Training loss:  124397.4921875
Batch_id:  14 Training loss:  132309.265625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1706.39it/s]


Batch_id:  0 Eval loss:  45198.55078125
Batch_id:  1 Eval loss:  145380.09375
Batch_id:  2 Eval loss:  225887.859375
Batch_id:  3 Eval loss:  83029.5
Batch_id:  4 Eval loss:  126121.8984375
Batch_id:  5 Eval loss:  45992.46875
Batch_id:  6 Eval loss:  15946.759765625
Epoch  194 Training loss:  170487.66311233107
Epoch  194 Evaluation loss:  107430.80169677734
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  105359.4921875


100%|██████████| 15/15 [00:00<00:00, 427.44it/s]

Batch_id:  1 Training loss:  67760.265625
Batch_id:  2 Training loss:  88176.7109375
Batch_id:  3 Training loss:  74003.5859375
Batch_id:  4 Training loss:  287655.96875
Batch_id:  5 Training loss:  364767.0625
Batch_id:  6 Training loss:  81622.125
Batch_id:  7 Training loss:  140652.828125
Batch_id:  8 Training loss:  457395.65625
Batch_id:  9 Training loss:  137494.046875
Batch_id:  10 Training loss:  184832.421875
Batch_id:  11 Training loss:  270829.75
Batch_id:  12 Training loss:  52678.70703125
Batch_id:  13 Training loss:  62717.01171875
Batch_id:  14 Training loss:  151012.40625


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1746.90it/s]


Batch_id:  0 Eval loss:  25091.384765625
Batch_id:  1 Eval loss:  237287.15625
Batch_id:  2 Eval loss:  40739.18359375
Batch_id:  3 Eval loss:  62663.5
Batch_id:  4 Eval loss:  191493.28125
Batch_id:  5 Eval loss:  105298.203125
Batch_id:  6 Eval loss:  38541.06640625
Epoch  195 Training loss:  170740.4080447635
Epoch  195 Evaluation loss:  109549.02740478516
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 413.70it/s]

Batch_id:  0 Training loss:  312848.59375
Batch_id:  1 Training loss:  126241.4921875
Batch_id:  2 Training loss:  152186.53125
Batch_id:  3 Training loss:  171788.46875
Batch_id:  4 Training loss:  151170.546875
Batch_id:  5 Training loss:  84559.1171875
Batch_id:  6 Training loss:  41341.62109375
Batch_id:  7 Training loss:  111035.1171875
Batch_id:  8 Training loss:  222019.078125
Batch_id:  9 Training loss:  209617.078125
Batch_id:  10 Training loss:  167013.40625
Batch_id:  11 Training loss:  372501.65625
Batch_id:  12 Training loss:  142300.640625
Batch_id:  13 Training loss:  177376.46875
Batch_id:  14 Training loss:  68444.4609375


Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1389.76it/s]


Batch_id:  0 Eval loss:  56048.9375
Batch_id:  1 Eval loss:  292209.8125
Batch_id:  2 Eval loss:  142835.421875
Batch_id:  3 Eval loss:  40644.9453125
Batch_id:  4 Eval loss:  54369.51171875
Batch_id:  5 Eval loss:  60941.15625
Batch_id:  6 Eval loss:  77348.3203125
Epoch  196 Training loss:  169624.61333403716
Epoch  196 Evaluation loss:  113187.20397949219
Training in session: 


100%|██████████| 15/15 [00:00<00:00, 442.93it/s]


Batch_id:  0 Training loss:  144254.796875
Batch_id:  1 Training loss:  70142.09375
Batch_id:  2 Training loss:  141662.515625
Batch_id:  3 Training loss:  472234.21875
Batch_id:  4 Training loss:  69951.8125
Batch_id:  5 Training loss:  184087.6875
Batch_id:  6 Training loss:  421960.40625
Batch_id:  7 Training loss:  205018.03125
Batch_id:  8 Training loss:  45831.55859375
Batch_id:  9 Training loss:  114702.1796875
Batch_id:  10 Training loss:  141088.125
Batch_id:  11 Training loss:  314910.4375
Batch_id:  12 Training loss:  65160.6015625
Batch_id:  13 Training loss:  72514.5859375
Batch_id:  14 Training loss:  41545.32421875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 1750.54it/s]

Batch_id:  0 Eval loss:  230443.328125
Batch_id:  1 Eval loss:  77147.3359375
Batch_id:  2 Eval loss:  15838.486328125
Batch_id:  3 Eval loss:  126077.3515625
Batch_id:  4 Eval loss:  81064.71875
Batch_id:  5 Eval loss:  130051.7421875
Batch_id:  6 Eval loss:  43415.3671875


Epoch  197 Training loss:  169261.1064189189
Epoch  197 Evaluation loss:  110005.98907470703
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  168804.296875
Batch_id:  1 Training loss:  37030.9140625
Batch_id:  2 Training loss:  149257.515625
Batch_id:  3 Training loss:  60210.15234375
Batch_id:  4 Training loss:  264766.6875
Batch_id:  5 Training loss:  51636.01171875
Batch_id:  6 Training loss:  408724.9375
Batch_id:  7 Training loss:  239766.359375
Batch_id:  8 Training loss:  71585.1171875
Batch_id:  9 Training loss:  212133.78125
Batch_id:  10 Training loss:  154015.296875
Batch_id:  11 Training loss:  74208.921875
Batch_id:  12 Training loss:  64652.15234375
Batch_id:  13 Training loss:  377573.84375


100%|██████████| 15/15 [00:00<00:00, 415.57it/s]


Batch_id:  14 Training loss:  202986.46875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 924.61it/s]

Batch_id:  0 Eval loss:  164059.953125
Batch_id:  1 Eval loss:  66283.2421875
Batch_id:  2 Eval loss:  32583.869140625
Batch_id:  3 Eval loss:  25321.2578125
Batch_id:  4 Eval loss:  94228.3046875
Batch_id:  5 Eval loss:  268342.34375
Batch_id:  6 Eval loss:  67925.140625


Epoch  198 Training loss:  171442.73358319257
Epoch  198 Evaluation loss:  112303.76739501953
Training in session: 


  0%|          | 0/15 [00:00<?, ?it/s]

Batch_id:  0 Training loss:  174811.421875
Batch_id:  1 Training loss:  62319.1640625
Batch_id:  2 Training loss:  263232.3125
Batch_id:  3 Training loss:  137753.03125
Batch_id:  4 Training loss:  171454.375
Batch_id:  5 Training loss:  246912.453125
Batch_id:  6 Training loss:  310782.5
Batch_id:  7 Training loss:  59893.80078125
Batch_id:  8 Training loss:  280804.28125
Batch_id:  9 Training loss:  111812.8984375
Batch_id:  10 Training loss:  90890.2109375
Batch_id:  11 Training loss:  290819.34375
Batch_id:  12 Training loss:  123294.1171875


100%|██████████| 15/15 [00:00<00:00, 394.61it/s]


Batch_id:  13 Training loss:  67137.3125
Batch_id:  14 Training loss:  131047.4921875
Eval in session: 


100%|██████████| 7/7 [00:00<00:00, 2348.43it/s]


Batch_id:  0 Eval loss:  74846.5234375
Batch_id:  1 Eval loss:  74210.1015625
Batch_id:  2 Eval loss:  126355.9296875
Batch_id:  3 Eval loss:  293430.21875
Batch_id:  4 Eval loss:  57052.10546875
Batch_id:  5 Eval loss:  34556.3671875
Batch_id:  6 Eval loss:  43844.609375
Epoch  199 Training loss:  170470.5888407939
Epoch  199 Evaluation loss:  110046.22741699219


Best epoch:  141  and loss value is  106320.26126861572


In [48]:
main(train)